In [ ]:
#check recent 2months per of the boom_time and compare check when lower(test)
#deficit to surplus means it will be profitable and check if this continues it will be better and sell when they anounce how the earned
#compare with last year if double it worths
import pandas as pd
import OpenDartReader
# ==== 0. 객체 생성 ====
# 객체 생성 (API KEY 지정) 
api_key = '84acf17c852e9430843a589191330e0630cc19a7'

dart = OpenDartReader(api_key) 
ANALY_KOSDAQ = pd.read_csv('ANALY_KOSDAQ(5).csv')
#buy on the time boom and sell when earn reveals and see how much it worths
for i in range(ANALY_KOSDAQ.index.size):
    stock_symbol = 

In [ ]:



#test




import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import datetime as dt 
from datetime import date
import time
from dateutil.relativedelta import relativedelta
import math
import warnings
import OpenDartReader
from pandas.core.common import SettingWithCopyWarning
import re
from collections.abc import Mapping

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)



class how_per():
    def __init__(self, api_key):
        self.api_key = api_key
        self.OLD_COUNT = 0
        self.OLD_SYMBOL = '가나다'
        self.SYMBOL = '가나다'
        self.TODAY = dt.date.today()
        self.BOOM_PER_LIST = []
        self.dart = OpenDartReader(self.api_key)
        
    def outlier_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q4 = np.percentile(datacolumn , 100)
        except:
            return True
        return Q4
        
    def has_no_fix(self,word):
        if '[기재정정]' in word:
            return True
        else:
            return False

    def get_want_fast_report(self, a_s_list):
        a_s_list = a_s_list[a_s_list['report_nm'].str.contains('연결재무제표기준영업', regex = False)]
        #a_s_list['rcept_dt'] = pd.to_datetime(a_s_list['rcept_dt'], format = '%Y%m%d').dt.date
        #a_s_list = a_s_list[a_s_list['rcept_dt']]
        #a_s_list.drop(['Unnamed: 0'], axis =1, inplace = True)
        a_s_list.reset_index(drop = True, inplace = True)
        return a_s_list

    def get_before_report_time(self, report_year, report_month):
        before_report_year = report_year-1
        before_report_month = 12
        return before_report_year, before_report_month

    def get_earn_fast_earn(self, fast_report_list):
        if not self.check_lenz(fast_report_list) :
            try:
                REPORT_NO = str(fast_report_list.loc[0, 'rcept_no'])
            except:
                return 0
            html_text = self.dart.document(REPORT_NO)
            DF_LIST = pd.read_html(html_text)
            for df_list in DF_LIST:
                if df_list[0].str.contains('※ 동 정보는 확정치가 아닌 잠정치로서 향후 확정치와는 다를 수 있음.').any():
                    df_list.set_index([0,1], inplace = True)
                    #if df_list[2].str.contains('당기이익').any():
                    AFTER_EARN = df_list.loc[('당기순이익','당해실적'), 2].item()
                    if AFTER_EARN == '-':
                        return 0
                    return int(AFTER_EARN)*(10**6)
        return 0
    
    def check_price_data(self, report_date):
        report_stock_date = date(2000,1,1)
        before_report_date = report_date - relativedelta(days = 14)
        now_report_date = report_date + relativedelta(days = 1)
        before_report_date = before_report_date.strftime("%Y-%m-%d")
        now_report_date = now_report_date.strftime("%Y-%m-%d")
        report_stock_price = 0
        df_stock_price = fdr.DataReader(self.SYMBOL, before_report_date, now_report_date)
        
        for i in range(0,14):
            report_stock_date = report_date - relativedelta(days = i)
            report_stock_date = report_stock_date.strftime("%Y-%m-%d")
            try:
                report_stock_price = df_stock_price.loc[report_stock_date, 'Close']
            except:
                continue
            if report_stock_price:
                break
            else:
                continue
        return int(report_stock_price)
    
    def get_top_last_year(self, last_year):
        now_last_year = str(last_year + 1) + '-03-30'
        before_last_year = str(last_year) + '-03-30'
        df_stock_price = fdr.DataReader(self.SYMBOL, before_last_year, now_last_year)
        price_column = df_stock_price[['Close']]
        top_price_last_year = self.outlier_treatment(price_column)
        return int(top_price_last_year)
        
    
    def give_earn_data(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        self.report.set_index(['sj_div','account_id'], inplace = True)
        #'ifrs-full_ProfitLossAttributableToOwnersOfParent'
        'ifrs-full_ProfitLossFromContinuingOperations'
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if self.report.index.isin([('CIS', 'ifrs-full_BasicEarningsLossPerShareFromContinuingOperations')]).any():
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('CIS','ifrs-full_BasicEarningsLossPerShareFromContinuingOperations'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn), int(quarter_earn_per_stock)
            report = self.report[self.report['account_nm'].str.contains('계속')==True]
            report = report[report['account_nm'].str.contains('기본주당')==True]
            report.reset_index(drop = True, inplace = True)
            if not self.check_empty(report):
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                quarter_earn_per_stock = report.loc[0, 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
            if self.report.index.isin([('CIS', 'ifrs-full_BasicEarningsLossPerShare')]).any():
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('CIS','ifrs-full_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if self.report.index.isin([('CIS', 'ifrs-full_BasicEarningsLossPerShare')]).any():
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('CIS','ifrs-full_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if self.report.index.isin([('CIS', 'ifrs_BasicEarningsLossPerShare')]).any():
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('CIS','ifrs_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if self.report.index.isin([('IS', 'ifrs-full_BasicEarningsLossPerShareFromContinuingOperations')]).any():
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('IS','ifrs-full_BasicEarningsLossPerShareFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
            if self.report.index.isin([('IS', 'ifrs-full_BasicEarningsLossPerShare')]).any():
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('IS','ifrs-full_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if self.report.index.isin([('IS', 'ifrs-full_BasicEarningsLossPerShare')]).any():
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('IS','ifrs-full_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if self.report.index.isin([('IS', 'ifrs_BasicEarningsLossPerShare')]).any():
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('IS','ifrs_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
        else:
            return
        
    def check_lenz(self, df):
        return df is None
    
    def check_empty(self, df):
        return df.empty
        
    def get_fs_of_recent(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL , str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        if report_month == 6:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        if report_month == 9:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        if report_month == 12:
            #if True:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        else:
            return 
        
    def get_fs_of_time(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data() 
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data() 
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        if report_month == 6:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        if report_month == 9:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                #self.report.to_csv('kg.csv')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                                if not self.check_lenz(self.report):
                                    if not self.check_empty(self.report):
                                        PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                        QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                        QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + PLUS_EARN_PER_STOCK
                                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                                if not self.check_lenz(self.report):
                                    if not self.check_empty(self.report):
                                        PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                        QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                        QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        if report_month == 12:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        else:
            return
    
    def get_real_report_month(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year
            report_month = 6
        if report_month == 6:
            report_year = report_year
            report_month = 9
        if report_month == 9:
            report_year = report_year
            report_month = 12
        if report_month == 12:
            report_year = report_year + 1
            report_month = 3
        return report_year, report_month
    
    def get_predict_earn_per_stock(self, report_month, earn_per_stock):
        earn_per_stock = int(earn_per_stock)
        
        if report_month == 3:
            earn_per_stock = earn_per_stock*4
        if report_month == 6:
            earn_per_stock = earn_per_stock*2
        if report_month == 9:
            earn_per_stock = earn_per_stock*(4/3)
        if report_month == 12:
            earn_per_stock = earn_per_stock
        
        return earn_per_stock
    
    def get_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_last_year_report(self, report_date):
        delimiters = "-"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_date)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        last_year = int(nm_num_list[0])-1
        return last_year
    
    def get_report_year_month(self, report_nm):
        delimiters = "(", ".", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_one_back(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year - 1
            report_month = 12
        if report_month == 6:
            report_year = report_year
            report_month = 3
        if report_month == 9:
            report_year = report_year
            report_month = 6
        if report_month == 12:
            report_year = report_year
            report_month = 9
        return report_year, report_month
            
    def get_symbol_six(self, analy_kosdaq, i):
        new_symbol = analy_kosdaq.loc[i, 'Symbol']
        if new_symbol != self.OLD_SYMBOL:
            self.OLD_COUNT = 0
            symbol = analy_kosdaq.loc[i, 'Symbol']
            symbol_length = self.digit_length(symbol)
            if symbol_length != 6:
                zero_need = 6 - symbol_length
                self.SYMBOL = '0'*zero_need + str(symbol)
            if symbol_length == 6:
                self.SYMBOL = str(symbol)
            self.OLD_SYMBOL = new_symbol
            return
        if new_symbol == self.OLD_SYMBOL:
            self.OLD_COUNT += 1
        return 
    
    #자릿수 찾기
    def digit_length(self, n):
        ans = 0
        n= int(n)

        while n:
            n //= 10
            ans += 1

        return ans


    def how_report(self, analyse_date, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(0,500):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            A_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, final = False)
            FAST_REPORT_LIST = self.get_want_fast_report(A_S_LIST)
            FAST_REPORT_NM, FAST_REPORT_DT = self.get_report_date(FAST_REPORT_LIST)
            if not self.check_empty(FAST_REPORT_LIST):
                #check if report is between real_report
                if REPORT_DT - relativedelta(months = 1) < FAST_REPORT_DT and REPORT_DT > FAST_REPORT_DT:
                    #if True:
                    try:
                        AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    except:
                        pass
                    try:
                        if BACK_REPORT_MONTH != 12:
                            BOOM_QUARTER_EARN, BOOM_QUARTER_EARN_PER_STOCK = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                            a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    except:
                        pass

                elif REPORT_DT + relativedelta(months = 4) > FAST_REPORT_DT and REPORT_DT < FAST_REPORT_DT:
                    #if True:
                    try:
                        AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        REPORT_YEAR, REPORT_MONTH = self.get_real_report_month(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    except:
                        pass
                    try:
                        if REPORT_MONTH != 12:
                            BOOM_QUARTER_EARN, BOOM_QUARTER_EARN_PER_STOCK = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                            a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    except:
                        pass
            else:
                #if True:
                try:
                    BOOM_QUARTER_EARN, BOOM_QUARTER_EARN_PER_STOCK = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                    a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                    a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                    a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                    time.sleep(1)
                except:
                    pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            try:
                RECENT_QUARTER_EARN,RECENT_QUARTER_EARN_PER_STOCK= self.get_fs_of_recent(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['RECENT_QUARTER_EARN'] = RECENT_QUARTER_EARN
                a_dictionary['RECENT_QUARTER_EARN_PER_STOCK'] = RECENT_QUARTER_EARN_PER_STOCK
                time.sleep(1)
            except:
                pass
            #GET OLD REPORT EARN TO COMPARE
            try:
                OLD_EARN, OLD_EARN_PER_STOCK = self.get_fs_of_recent(OLD_REPORT_YEAR, OLD_REPORT_MONTH)
                a_dictionary['OLD_EARN'] = OLD_EARN
                a_dictionary['OLD_EARN_PER_STOCK'] = OLD_EARN_PER_STOCK
                time.sleep(1)
            except:
                pass
            if True:
            #try:
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = a_dictionary['QUARTER_EARN'] + a_dictionary['AFTER_EARN']
            #except:
                pass
            #try:
                if 'RECENT_QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'RECENT_QUARTER_EARN' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = int(a_dictionary['QUARTER_EARN']*(a_dictionary['RECENT_QUARTER_EARN_PER_STOCK']/a_dictionary['RECENT_QUARTER_EARN']))
            #except:
                #pass
            #try:
                if 'QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN_PER_STOCK'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN_PER_STOCK'])
                        a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                    #a_dictionary['PER'] = int(STOCK_PRICE/a_dictionary['QUARTER_EARN_PER_STOCK'])
            #except:
                #a_dictionary['PER'] = 0
                #pass
                if 'RECENT_QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'RECENT_QUARTER_EARN' in a_dictionary.keys():
                        if 'OLD_EARN' in a_dictionary.keys():
                            a_dictionary['OLD_EARN_PER_STOCK'] = int(a_dictionary['OLD_EARN']*(a_dictionary['RECENT_QUARTER_EARN_PER_STOCK']/a_dictionary['RECENT_QUARTER_EARN']))
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_EARN_PER_STOCK'], a_dictionary['OLD_EARN'], a_dictionary['OLD_EARN_PER_STOCK']])
                print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Quarter_Earn', 'Quarter_Earn_Per_Stock', 'Last_Earn', 'Last_Earn_Per_Stock'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
    
    def get_per(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            LAST_YEAR = self.get_last_year_report(REPORT_DATE.strftime("%Y-%m-%d"))
            LAST_STOCK_PRICE = self.get_top_last_year(LAST_YEAR)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #if 'LAST_STOCK_PRICE' in locals():
            LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Old_Earn_Per_Stock'])
            if LAST_EARN_PER_STOCK != 0:
                LAST_PER = int(LAST_STOCK_PRICE/LAST_EARN_PER_STOCK)
            if LAST_EARN_PER_STOCK == 0:
                LAST_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER), str(LAST_STOCK_PRICE), str(LAST_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price', 'Quarter_Per', 'Last_Stock_Price', 'Last_Per'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
#have 2 keys 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788'
akaps = how_per('d088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9')
akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(1).csv')

In [ ]:



#test




import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import datetime as dt 
from datetime import date
import time
from dateutil.relativedelta import relativedelta
import math
import warnings
import OpenDartReader
from pandas.core.common import SettingWithCopyWarning
import re
from collections.abc import Mapping

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)



class how_per():
    def __init__(self, api_key):
        self.api_key = api_key
        self.OLD_COUNT = 0
        self.OLD_SYMBOL = '가나다'
        self.SYMBOL = '가나다'
        self.TODAY = dt.date.today()
        self.BOOM_PER_LIST = []
        self.dart = OpenDartReader(self.api_key)
        
    def outlier_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q4 = np.percentile(datacolumn , 100)
        except:
            return True
        return Q4
        
    def has_no_fix(self,word):
        if '[기재정정]' in word:
            return True
        else:
            return False

    def get_want_fast_report(self, a_s_list):
        a_s_list = a_s_list[a_s_list['report_nm'].str.contains('연결재무제표기준영업', regex = False)]
        #a_s_list['rcept_dt'] = pd.to_datetime(a_s_list['rcept_dt'], format = '%Y%m%d').dt.date
        #a_s_list = a_s_list[a_s_list['rcept_dt']]
        #a_s_list.drop(['Unnamed: 0'], axis =1, inplace = True)
        a_s_list.reset_index(drop = True, inplace = True)
        return a_s_list

    def get_before_report_time(self, report_year, report_month):
        before_report_year = report_year-1
        before_report_month = 12
        return before_report_year, before_report_month

    def get_earn_fast_earn(self, fast_report_list):
        if not self.check_lenz(fast_report_list) :
            try:
                REPORT_NO = str(fast_report_list.loc[0, 'rcept_no'])
            except:
                return 0
            html_text = self.dart.document(REPORT_NO)
            DF_LIST = pd.read_html(html_text)
            for df_list in DF_LIST:
                if df_list[0].str.contains('※ 동 정보는 확정치가 아닌 잠정치로서 향후 확정치와는 다를 수 있음.').any():
                    df_list.set_index([0,1], inplace = True)
                    #if df_list[2].str.contains('당기이익').any():
                    AFTER_EARN = df_list.loc[('당기순이익','당해실적'), 2].item()
                    if AFTER_EARN == '-':
                        return 0
                    return int(AFTER_EARN)*(10**6)
        return 0
    
    def check_price_data(self, report_date):
        report_stock_date = date(2000,1,1)
        before_report_date = report_date - relativedelta(days = 14)
        now_report_date = report_date + relativedelta(days = 1)
        before_report_date = before_report_date.strftime("%Y-%m-%d")
        now_report_date = now_report_date.strftime("%Y-%m-%d")
        report_stock_price = 0
        df_stock_price = fdr.DataReader(self.SYMBOL, before_report_date, now_report_date)
        
        for i in range(0,14):
            report_stock_date = report_date - relativedelta(days = i)
            report_stock_date = report_stock_date.strftime("%Y-%m-%d")
            try:
                report_stock_price = df_stock_price.loc[report_stock_date, 'Close']
            except:
                continue
            if report_stock_price:
                break
            else:
                continue
        return int(report_stock_price)
    
    def get_top_last_year(self, last_year):
        now_last_year = str(last_year + 1) + '-03-30'
        before_last_year = str(last_year) + '-03-30'
        df_stock_price = fdr.DataReader(self.SYMBOL, before_last_year, now_last_year)
        price_column = df_stock_price[['Close']]
        top_price_last_year = self.outlier_treatment(price_column)
        return int(top_price_last_year)
        
    
    def give_earn_data(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        self.report.set_index(['sj_div','account_id'], inplace = True)
        #'ifrs-full_ProfitLossAttributableToOwnersOfParent'
        'ifrs-full_ProfitLossFromContinuingOperations'
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if self.report.index.isin([('CIS', 'ifrs-full_BasicEarningsLossPerShareFromContinuingOperations')]).any():
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('CIS','ifrs-full_BasicEarningsLossPerShareFromContinuingOperations'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn), int(quarter_earn_per_stock)
            report = self.report[self.report['account_nm'].str.contains('계속')==True]
            report = report[report['account_nm'].str.contains('기본주당')==True]
            report.reset_index(drop = True, inplace = True)
            if not self.check_empty(report):
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                quarter_earn_per_stock = report.loc[0, 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
            if self.report.index.isin([('CIS', 'ifrs-full_BasicEarningsLossPerShare')]).any():
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('CIS','ifrs-full_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if self.report.index.isin([('CIS', 'ifrs-full_BasicEarningsLossPerShare')]).any():
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('CIS','ifrs-full_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if self.report.index.isin([('CIS', 'ifrs_BasicEarningsLossPerShare')]).any():
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('CIS','ifrs_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if self.report.index.isin([('IS', 'ifrs-full_BasicEarningsLossPerShareFromContinuingOperations')]).any():
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('IS','ifrs-full_BasicEarningsLossPerShareFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
            if self.report.index.isin([('IS', 'ifrs-full_BasicEarningsLossPerShare')]).any():
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('IS','ifrs-full_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if self.report.index.isin([('IS', 'ifrs-full_BasicEarningsLossPerShare')]).any():
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('IS','ifrs-full_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if self.report.index.isin([('IS', 'ifrs_BasicEarningsLossPerShare')]).any():
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('IS','ifrs_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn), int(quarter_earn_per_stock)
        else:
            return
        
    def check_lenz(self, df):
        return df is None
    
    def check_empty(self, df):
        return df.empty
        
    def get_fs_of_recent(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL , str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        if report_month == 6:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        if report_month == 9:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        if report_month == 12:
            #if True:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        else:
            return 
        
    def get_fs_of_time(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data() 
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data() 
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        if report_month == 6:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        if report_month == 9:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                #self.report.to_csv('kg.csv')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                                if not self.check_lenz(self.report):
                                    if not self.check_empty(self.report):
                                        PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                        QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                        QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + PLUS_EARN_PER_STOCK
                                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                                if not self.check_lenz(self.report):
                                    if not self.check_empty(self.report):
                                        PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                        QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                        QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        if report_month == 12:
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
            try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        return int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            except:
                pass
        else:
            return
    
    def get_real_report_month(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year
            report_month = 6
        if report_month == 6:
            report_year = report_year
            report_month = 9
        if report_month == 9:
            report_year = report_year
            report_month = 12
        if report_month == 12:
            report_year = report_year + 1
            report_month = 3
        return report_year, report_month
    
    def get_predict_earn_per_stock(self, report_month, earn_per_stock):
        earn_per_stock = int(earn_per_stock)
        
        if report_month == 3:
            earn_per_stock = earn_per_stock*4
        if report_month == 6:
            earn_per_stock = earn_per_stock*2
        if report_month == 9:
            earn_per_stock = earn_per_stock*(4/3)
        if report_month == 12:
            earn_per_stock = earn_per_stock
        
        return earn_per_stock
    
    def get_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_last_year_report(self, report_date):
        delimiters = "-"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_date)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        last_year = int(nm_num_list[0])-1
        return last_year
    
    def get_report_year_month(self, report_nm):
        delimiters = "(", ".", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_one_back(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year - 1
            report_month = 12
        if report_month == 6:
            report_year = report_year
            report_month = 3
        if report_month == 9:
            report_year = report_year
            report_month = 6
        if report_month == 12:
            report_year = report_year
            report_month = 9
        return report_year, report_month
            
    def get_symbol_six(self, analy_kosdaq, i):
        new_symbol = analy_kosdaq.loc[i, 'Symbol']
        if new_symbol != self.OLD_SYMBOL:
            self.OLD_COUNT = 0
            symbol = analy_kosdaq.loc[i, 'Symbol']
            symbol_length = self.digit_length(symbol)
            if symbol_length != 6:
                zero_need = 6 - symbol_length
                self.SYMBOL = '0'*zero_need + str(symbol)
            if symbol_length == 6:
                self.SYMBOL = str(symbol)
            self.OLD_SYMBOL = new_symbol
            return
        if new_symbol == self.OLD_SYMBOL:
            self.OLD_COUNT += 1
        return 
    
    #자릿수 찾기
    def digit_length(self, n):
        ans = 0
        n= int(n)

        while n:
            n //= 10
            ans += 1

        return ans


    def how_report(self, analyse_date, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(1,2):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            A_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, final = False)
            FAST_REPORT_LIST = self.get_want_fast_report(A_S_LIST)
            FAST_REPORT_NM, FAST_REPORT_DT = self.get_report_date(FAST_REPORT_LIST)
            if not self.check_empty(FAST_REPORT_LIST):
                #check if report is between real_report
                if REPORT_DT - relativedelta(months = 1) < FAST_REPORT_DT and REPORT_DT > FAST_REPORT_DT:
                    if True:
                    #try:
                        AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    #except:
                        #pass
                    #try:
                        if BACK_REPORT_MONTH != 12:
                            BOOM_QUARTER_EARN, BOOM_QUARTER_EARN_PER_STOCK = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                            a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    #except:
                        #pass

                elif REPORT_DT + relativedelta(months = 4) > FAST_REPORT_DT and REPORT_DT < FAST_REPORT_DT:
                    if True:
                    #try:
                        AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        REPORT_YEAR, REPORT_MONTH = self.get_real_report_month(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    #except:
                        #pass
                    #try:
                        if REPORT_MONTH != 12:
                            BOOM_QUARTER_EARN, BOOM_QUARTER_EARN_PER_STOCK = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                            a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    #except:
                        #pass
            else:
                if True:
                #try:
                    BOOM_QUARTER_EARN, BOOM_QUARTER_EARN_PER_STOCK = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                    a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                    a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                    a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                    time.sleep(1)
                #except:
                    #pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            if True:
            #try:
                RECENT_QUARTER_EARN,RECENT_QUARTER_EARN_PER_STOCK= self.get_fs_of_recent(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['RECENT_QUARTER_EARN'] = RECENT_QUARTER_EARN
                a_dictionary['RECENT_QUARTER_EARN_PER_STOCK'] = RECENT_QUARTER_EARN_PER_STOCK
                time.sleep(1)
            #except:
                #pass
            #GET OLD REPORT EARN TO COMPARE
            #try:
                OLD_EARN, OLD_EARN_PER_STOCK = self.get_fs_of_recent(OLD_REPORT_YEAR, OLD_REPORT_MONTH)
                a_dictionary['OLD_EARN'] = OLD_EARN
                a_dictionary['OLD_EARN_PER_STOCK'] = OLD_EARN_PER_STOCK
                time.sleep(1)
            #except:
                #pass
            if True:
            #try:
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = a_dictionary['QUARTER_EARN'] + a_dictionary['AFTER_EARN']
            #except:
                pass
            #try:
                if 'RECENT_QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'RECENT_QUARTER_EARN' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = int(a_dictionary['QUARTER_EARN']*(a_dictionary['RECENT_QUARTER_EARN_PER_STOCK']/a_dictionary['RECENT_QUARTER_EARN']))
            #except:
                #pass
            #try:
                if 'QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN_PER_STOCK'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN_PER_STOCK'])
                        a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                    #a_dictionary['PER'] = int(STOCK_PRICE/a_dictionary['QUARTER_EARN_PER_STOCK'])
            #except:
                #a_dictionary['PER'] = 0
                #pass
                if 'RECENT_QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'RECENT_QUARTER_EARN' in a_dictionary.keys():
                        if 'OLD_EARN' in a_dictionary.keys():
                            a_dictionary['OLD_EARN_PER_STOCK'] = int(a_dictionary['OLD_EARN']*(a_dictionary['RECENT_QUARTER_EARN_PER_STOCK']/a_dictionary['RECENT_QUARTER_EARN']))
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_EARN_PER_STOCK'], a_dictionary['OLD_EARN'], a_dictionary['OLD_EARN_PER_STOCK']])
                print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Quarter_Earn', 'Quarter_Earn_Per_Stock', 'Last_Earn', 'Last_Earn_Per_Stock'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
    
    def get_per(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            LAST_YEAR = self.get_last_year_report(REPORT_DATE.strftime("%Y-%m-%d"))
            LAST_STOCK_PRICE = self.get_top_last_year(LAST_YEAR)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #if 'LAST_STOCK_PRICE' in locals():
            LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Old_Earn_Per_Stock'])
            if LAST_EARN_PER_STOCK != 0:
                LAST_PER = int(LAST_STOCK_PRICE/LAST_EARN_PER_STOCK)
            if LAST_EARN_PER_STOCK == 0:
                LAST_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER), str(LAST_STOCK_PRICE), str(LAST_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price', 'Quarter_Per', 'Last_Stock_Price', 'Last_Per'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
#have 2 keys 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788'
akaps = how_per('d088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9')
akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(5).csv')

In [ ]:



#real_test




import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import datetime as dt 
from datetime import date
import time
from dateutil.relativedelta import relativedelta
import math
import warnings
import OpenDartReader
from pandas.core.common import SettingWithCopyWarning
import re
from collections.abc import Mapping

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)



class how_per():
    def __init__(self, api_key):
        self.api_key = api_key
        self.OLD_COUNT = 0
        self.OLD_SYMBOL = '가나다'
        self.SYMBOL = '가나다'
        self.TODAY = dt.date.today()
        self.BOOM_PER_LIST = []
        self.dart = OpenDartReader(self.api_key)
        
    def outlier_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q4 = np.percentile(datacolumn , 100)
        except:
            return True
        return Q4
        
    def has_no_fix(self,word):
        if '[기재정정]' in word:
            return True
        else:
            return False

    def get_want_fast_report(self, a_s_list, report_dt):
        a_s_list = a_s_list[a_s_list['report_nm'].str.contains('연결재무제표기준영업', regex = False)]
        a_s_list['rcept_dt'] = pd.to_datetime(a_s_list['rcept_dt'], format = '%Y%m%d').dt.date
        a_s_list = a_s_list[a_s_list['rcept_dt'] +  relativedelta(months = 1) > report_dt]
        a_s_list = a_s_list[a_s_list['rcept_dt'] - relativedelta(months = 3) < report_dt]
        a_s_list.reset_index(drop = True, inplace = True)
        return a_s_list

    def get_before_report_time(self, report_year, report_month):
        before_report_year = report_year-1
        before_report_month = 12
        return before_report_year, before_report_month

    def get_earn_fast_earn(self, fast_report_list):
        if not self.check_lenz(fast_report_list) :
            try:
                REPORT_NO = str(fast_report_list.loc[0, 'rcept_no'])
            except:
                return 0, 0, 0
            html_text = self.dart.document(REPORT_NO)
            DF_LIST = pd.read_html(html_text)
            for df_list in DF_LIST:
                if df_list[0].str.contains('※ 동 정보는 확정치가 아닌 잠정치로서 향후 확정치와는 다를 수 있음.').any():
                    df_list.set_index([0,1], inplace = True)
                    #if df_list[2].str.contains('당기이익').any():
                    after_earn = df_list.loc[('당기순이익','당해실적'), 2].item()
                    after_revenue = df_list.loc[('매출액','당해실적'), 2].item()
                    after_operating_profit = df_list.loc[('영업이익','당해실적'), 2].item()
                    if after_revenue == '-':
                        after_revenue =0
                    if after_operating_profit == '-':
                        after_operating_profit = 0
                    if after_earn == '-':
                        after_earn = 0
                    return int(after_revenue)*(10**6), int(after_operating_profit)*(10**6), int(after_earn)*(10**6)
        return 0, 0, 0
    
    def check_price_data(self, report_date):
        report_stock_date = date(2000,1,1)
        before_report_date = report_date - relativedelta(days = 14)
        now_report_date = report_date + relativedelta(days = 1)
        before_report_date = before_report_date.strftime("%Y-%m-%d")
        now_report_date = now_report_date.strftime("%Y-%m-%d")
        report_stock_price = 0
        df_stock_price = fdr.DataReader(self.SYMBOL, before_report_date, now_report_date)
        
        for i in range(0,14):
            report_stock_date = report_date - relativedelta(days = i)
            report_stock_date = report_stock_date.strftime("%Y-%m-%d")
            try:
                report_stock_price = df_stock_price.loc[report_stock_date, 'Close']
            except:
                continue
            if report_stock_price:
                break
            else:
                continue
        return int(report_stock_price)
    
    def get_top_last_year(self, last_year):
        now_last_year = str(last_year + 1) + '-03-30'
        before_last_year = str(last_year) + '-03-30'
        df_stock_price = fdr.DataReader(self.SYMBOL, before_last_year, now_last_year)
        price_column = df_stock_price[['Close']]
        top_price_last_year = self.outlier_treatment(price_column)
        return int(top_price_last_year)
    
    def give_revenue_data(self):
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        else:
            return
        
    def give_operating_profit(self):
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit= self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_operating_profit))
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit = self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_operating_profit))
        else:
            return
        
    def give_earn_data(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if self.report.index.isin([('CIS', 'ifrs-full_BasicEarningsLossPerShareFromContinuingOperations')]).any():
                if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                    if ('CIS', 'ifrs-full_BasicEarningsLossPerShareFromContinuingOperations') not in report_nan_list:
                        quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                        quarter_earn_per_stock = self.report.loc[('CIS','ifrs-full_BasicEarningsLossPerShareFromContinuingOperations'), 'thstrm_amount']
                        #print(quarter_earn, quarter_earn_per_stock)
                        return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
            report = self.report[self.report['account_nm'].str.contains('계속')==True]
            report = report[report['account_nm'].str.contains('기본주당')==True]
            report.reset_index(drop = True, inplace = True)
            if not self.check_empty(report):
                if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                    quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                    quarter_earn_per_stock = report.loc[0, 'thstrm_amount']
                    return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
            if self.report.index.isin([('CIS', 'ifrs-full_BasicEarningsLossPerShare')]).any():
                if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                    if ('CIS', 'ifrs-full_BasicEarningsLossPerShare') not in report_nan_list:
                        quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                        quarter_earn_per_stock = self.report.loc[('CIS','ifrs-full_BasicEarningsLossPerShare'), 'thstrm_amount']
                        print(quarter_earn, quarter_earn_per_stock)
                        return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
            report = self.report[self.report['account_nm'].str.contains('기본주당')==True]
            report.reset_index(drop = True, inplace = True)
            if not self.check_empty(report):
                if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                    quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                    quarter_earn_per_stock = report.loc[0, 'thstrm_amount']
                    return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if self.report.index.isin([('CIS', 'ifrs-full_BasicEarningsLossPerShare')]).any():
                if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                    if ('CIS', 'ifrs-full_BasicEarningsLossPerShare') not in report_nan_list:
                        quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                        quarter_earn_per_stock = self.report.loc[('CIS','ifrs-full_BasicEarningsLossPerShare'), 'thstrm_amount']
                        return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
            report = self.report[self.report['account_nm'].str.contains('기본주당')==True]
            report.reset_index(drop = True, inplace = True)
            if not self.check_empty(report):
                if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                    quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                    quarter_earn_per_stock = report.loc[0, 'thstrm_amount']
                    return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if self.report.index.isin([('CIS', 'ifrs_BasicEarningsLossPerShare')]).any():
                if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                    if ('CIS', 'ifrs_BasicEarningsLossPerShare') not in report_nan_list:
                        quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                        quarter_earn_per_stock = self.report.loc[('CIS','ifrs_BasicEarningsLossPerShare'), 'thstrm_amount']
                        return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
            report = self.report[self.report['account_nm'].str.contains('기본주당')==True]
            report.reset_index(drop = True, inplace = True)
            if not self.check_empty(report):
                if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                    quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                    quarter_earn_per_stock = report.loc[0, 'thstrm_amount']
                    return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if self.report.index.isin([('IS', 'ifrs-full_BasicEarningsLossPerShareFromContinuingOperations')]).any():
                if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                    if ('CIS', 'ifrs-full_BasicEarningsLossPerShareFromContinuingOperations') not in report_nan_list:
                        quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                        quarter_earn_per_stock = self.report.loc[('IS','ifrs-full_BasicEarningsLossPerShareFromContinuingOperations'), 'thstrm_amount']
                        return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
            report = self.report[self.report['account_nm'].str.contains('계속')==True]
            report = report[report['account_nm'].str.contains('기본주당')==True]
            report.reset_index(drop = True, inplace = True)
            if not self.check_empty(report):
                if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                    quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                    quarter_earn_per_stock = report.loc[0, 'thstrm_amount']
            if self.report.index.isin([('IS', 'ifrs-full_BasicEarningsLossPerShare')]).any():
                if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                    if ('CIS', 'ifrs-full_BasicEarningsLossPerShare') not in report_nan_list:
                        quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                        quarter_earn_per_stock = self.report.loc[('IS','ifrs-full_BasicEarningsLossPerShare'), 'thstrm_amount']
                        return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
            report = self.report[self.report['account_nm'].str.contains('기본주당')==True]
            report.reset_index(drop = True, inplace = True)
            if not self.check_empty(report):
                if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                    quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                    quarter_earn_per_stock = report.loc[0, 'thstrm_amount']
                    return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if self.report.index.isin([('IS', 'ifrs-full_BasicEarningsLossPerShare')]).any():
                if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                    if ('CIS', 'ifrs-full_BasicEarningsLossPerShare') not in report_nan_list:
                        quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                        quarter_earn_per_stock = self.report.loc[('IS','ifrs-full_BasicEarningsLossPerShare'), 'thstrm_amount']
                        return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
            report = self.report[self.report['account_nm'].str.contains('기본주당')==True]
            report.reset_index(drop = True, inplace = True)
            if not self.check_empty(report):
                if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                    quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                    quarter_earn_per_stock = report.loc[0, 'thstrm_amount']
                    return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if self.report.index.isin([('IS', 'ifrs_BasicEarningsLossPerShare')]).any():
                if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                    if ('CIS', 'ifrs_BasicEarningsLossPerShare') not in report_nan_list:
                        quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                        quarter_earn_per_stock = self.report.loc[('IS','ifrs_BasicEarningsLossPerShare'), 'thstrm_amount']
                        return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
            report = self.report[self.report['account_nm'].str.contains('기본주당')==True]
            report.reset_index(drop = True, inplace = True)
            if not self.check_empty(report):
                if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                    quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                    quarter_earn_per_stock = report.loc[0, 'thstrm_amount']
                    return int(float(quarter_earn)), int(float(quarter_earn_per_stock))
        else:
            return
        
    def check_lenz(self, df):
        return df is None
    
    def check_empty(self, df):
        return df.empty
        
    def get_fs_of_recent(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL , str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    #if not self.check_empty(self.report):
                    QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                    QUARTER_REVENUE = self.give_revenue_data()
                    QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                    return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    #if not self.check_empty(self.report):
                    QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                    QUARTER_REVENUE = self.give_revenue_data()
                    QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                    return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
        if report_month == 6:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    #if not self.check_empty(self.report):
                    QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                    QUARTER_REVENUE = self.give_revenue_data()
                    QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                    return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    #if not self.check_empty(self.report):
                    QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                    QUARTER_REVENUE = self.give_revenue_data()
                    QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                    return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    #if not self.check_empty(self.report):
                    QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                    QUARTER_REVENUE = self.give_revenue_data()
                    QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                    return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    #if not self.check_empty(self.report):
                    QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                    QUARTER_REVENUE = self.give_revenue_data()
                    QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                    return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    #if not self.check_empty(self.report):
                    QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                    QUARTER_REVENUE = self.give_revenue_data()
                    QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                    return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    #if not self.check_empty(self.report):
                    QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                    QUARTER_REVENUE = self.give_revenue_data()
                    QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                    return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
        else:
            return 
        
    def get_fs_of_time(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data() 
                        QUARTER_REVENUE = self.give_revenue_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data() 
                        QUARTER_REVENUE = self.give_revenue_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
        if report_month == 6:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        QUARTER_REVENUE = self.give_revenue_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                PLUS_QUARTER_REVENUE = self.give_revenue_data()
                                PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_REVENUE = QUARTER_REVENUE + PLUS_QUARTER_REVENUE
                                QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        QUARTER_REVENUE = self.give_revenue_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                PLUS_QUARTER_REVENUE = self.give_revenue_data()
                                PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_REVENUE = QUARTER_REVENUE + PLUS_QUARTER_REVENUE
                                QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                #self.report.to_csv('kg.csv')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        QUARTER_REVENUE = self.give_revenue_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                PLUS_QUARTER_REVENUE = self.give_revenue_data()
                                PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_REVENUE = QUARTER_REVENUE + PLUS_QUARTER_REVENUE
                                QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                                if not self.check_lenz(self.report):
                                    if not self.check_empty(self.report):
                                        PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                        PLUS_QUARTER_REVENUE = self.give_revenue_data()
                                        PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                        QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                        QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                        QUARTER_REVENUE = QUARTER_REVENUE + PLUS_QUARTER_REVENUE
                                        QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                        return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        QUARTER_REVENUE = self.give_revenue_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                PLUS_QUARTER_REVENUE = self.give_revenue_data()
                                PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_REVENUE = QUARTER_REVENUE + PLUS_QUARTER_REVENUE
                                QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                                if not self.check_lenz(self.report):
                                    if not self.check_empty(self.report):
                                        PLUS_EARN, PLUS_EARN_PER_STOCK = self.give_earn_data()
                                        PLUS_QUARTER_REVENUE = self.give_revenue_data()
                                        PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                        QUARTER_EARN_PER_STOCK = QUARTER_EARN_PER_STOCK + int(PLUS_EARN*(QUARTER_EARN_PER_STOCK/QUARTER_EARN))
                                        QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                        QUARTER_REVENUE = QUARTER_REVENUE + PLUS_QUARTER_REVENUE
                                        QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                        return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        QUARTER_REVENUE = self.give_revenue_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN, QUARTER_EARN_PER_STOCK = self.give_earn_data()
                        QUARTER_REVENUE = self.give_revenue_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_REVENUE), int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN), int(QUARTER_EARN_PER_STOCK)
            #except:
                #pass
        else:
            return
    
    def get_real_report_month(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year
            report_month = 6
        if report_month == 6:
            report_year = report_year
            report_month = 9
        if report_month == 9:
            report_year = report_year
            report_month = 12
        if report_month == 12:
            report_year = report_year + 1
            report_month = 3
        return report_year, report_month
    
    def get_predict_earn_per_stock(self, report_month, earn_per_stock):
        earn_per_stock = int(earn_per_stock)
        
        if report_month == 3:
            earn_per_stock = earn_per_stock*4
        if report_month == 6:
            earn_per_stock = earn_per_stock*2
        if report_month == 9:
            earn_per_stock = earn_per_stock*(4/3)
        if report_month == 12:
            earn_per_stock = earn_per_stock
        
        return earn_per_stock
    
    def get_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_fast_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            #report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            #report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_last_year_report(self, report_date):
        delimiters = "-"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_date)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        last_year = int(nm_num_list[0])-1
        return last_year
    
    def get_report_year_month(self, report_nm):
        delimiters = "(", ".", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_one_back(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year - 1
            report_month = 12
        if report_month == 6:
            report_year = report_year
            report_month = 3
        if report_month == 9:
            report_year = report_year
            report_month = 6
        if report_month == 12:
            report_year = report_year
            report_month = 9
        return report_year, report_month
            
    def get_symbol_six(self, analy_kosdaq, i):
        new_symbol = analy_kosdaq.loc[i, 'Symbol']
        if new_symbol != self.OLD_SYMBOL:
            self.OLD_COUNT = 0
            symbol = analy_kosdaq.loc[i, 'Symbol']
            symbol_length = self.digit_length(symbol)
            if symbol_length != 6:
                zero_need = 6 - symbol_length
                self.SYMBOL = '0'*zero_need + str(symbol)
            if symbol_length == 6:
                self.SYMBOL = str(symbol)
            self.OLD_SYMBOL = new_symbol
            return
        if new_symbol == self.OLD_SYMBOL:
            self.OLD_COUNT += 1
        return 
    
    #자릿수 찾기
    def digit_length(self, n):
        ans = 0
        n= int(n)

        while n:
            n //= 10
            ans += 1

        return ans


    def how_report(self, analyse_date, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(1,10):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            A_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, final = False)
            FAST_REPORT_LIST = self.get_want_fast_report(A_S_LIST, REPORT_DT)
            #print(FAST_REPORT_LIST)
            FAST_REPORT_NM, FAST_REPORT_DT = self.get_fast_report_date(FAST_REPORT_LIST)
            if not self.check_empty(FAST_REPORT_LIST):
                print('true')
                #check if report is between real_report
                if REPORT_DT - relativedelta(months = 1) < FAST_REPORT_DT and REPORT_DT > FAST_REPORT_DT:
                    if True:
                    #try:
                        AFTER_REVENUE, AFTER_OPERATING_PROFIT, AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        #print(a_dictionary['REPORT_DATE'])
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    #except:
                        #pass
                    #try:
                        if BACK_REPORT_MONTH != 12:
                            FULL_QUARTER_REVENUE, FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN, BOOM_QUARTER_EARN_PER_STOCK = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                            a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                            a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    #except:
                        #pass

                if REPORT_DT + relativedelta(months = 4) > FAST_REPORT_DT and REPORT_DT < FAST_REPORT_DT:
                    if True:
                    #try:
                        AFTER_REVENUE, AFTER_OPERATING_PROFIT, AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        REPORT_YEAR, REPORT_MONTH = self.get_real_report_month(REPORT_YEAR, REPORT_MONTH)
                        #print(a_dictionary['REPORT_DATE'])
                        time.sleep(1)
                    #except:
                        #pass
                    #try:
                        if REPORT_MONTH != 12:
                            FULL_QUARTER_REVENUE, FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN, BOOM_QUARTER_EARN_PER_STOCK = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                            a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                            a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    #except:
                        #pass
            else:
                if True:
                #try:
                    FULL_QUARTER_REVENUE, FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN, BOOM_QUARTER_EARN_PER_STOCK = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                    a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                    a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                    a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                    a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                    #print(a_dictionary['REPORT_DATE'])
                    time.sleep(1)
                #except:
                    #pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            if True:
            #try:
                RECENT_QUARTER_REVENUE, RECENT_QUARTER_OPERATING_PROFIT, RECENT_QUARTER_EARN,RECENT_QUARTER_EARN_PER_STOCK= self.get_fs_of_recent(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['RECENT_QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                a_dictionary['RECENT_QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                a_dictionary['RECENT_QUARTER_EARN'] = RECENT_QUARTER_EARN
                a_dictionary['RECENT_QUARTER_EARN_PER_STOCK'] = RECENT_QUARTER_EARN_PER_STOCK
                time.sleep(1)
            #except:
                #pass
            #GET OLD REPORT EARN TO COMPARE
            #try:
                #print(OLD_REPORT_YEAR, OLD_REPORT_MONTH)
                OLD_QUARTER_REVENUE, OLD_QUARTER_OPERATING_PROFIT, OLD_EARN, OLD_EARN_PER_STOCK = self.get_fs_of_recent(OLD_REPORT_YEAR, OLD_REPORT_MONTH)
                a_dictionary['OLD_QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                a_dictionary['OLD_QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                a_dictionary['OLD_EARN'] = OLD_EARN
                a_dictionary['OLD_EARN_PER_STOCK'] = OLD_EARN_PER_STOCK
                time.sleep(1)
            #except:
                #pass
            if True:
            #try:
                if 'AFTER_REVENUE' in a_dictionary.keys():
                    if 'QUARTER_REVENUE' in a_dictionary.keys():
                        a_dictionary['QUARTER_REVENUE'] = a_dictionary['QUARTER_REVENUE'] + a_dictionary['AFTER_REVENUE']
            #except:
                #pass
                if 'AFTER_REVENUE' in a_dictionary.keys():
                    if 'QUARTER_REVENUE' not in a_dictionary.keys():
                        a_dictionary['QUARTER_REVENUE'] = a_dictionary['AFTER_REVENUE']
                        
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = a_dictionary['QUARTER_OPERATING_PROFIT'] + a_dictionary['AFTER_OPERATING_PROFIT']
                        
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' not in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = a_dictionary['AFTER_OPERATING_PROFIT']
                        
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = a_dictionary['QUARTER_EARN'] + a_dictionary['AFTER_EARN']
                
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' not in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = a_dictionary['AFTER_EARN']
            #try:
                if 'RECENT_QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'RECENT_QUARTER_EARN' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = int(a_dictionary['QUARTER_EARN']*(a_dictionary['RECENT_QUARTER_EARN_PER_STOCK']/a_dictionary['RECENT_QUARTER_EARN']))
            #except:
                #pass
            #try:
                if 'QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN_PER_STOCK'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN_PER_STOCK'])
                        a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                    #a_dictionary['PER'] = int(STOCK_PRICE/a_dictionary['QUARTER_EARN_PER_STOCK'])
            #except:
                #a_dictionary['PER'] = 0
                #pass
                if 'RECENT_QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'RECENT_QUARTER_EARN' in a_dictionary.keys():
                        if 'OLD_EARN' in a_dictionary.keys():
                            a_dictionary['OLD_EARN_PER_STOCK'] = int(a_dictionary['OLD_EARN']*(a_dictionary['RECENT_QUARTER_EARN_PER_STOCK']/a_dictionary['RECENT_QUARTER_EARN']))
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_EARN_PER_STOCK'], a_dictionary['OLD_QUARTER_REVENUE'], a_dictionary['OLD_QUARTER_OPERATING_PROFIT'],a_dictionary['OLD_EARN'], a_dictionary['OLD_EARN_PER_STOCK']])
                print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Quarter_Revenue', 'Quarter_Operating_Profit', 'Quarter_Earn', 'Quarter_Earn_Per_Stock', 'Last_Revenue', 'LAST_Operating_Profit', 'Last_Earn', 'Last_Earn_Per_Stock'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
    
    def get_per(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            LAST_YEAR = self.get_last_year_report(REPORT_DATE.strftime("%Y-%m-%d"))
            LAST_STOCK_PRICE = self.get_top_last_year(LAST_YEAR)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #if 'LAST_STOCK_PRICE' in locals():
            LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Old_Earn_Per_Stock'])
            if LAST_EARN_PER_STOCK != 0:
                LAST_PER = int(LAST_STOCK_PRICE/LAST_EARN_PER_STOCK)
            if LAST_EARN_PER_STOCK == 0:
                LAST_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER), str(LAST_STOCK_PRICE), str(LAST_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price', 'Quarter_Per', 'Last_Stock_Price', 'Last_Per'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
#have 2 keys 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788'
akaps = how_per('d088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9')
akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(5).csv')

In [ ]:



#real_test




import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import datetime as dt 
from datetime import date
import time
from dateutil.relativedelta import relativedelta
import math
import warnings
import OpenDartReader
from pandas.core.common import SettingWithCopyWarning
import re
from collections.abc import Mapping

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)



class how_per():
    def __init__(self, api_key):
        self.api_key = api_key
        self.OLD_COUNT = 0
        self.OLD_SYMBOL = '가나다'
        self.SYMBOL = '가나다'
        self.TODAY = dt.date.today()
        self.BOOM_PER_LIST = []
        self.dart = OpenDartReader(self.api_key)
        
    def outlier_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q4 = np.percentile(datacolumn , 100)
        except:
            return True
        return Q4
        
    def has_no_fix(self,word):
        if '[기재정정]' in word:
            return True
        else:
            return False

    def get_want_fast_report(self, a_s_list, report_dt):
        a_s_list = a_s_list[a_s_list['report_nm'].str.contains('영업(잠정)실적(공정공시)', regex = False)]
        a_s_list['rcept_dt'] = pd.to_datetime(a_s_list['rcept_dt'], format = '%Y%m%d').dt.date
        a_s_list = a_s_list[a_s_list['rcept_dt'] +  relativedelta(months = 1) > report_dt]
        a_s_list = a_s_list[a_s_list['rcept_dt'] - relativedelta(months = 3) < report_dt]
        a_s_list.reset_index(drop = True, inplace = True)
        return a_s_list

    def get_before_report_time(self, report_year, report_month):
        before_report_year = report_year-1
        before_report_month = 12
        return before_report_year, before_report_month

    def get_earn_fast_earn(self, fast_report_list):
        if not self.check_lenz(fast_report_list) :
            try:
                REPORT_NO = str(fast_report_list.loc[0, 'rcept_no'])
            except:
                return 
            html_text = self.dart.document(REPORT_NO)
            DF_LIST = pd.read_html(html_text)
            for df_list in DF_LIST:
                '''if df_list[0].str.contains('※ 당해 정보는 확정치가 아닌 잠정치로 결산 결과 등에 따라 변경될 수 있음.
                ※ 동 정보는 확정치가 아닌 잠정치로서 향후 확정치와는 다를 수 있음.'''
                if df_list[0].str.contains('확정치가 아닌 잠정치', regex = False).any():
                    df_list.set_index([0,1], inplace = True)
                    #if df_list[2].str.contains('당기이익').any():
                    after_earn = df_list.loc[('당기순이익','당해실적'), 2].item()
                    #after_revenue = df_list.loc[('매출액','당해실적'), 2].item()
                    after_operating_profit = df_list.loc[('영업이익','당해실적'), 2].item()
                    #if after_revenue == '-':
                        #after_revenue =0
                    if after_operating_profit == '-':
                        after_operating_profit = 0
                    if after_earn == '-':
                        after_earn = 0
                    #print(after_earn, after_operating_profit)
                    return int(after_operating_profit)*(10**6), int(after_earn)*(10**6)
        return 
    
    def check_price_data(self, report_date):
        report_stock_date = date(2000,1,1)
        before_report_date = report_date - relativedelta(days = 14)
        now_report_date = report_date + relativedelta(days = 1)
        before_report_date = before_report_date.strftime("%Y-%m-%d")
        now_report_date = now_report_date.strftime("%Y-%m-%d")
        report_stock_price = 0
        df_stock_price = fdr.DataReader(self.SYMBOL, before_report_date, now_report_date)
        
        for i in range(0,14):
            report_stock_date = report_date - relativedelta(days = i)
            report_stock_date = report_stock_date.strftime("%Y-%m-%d")
            try:
                report_stock_price = df_stock_price.loc[report_stock_date, 'Close']
            except:
                continue
            if report_stock_price:
                break
            else:
                continue
        return int(report_stock_price)
    
    def get_top_last_year(self, last_year):
        now_last_year = str(last_year + 1) + '-03-30'
        before_last_year = str(last_year) + '-03-30'
        df_stock_price = fdr.DataReader(self.SYMBOL, before_last_year, now_last_year)
        price_column = df_stock_price[['Close']]
        top_price_last_year = self.outlier_treatment(price_column)
        return int(top_price_last_year)
    
    '''def give_revenue_data(self):
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        else:
            return'''
    
    def get_stock_amount(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        if report_month == 3:
            report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[0])
            report.set_index(['stock_knd', 'nm'], inplace = True)
            individual_stock_amount = report.loc[('보통주','계'), 'trmend_posesn_stock_co']
            individual_stock_percent = report.loc[('보통주','계'), 'trmend_posesn_stock_qota_rt']
            individual_stock_amount = self.get_only_number(individual_stock_amount)
            individual_stock_percent = float(individual_stock_percent)
            stock_amount = individual_stock_amount*(100/individual_stock_percent)
            return int(stock_amount)
        if report_month == 6:
            report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[1])
            report.set_index(['stock_knd', 'nm'], inplace = True)
            individual_stock_amount = report.loc[('보통주','계'), 'trmend_posesn_stock_co']
            individual_stock_percent = report.loc[('보통주','계'), 'trmend_posesn_stock_qota_rt']
            individual_stock_amount = self.get_only_number(individual_stock_amount)
            individual_stock_percent = float(individual_stock_percent)
            stock_amount = individual_stock_amount*(100/individual_stock_percent)
            return int(stock_amount)
        if report_month == 9:
            report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[2])
            report.set_index(['stock_knd', 'nm'], inplace = True)
            individual_stock_amount = report.loc[('보통주','계'), 'trmend_posesn_stock_co']
            individual_stock_percent = report.loc[('보통주','계'), 'trmend_posesn_stock_qota_rt']
            individual_stock_amount = self.get_only_number(individual_stock_amount)
            individual_stock_percent = float(individual_stock_percent)
            stock_amount = individual_stock_amount*(100/individual_stock_percent)
            return int(stock_amount)
        if report_month == 12:
            report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[3])
            report.set_index(['stock_knd', 'nm'], inplace = True)
            individual_stock_amount = report.loc[('보통주','계'), 'trmend_posesn_stock_co']
            individual_stock_percent = report.loc[('보통주','계'), 'trmend_posesn_stock_qota_rt']
            individual_stock_amount = self.get_only_number(individual_stock_amount)
            individual_stock_percent = float(individual_stock_percent)
            stock_amount = individual_stock_amount*(100/individual_stock_percent)
            return int(stock_amount)
        else:
            return
        
    def give_operating_profit(self):
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit= self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit = self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        else:
            return
    def give_revenue_data(self):
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        else:
            return
        
    def give_earn_data(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                quarter_earn_per_stock = self.report.loc[('CIS','ifrs_BasicEarningsLossPerShare'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return
        
    def check_lenz(self, df):
        return df is None
    
    def check_empty(self, df):
        return df.empty
        
    def get_fs_of_recent(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL , str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        if report_month == 6:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        else:
            return 
        
    def get_fs_of_time(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        if report_month == 6:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN = self.give_earn_data()
                                PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN = self.give_earn_data()
                                PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                #self.report.to_csv('kg.csv')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN = self.give_earn_data()
                                PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                                if not self.check_lenz(self.report):
                                    if not self.check_empty(self.report):
                                        PLUS_EARN = self.give_earn_data()
                                        PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                        QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                        QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN = self.give_earn_data()
                                PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                                if not self.check_lenz(self.report):
                                    if not self.check_empty(self.report):
                                        PLUS_EARN = self.give_earn_data()
                                        PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                        QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                        QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN= self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN),
            #except:
                #pass
        else:
            return
    
    def get_real_report_month(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year
            report_month = 6
        if report_month == 6:
            report_year = report_year
            report_month = 9
        if report_month == 9:
            report_year = report_year
            report_month = 12
        if report_month == 12:
            report_year = report_year + 1
            report_month = 3
        return report_year, report_month
    
    def get_predict_earn_per_stock(self, report_month, earn_per_stock):
        earn_per_stock = int(earn_per_stock)
        
        if report_month == 3:
            earn_per_stock = earn_per_stock*4
        if report_month == 6:
            earn_per_stock = earn_per_stock*2
        if report_month == 9:
            earn_per_stock = earn_per_stock*(4/3)
        if report_month == 12:
            earn_per_stock = earn_per_stock
        
        return earn_per_stock
    
    def get_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_fast_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            #report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            #report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_last_year_report(self, report_date):
        delimiters = "-"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_date)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        last_year = int(nm_num_list[0])-1
        return last_year
    
    def get_only_number(self, report_data):
        delimiters = ","
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_data)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        element_amount = len(nm_num_list)
        report_data = 0
        for i in range(len(nm_num_list)):
            report_data = report_data + nm_num_list[i]*(10**((element_amount - i - 1)*3))
        return report_data
    
    def get_report_year_month(self, report_nm):
        delimiters = "(", ".", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_one_back(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year - 1
            report_month = 12
        if report_month == 6:
            report_year = report_year
            report_month = 3
        if report_month == 9:
            report_year = report_year
            report_month = 6
        if report_month == 12:
            report_year = report_year
            report_month = 9
        return report_year, report_month
            
    def get_symbol_six(self, analy_kosdaq, i):
        new_symbol = analy_kosdaq.loc[i, 'Symbol']
        if new_symbol != self.OLD_SYMBOL:
            self.OLD_COUNT = 0
            symbol = analy_kosdaq.loc[i, 'Symbol']
            symbol_length = self.digit_length(symbol)
            if symbol_length != 6:
                zero_need = 6 - symbol_length
                self.SYMBOL = '0'*zero_need + str(symbol)
            if symbol_length == 6:
                self.SYMBOL = str(symbol)
            self.OLD_SYMBOL = new_symbol
            return
        if new_symbol == self.OLD_SYMBOL:
            self.OLD_COUNT += 1
        return 
    
    #자릿수 찾기
    def digit_length(self, n):
        ans = 0
        n= int(n)

        while n:
            n //= 10
            ans += 1

        return ans


    def how_report(self, analyse_date, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(812,813):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            A_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, final = False)
            FAST_REPORT_LIST = self.get_want_fast_report(A_S_LIST, REPORT_DT)
            #print(FAST_REPORT_LIST)
            FAST_REPORT_NM, FAST_REPORT_DT = self.get_fast_report_date(FAST_REPORT_LIST)
            if not self.check_empty(FAST_REPORT_LIST):
                #print('true')
                #check if report is between real_report
                if REPORT_DT - relativedelta(months = 1) < FAST_REPORT_DT and REPORT_DT > FAST_REPORT_DT:
                    #if True:
                    try:
                        AFTER_OPERATING_PROFIT, AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        #a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        #print(a_dictionary['REPORT_DATE'])
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    except:
                        pass
                    try:
                        if BACK_REPORT_MONTH != 12:
                            FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                            #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                            #a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                            a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                            #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    except:
                        pass

                if REPORT_DT + relativedelta(months = 4) > FAST_REPORT_DT and REPORT_DT < FAST_REPORT_DT:
                    #if True:
                    try:
                        AFTER_OPERATING_PROFIT, AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        #a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        REPORT_YEAR, REPORT_MONTH = self.get_real_report_month(REPORT_YEAR, REPORT_MONTH)
                        #print(a_dictionary['REPORT_DATE'])
                        time.sleep(1)
                    except:
                        pass
                    try:
                        if REPORT_MONTH != 12:
                            FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                            #a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                            a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                            #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    except:
                        pass
            else:
                #if True:
                try:
                    FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                    #a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                    a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                    #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                    a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                    time.sleep(1)
                except:
                    pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            #GET OLD REPORT EARN TO COMPARE
            try:
                #print(OLD_REPORT_YEAR, OLD_REPORT_MONTH)
                OLD_QUARTER_OPERATING_PROFIT, OLD_EARN = self.get_fs_of_recent(OLD_REPORT_YEAR, OLD_REPORT_MONTH)
                #a_dictionary['OLD_QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                a_dictionary['OLD_QUARTER_OPERATING_PROFIT'] = OLD_QUARTER_OPERATING_PROFIT
                a_dictionary['OLD_EARN'] = OLD_EARN
                #a_dictionary['OLD_EARN_PER_STOCK'] = OLD_EARN_PER_STOCK
                time.sleep(1)
            except:
                pass
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
            except:
                pass
            if True:
                        
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = a_dictionary['QUARTER_OPERATING_PROFIT'] + a_dictionary['AFTER_OPERATING_PROFIT']
                        
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' not in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = a_dictionary['AFTER_OPERATING_PROFIT']
                        
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = a_dictionary['QUARTER_EARN'] + a_dictionary['AFTER_EARN']
                
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' not in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = a_dictionary['AFTER_EARN']
                try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = int(a_dictionary['QUARTER_EARN']/a_dictionary['STOCK_AMOUNT'])
                except:
                    continue
            #try:
                if 'QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN_PER_STOCK'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN_PER_STOCK'])
                        a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                    #a_dictionary['PER'] = int(STOCK_PRICE/a_dictionary['QUARTER_EARN_PER_STOCK'])
            #except:
                #a_dictionary['PER'] = 0
                
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                #pass
                try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'OLD_EARN' in a_dictionary.keys():
                            a_dictionary['OLD_EARN_PER_STOCK'] = int(a_dictionary['OLD_EARN']/a_dictionary['STOCK_AMOUNT'])
                except:
                    continue
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_EARN_PER_STOCK'], a_dictionary['OLD_QUARTER_OPERATING_PROFIT'],a_dictionary['OLD_EARN'], a_dictionary['OLD_EARN_PER_STOCK']])
                print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Quarter_Operating_Profit', 'Quarter_Earn', 'Quarter_Earn_Per_Stock', 'Last_Operating_Profit', 'Last_Earn', 'Last_Earn_Per_Stock'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
    
    def get_per(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            LAST_YEAR = self.get_last_year_report(REPORT_DATE.strftime("%Y-%m-%d"))
            LAST_STOCK_PRICE = self.get_top_last_year(LAST_YEAR)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #if 'LAST_STOCK_PRICE' in locals():
            LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Last_Earn_Per_Stock'])
            if LAST_EARN_PER_STOCK != 0:
                LAST_PER = int(LAST_STOCK_PRICE/LAST_EARN_PER_STOCK)
            if LAST_EARN_PER_STOCK == 0:
                LAST_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER), str(LAST_STOCK_PRICE), str(LAST_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price', 'Quarter_Per', 'Last_Stock_Price', 'Last_Per'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
#have 2 keys 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788'
akaps = how_per('1eca9bad2afb8becdc5b1714112dd097e73da788')
#akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(7).csv')
akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ_test.csv')

In [ ]:



#real_test




import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import datetime as dt 
from datetime import date
import time
from dateutil.relativedelta import relativedelta
import math
import warnings
import OpenDartReader
from pandas.core.common import SettingWithCopyWarning
import re
from collections.abc import Mapping

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)



class how_per():
    def __init__(self, api_key):
        self.api_key = api_key
        self.OLD_COUNT = 0
        self.OLD_SYMBOL = '가나다'
        self.SYMBOL = '가나다'
        self.TODAY = dt.date.today()
        self.BOOM_PER_LIST = []
        self.dart = OpenDartReader(self.api_key)
        
    def outlier_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q4 = np.percentile(datacolumn , 100)
        except:
            return True
        return Q4
        
    def has_no_fix(self,word):
        if '[기재정정]' in word:
            return True
        else:
            return False

    def get_want_fast_report(self, a_s_list, report_dt):
        a_s_list = a_s_list[a_s_list['report_nm'].str.contains('영업(잠정)실적(공정공시)', regex = False)]
        a_s_list['rcept_dt'] = pd.to_datetime(a_s_list['rcept_dt'], format = '%Y%m%d').dt.date
        a_s_list = a_s_list[a_s_list['rcept_dt'] +  relativedelta(months = 1) > report_dt]
        a_s_list = a_s_list[a_s_list['rcept_dt'] - relativedelta(months = 3) < report_dt]
        a_s_list.reset_index(drop = True, inplace = True)
        return a_s_list

    def get_before_report_time(self, report_year, report_month):
        before_report_year = report_year-1
        before_report_month = 12
        return before_report_year, before_report_month

    def get_earn_fast_earn(self, fast_report_list):
        if not self.check_lenz(fast_report_list) :
            try:
                REPORT_NO = str(fast_report_list.loc[0, 'rcept_no'])
            except:
                return 
            html_text = self.dart.document(REPORT_NO)
            DF_LIST = pd.read_html(html_text)
            for df_list in DF_LIST:
                '''if df_list[0].str.contains('※ 당해 정보는 확정치가 아닌 잠정치로 결산 결과 등에 따라 변경될 수 있음.
                ※ 동 정보는 확정치가 아닌 잠정치로서 향후 확정치와는 다를 수 있음.'''
                if df_list[0].str.contains('확정치가 아닌 잠정치', regex = False).any():
                    df_list.set_index([0,1], inplace = True)
                    #if df_list[2].str.contains('당기이익').any():
                    after_earn = df_list.loc[('당기순이익','당해실적'), 2].item()
                    #after_revenue = df_list.loc[('매출액','당해실적'), 2].item()
                    after_operating_profit = df_list.loc[('영업이익','당해실적'), 2].item()
                    #if after_revenue == '-':
                        #after_revenue =0
                    if after_operating_profit == '-':
                        after_operating_profit = 0
                    if after_earn == '-':
                        after_earn = 0
                    #print(after_earn, after_operating_profit)
                    return int(after_operating_profit)*(10**6), int(after_earn)*(10**6)
        return 
    
    def check_price_data(self, report_date):
        report_stock_date = date(2000,1,1)
        before_report_date = report_date - relativedelta(days = 14)
        now_report_date = report_date + relativedelta(days = 1)
        before_report_date = before_report_date.strftime("%Y-%m-%d")
        now_report_date = now_report_date.strftime("%Y-%m-%d")
        report_stock_price = 0
        df_stock_price = fdr.DataReader(self.SYMBOL, before_report_date, now_report_date)
        
        for i in range(0,14):
            report_stock_date = report_date - relativedelta(days = i)
            report_stock_date = report_stock_date.strftime("%Y-%m-%d")
            try:
                report_stock_price = df_stock_price.loc[report_stock_date, 'Close']
            except:
                continue
            if report_stock_price:
                break
            else:
                continue
        return int(report_stock_price)
    
    def get_top_last_year(self, last_year):
        now_last_year = str(last_year + 1) + '-03-30'
        before_last_year = str(last_year) + '-03-30'
        df_stock_price = fdr.DataReader(self.SYMBOL, before_last_year, now_last_year)
        price_column = df_stock_price[['Close']]
        top_price_last_year = self.outlier_treatment(price_column)
        return int(top_price_last_year)
    
    '''def give_revenue_data(self):
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        else:
            return'''
    
    def get_stock_amount(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        if report_month == 3:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[0])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 6:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[1])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 9:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[2])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 12:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[3])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        else:
            return
        
    def give_operating_profit(self):
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit= self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit = self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return
        
    def give_earn_data(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return
        
    def check_lenz(self, df):
        return df is None
    
    def check_empty(self, df):
        return df.empty
        
    def get_fs_of_recent(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL , str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        if report_month == 6:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        else:
            return 
        
    def get_fs_of_time(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        if report_month == 6:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                self.report.to_csv('test.csv')
                                PLUS_EARN = self.give_earn_data()
                                PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN = self.give_earn_data()
                                PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                #self.report.to_csv('kg.csv')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN = self.give_earn_data()
                                PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='CFS')
                                if not self.check_lenz(self.report):
                                    if not self.check_empty(self.report):
                                        PLUS_EARN = self.give_earn_data()
                                        PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                        QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                        QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                        if not self.check_lenz(self.report):
                            if not self.check_empty(self.report):
                                PLUS_EARN = self.give_earn_data()
                                PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                                if not self.check_lenz(self.report):
                                    if not self.check_empty(self.report):
                                        PLUS_EARN = self.give_earn_data()
                                        PLUS_QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                                        QUARTER_EARN = QUARTER_EARN + PLUS_EARN
                                        QUARTER_OPERATING_PROFIT = QUARTER_OPERATING_PROFIT + PLUS_QUARTER_OPERATING_PROFIT
                                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN= self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN)
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        return int(QUARTER_OPERATING_PROFIT), int(QUARTER_EARN),
            #except:
                #pass
        else:
            return
    
    def get_real_report_month(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year
            report_month = 6
        if report_month == 6:
            report_year = report_year
            report_month = 9
        if report_month == 9:
            report_year = report_year
            report_month = 12
        if report_month == 12:
            report_year = report_year + 1
            report_month = 3
        return report_year, report_month
    
    def get_predict_earn_per_stock(self, report_month, earn_per_stock):
        earn_per_stock = int(earn_per_stock)
        
        if report_month == 3:
            earn_per_stock = earn_per_stock*4
        if report_month == 6:
            earn_per_stock = earn_per_stock*2
        if report_month == 9:
            earn_per_stock = earn_per_stock*(4/3)
        if report_month == 12:
            earn_per_stock = earn_per_stock
        
        return earn_per_stock
    
    def get_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_fast_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            #report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            #report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_last_year_report(self, report_date):
        delimiters = "-"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_date)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        last_year = int(nm_num_list[0])-1
        return last_year
    
    def get_only_number(self, report_data):
        delimiters = ","
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_data)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        element_amount = len(nm_num_list)
        report_data = 0
        for i in range(len(nm_num_list)):
            report_data = report_data + nm_num_list[i]*(10**((element_amount - i - 1)*3))
        return report_data
    
    def get_report_year_month(self, report_nm):
        delimiters = "(", ".", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_one_back(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year - 1
            report_month = 12
        if report_month == 6:
            report_year = report_year
            report_month = 3
        if report_month == 9:
            report_year = report_year
            report_month = 6
        if report_month == 12:
            report_year = report_year
            report_month = 9
        return report_year, report_month
            
    def get_symbol_six(self, analy_kosdaq, i):
        new_symbol = analy_kosdaq.loc[i, 'Symbol']
        if new_symbol != self.OLD_SYMBOL:
            self.OLD_COUNT = 0
            symbol = analy_kosdaq.loc[i, 'Symbol']
            symbol_length = self.digit_length(symbol)
            if symbol_length != 6:
                zero_need = 6 - symbol_length
                self.SYMBOL = '0'*zero_need + str(symbol)
            if symbol_length == 6:
                self.SYMBOL = str(symbol)
            self.OLD_SYMBOL = new_symbol
            return
        if new_symbol == self.OLD_SYMBOL:
            self.OLD_COUNT += 1
        return 
    
    #자릿수 찾기
    def digit_length(self, n):
        ans = 0
        n= int(n)

        while n:
            n //= 10
            ans += 1

        return ans


    def how_report(self, analyse_date, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(1000,1270):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            A_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, final = False)
            FAST_REPORT_LIST = self.get_want_fast_report(A_S_LIST, REPORT_DT)
            #print(FAST_REPORT_LIST)
            FAST_REPORT_NM, FAST_REPORT_DT = self.get_fast_report_date(FAST_REPORT_LIST)
            if not self.check_empty(FAST_REPORT_LIST):
                #print('true')
                #check if report is between real_report
                if REPORT_DT - relativedelta(months = 1) < FAST_REPORT_DT and REPORT_DT > FAST_REPORT_DT:
                    #if True:
                    try:
                        AFTER_OPERATING_PROFIT, AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        #a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        #print(a_dictionary['REPORT_DATE'])
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    except:
                        pass
                    try:
                        if BACK_REPORT_MONTH != 12:
                            FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                            #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                            #a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                            a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                            #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    except:
                        pass

                if REPORT_DT + relativedelta(months = 4) > FAST_REPORT_DT and REPORT_DT < FAST_REPORT_DT:
                    #if True:
                    try:
                        AFTER_OPERATING_PROFIT, AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        #a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        REPORT_YEAR, REPORT_MONTH = self.get_real_report_month(REPORT_YEAR, REPORT_MONTH)
                        #print(a_dictionary['REPORT_DATE'])
                        time.sleep(1)
                    except:
                        pass
                    try:
                        if REPORT_MONTH != 12:
                            FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                            #a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                            a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                            #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    except:
                        pass
            else:
                #if True:
                try:
                    FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                    #a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                    a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                    #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                    a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                    time.sleep(1)
                except:
                    pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            #GET OLD REPORT EARN TO COMPARE
            try:
                #print(OLD_REPORT_YEAR, OLD_REPORT_MONTH)
                OLD_QUARTER_OPERATING_PROFIT, OLD_EARN = self.get_fs_of_recent(OLD_REPORT_YEAR, OLD_REPORT_MONTH)
                #a_dictionary['OLD_QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                a_dictionary['OLD_QUARTER_OPERATING_PROFIT'] = OLD_QUARTER_OPERATING_PROFIT
                a_dictionary['OLD_EARN'] = OLD_EARN
                #a_dictionary['OLD_EARN_PER_STOCK'] = OLD_EARN_PER_STOCK
                time.sleep(1)
            except:
                pass
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
            except:
                pass
            if True:
                        
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = a_dictionary['QUARTER_OPERATING_PROFIT'] + a_dictionary['AFTER_OPERATING_PROFIT']
                        
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' not in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = a_dictionary['AFTER_OPERATING_PROFIT']
                        
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = a_dictionary['QUARTER_EARN'] + a_dictionary['AFTER_EARN']
                
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' not in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = a_dictionary['AFTER_EARN']
                try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = int(a_dictionary['QUARTER_EARN']/a_dictionary['STOCK_AMOUNT'])
                except:
                    pass
            #try:
                if 'QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN_PER_STOCK'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN_PER_STOCK'])
                        a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                    #a_dictionary['PER'] = int(STOCK_PRICE/a_dictionary['QUARTER_EARN_PER_STOCK'])
            #except:
                #a_dictionary['PER'] = 0
                
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                #pass
                try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'OLD_EARN' in a_dictionary.keys():
                            a_dictionary['OLD_EARN_PER_STOCK'] = int(a_dictionary['OLD_EARN']/a_dictionary['STOCK_AMOUNT'])
                except:
                    pass
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_EARN_PER_STOCK'], a_dictionary['OLD_QUARTER_OPERATING_PROFIT'],a_dictionary['OLD_EARN'], a_dictionary['OLD_EARN_PER_STOCK']])
                print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Quarter_Operating_Profit', 'Quarter_Earn', 'Quarter_Earn_Per_Stock', 'Last_Operating_Profit', 'Last_Earn', 'Last_Earn_Per_Stock'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
    
    def get_per(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            LAST_YEAR = self.get_last_year_report(REPORT_DATE.strftime("%Y-%m-%d"))
            LAST_STOCK_PRICE = self.get_top_last_year(LAST_YEAR)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #if 'LAST_STOCK_PRICE' in locals():
            LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Last_Earn_Per_Stock'])
            if LAST_EARN_PER_STOCK != 0:
                LAST_PER = int(LAST_STOCK_PRICE/LAST_EARN_PER_STOCK)
            if LAST_EARN_PER_STOCK == 0:
                LAST_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER), str(LAST_STOCK_PRICE), str(LAST_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price', 'Quarter_Per', 'Last_Stock_Price', 'Last_Per'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def how_report(self, analyse_date, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(1000,1270):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            A_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, final = False)
            FAST_REPORT_LIST = self.get_want_fast_report(A_S_LIST, REPORT_DT)
            #print(FAST_REPORT_LIST)
            FAST_REPORT_NM, FAST_REPORT_DT = self.get_fast_report_date(FAST_REPORT_LIST)
            if not self.check_empty(FAST_REPORT_LIST):
                #print('true')
                #check if report is between real_report
                if REPORT_DT - relativedelta(months = 1) < FAST_REPORT_DT and REPORT_DT > FAST_REPORT_DT:
                    #if True:
                    try:
                        AFTER_OPERATING_PROFIT, AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        #a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        #print(a_dictionary['REPORT_DATE'])
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    except:
                        pass
                    try:
                        if BACK_REPORT_MONTH != 12:
                            FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                            #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                            #a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                            a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                            #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    except:
                        pass

                if REPORT_DT + relativedelta(months = 4) > FAST_REPORT_DT and REPORT_DT < FAST_REPORT_DT:
                    #if True:
                    try:
                        AFTER_OPERATING_PROFIT, AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        #a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        REPORT_YEAR, REPORT_MONTH = self.get_real_report_month(REPORT_YEAR, REPORT_MONTH)
                        #print(a_dictionary['REPORT_DATE'])
                        time.sleep(1)
                    except:
                        pass
                    try:
                        if REPORT_MONTH != 12:
                            FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                            #a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                            a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                            #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    except:
                        pass
            else:
                #if True:
                try:
                    FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                    #a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                    a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                    #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                    a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                    time.sleep(1)
                except:
                    pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            #GET OLD REPORT EARN TO COMPARE
            try:
                #print(OLD_REPORT_YEAR, OLD_REPORT_MONTH)
                OLD_QUARTER_OPERATING_PROFIT, OLD_EARN = self.get_fs_of_recent(OLD_REPORT_YEAR, OLD_REPORT_MONTH)
                #a_dictionary['OLD_QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                a_dictionary['OLD_QUARTER_OPERATING_PROFIT'] = OLD_QUARTER_OPERATING_PROFIT
                a_dictionary['OLD_EARN'] = OLD_EARN
                #a_dictionary['OLD_EARN_PER_STOCK'] = OLD_EARN_PER_STOCK
                time.sleep(1)
            except:
                pass
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
            except:
                pass
            if True:
                        
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = a_dictionary['QUARTER_OPERATING_PROFIT'] + a_dictionary['AFTER_OPERATING_PROFIT']
                        
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' not in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = a_dictionary['AFTER_OPERATING_PROFIT']
                        
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = a_dictionary['QUARTER_EARN'] + a_dictionary['AFTER_EARN']
                
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' not in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = a_dictionary['AFTER_EARN']
                try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = int(a_dictionary['QUARTER_EARN']/a_dictionary['STOCK_AMOUNT'])
                except:
                    pass
            #try:
                if 'QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN_PER_STOCK'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN_PER_STOCK'])
                        a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                    #a_dictionary['PER'] = int(STOCK_PRICE/a_dictionary['QUARTER_EARN_PER_STOCK'])
            #except:
                #a_dictionary['PER'] = 0
                
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                #pass
                try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'OLD_EARN' in a_dictionary.keys():
                            a_dictionary['OLD_EARN_PER_STOCK'] = int(a_dictionary['OLD_EARN']/a_dictionary['STOCK_AMOUNT'])
                except:
                    pass
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_EARN_PER_STOCK'], a_dictionary['OLD_QUARTER_OPERATING_PROFIT'],a_dictionary['OLD_EARN'], a_dictionary['OLD_EARN_PER_STOCK']])
                print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Quarter_Operating_Profit', 'Quarter_Earn', 'Quarter_Earn_Per_Stock', 'Last_Operating_Profit', 'Last_Earn', 'Last_Earn_Per_Stock'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
#have 2 keys 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788'
akaps = how_per('1eca9bad2afb8becdc5b1714112dd097e73da788')
#akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(7).csv')
akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ_test.csv')

In [ ]:



#real_test




import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import datetime as dt 
from datetime import date
import time
from dateutil.relativedelta import relativedelta
import math
import warnings
import OpenDartReader
from pandas.core.common import SettingWithCopyWarning
import re
from collections.abc import Mapping

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)



class how_per():
    def __init__(self, api_key):
        self.api_key = api_key
        self.OLD_COUNT = 0
        self.OLD_SYMBOL = '가나다'
        self.SYMBOL = '가나다'
        self.TODAY = dt.date.today()
        self.BOOM_PER_LIST = []
        self.dart = OpenDartReader(self.api_key)
        
    def outlier_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q4 = np.percentile(datacolumn , 100)
        except:
            return True
        return Q4
        
    def has_no_fix(self,word):
        if '[기재정정]' in word:
            return True
        else:
            return False

    def get_want_fast_report(self, a_s_list, report_dt):
        a_s_list = a_s_list[a_s_list['report_nm'].str.contains('영업(잠정)실적(공정공시)', regex = False)]
        a_s_list['rcept_dt'] = pd.to_datetime(a_s_list['rcept_dt'], format = '%Y%m%d').dt.date
        a_s_list = a_s_list[a_s_list['rcept_dt'] +  relativedelta(months = 1) > report_dt]
        a_s_list = a_s_list[a_s_list['rcept_dt'] - relativedelta(months = 3) < report_dt]
        a_s_list.reset_index(drop = True, inplace = True)
        return a_s_list

    def get_before_report_time(self, report_year, report_month):
        before_report_year = report_year-1
        before_report_month = report_month
        return before_report_year, before_report_month

    def get_earn_fast_earn(self, fast_report_list):
        if not self.check_lenz(fast_report_list) :
            try:
                REPORT_NO = str(fast_report_list.loc[0, 'rcept_no'])
            except:
                return 
            html_text = self.dart.document(REPORT_NO)
            DF_LIST = pd.read_html(html_text)
            for df_list in DF_LIST:
                '''if df_list[0].str.contains('※ 당해 정보는 확정치가 아닌 잠정치로 결산 결과 등에 따라 변경될 수 있음.
                ※ 동 정보는 확정치가 아닌 잠정치로서 향후 확정치와는 다를 수 있음.'''
                if df_list[0].str.contains('확정치가 아닌 잠정치', regex = False).any():
                    #df_list.set_index([0,1], inplace = True)
                    #if df_list[2].str.contains('당기이익').any():
                    if df_list[0].str.contains('지배기업', regex = False).any():
                        earn_name = '지배기업 소유주지분 순이익'
                    elif df_list[0].str.contains('당기순이익', regex = False).any():
                        earn_name = '당기순이익'
                    else:
                        after_earn = 0
                    if df_list[0].str.contains('계속영업', regex = False).any():
                        continue_name = '계속영업당기순이익'
                    after_continue_earn = df_list.loc[(continue_name,'당해실적'), 2].item()
                    after_earn = df_list.loc[(earn_name,'당해실적'), 2].item()
                    after_revenue = df_list.loc[('매출액','당해실적'), 2].item()
                    after_operating_profit = df_list.loc[('영업이익','당해실적'), 2].item()
                    if after_revenue == '-':
                        after_revenue =0
                    if after_operating_profit == '-':
                        after_operating_profit = 0
                    if after_earn == '-':
                        after_earn = 0
                    #print(after_earn, after_operating_profit)
                    return int(after_revenue)*(10**6), int(after_operating_profit)*(10**6), int(after_earn)*(10**6)
        return 
    
    def check_price_data(self, report_date):
        report_stock_date = date(2000,1,1)
        before_report_date = report_date - relativedelta(days = 14)
        now_report_date = report_date + relativedelta(days = 1)
        before_report_date = before_report_date.strftime("%Y-%m-%d")
        now_report_date = now_report_date.strftime("%Y-%m-%d")
        report_stock_price = 0
        df_stock_price = fdr.DataReader(self.SYMBOL, before_report_date, now_report_date)
        
        for i in range(0,14):
            report_stock_date = report_date - relativedelta(days = i)
            report_stock_date = report_stock_date.strftime("%Y-%m-%d")
            try:
                report_stock_price = df_stock_price.loc[report_stock_date, 'Close']
            except:
                continue
            if report_stock_price:
                break
            else:
                continue
        return int(report_stock_price)
    
    def get_top_last_year(self, last_year):
        now_last_year = str(last_year + 1) + '-03-30'
        before_last_year = str(last_year) + '-03-30'
        df_stock_price = fdr.DataReader(self.SYMBOL, before_last_year, now_last_year)
        price_column = df_stock_price[['Close']]
        top_price_last_year = self.outlier_treatment(price_column)
        return int(top_price_last_year)
    
    '''def give_revenue_data(self):
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        else:
            return'''
    
    def get_stock_amount(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        if report_month == 3:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[0])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 6:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[1])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 9:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[2])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 12:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[3])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        else:
            return

        
    def give_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의 소유주', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('귀속될', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        
    def give_operating_profit(self):
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit= self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit = self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return
        
    def give_revenue_data(self):
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_last_earn_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의 소유주', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('귀속될', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
    
   
    def give_last_revenue_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_last_operating_profit(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return
        
    def give_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return  
        
    def give_continue_earn(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def give_last_continue_earn(self):
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_cashflow(self):
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs-CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 
        
    def check_lenz(self, df):
        return df is None
    
    def check_empty(self, df):
        return df.empty
        
    def get_fs_of_time(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL , str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        if report_month == 6:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        else:
            return 
    
    def give_last_full_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의 소유주', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('귀속될', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return
    
    def give_last_full_revenue_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_last_full_operating_profit(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    '''def give_full_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_Assets')]).any():
            if ('IS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('IS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('CIS', 'ifrs-full_Assets')]).any():
            if ('CIS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('CIS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return
        
    def give_full_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_Liabilities')]).any():
            if ('IS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('IS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('CIS', 'ifrs-full_Liabilities')]).any():
            if ('CIS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('CIS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return
        
    def give_last_continue_earn(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        else:
            return 0'''
        
    def give_last_full_continue_earn(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def get_fs_of_last_full(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        LAST_FULL_EARN = self.give_last_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        LAST_FULL_EARN = self.give_last_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN
            #except:
                #pass
        else:
            return
        
    
    def get_real_report_month(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year
            report_month = 6
        if report_month == 6:
            report_year = report_year
            report_month = 9
        if report_month == 9:
            report_year = report_year
            report_month = 12
        if report_month == 12:
            report_year = report_year + 1
            report_month = 3
        return report_year, report_month
    
    def get_predict_earn_per_stock(self, report_month, earn_per_stock):
        earn_per_stock = int(earn_per_stock)
        
        if report_month == 3:
            earn_per_stock = earn_per_stock*4
        if report_month == 6:
            earn_per_stock = earn_per_stock*2
        if report_month == 9:
            earn_per_stock = earn_per_stock*(4/3)
        if report_month == 12:
            earn_per_stock = earn_per_stock
        
        return earn_per_stock
    
    def get_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_fast_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            #report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            #report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_last_year_report(self, report_date):
        delimiters = "-"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_date)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        last_year = int(nm_num_list[0])-1
        return last_year
    
    def get_only_number(self, report_data):
        delimiters = ","
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_data)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        element_amount = len(nm_num_list)
        report_data = 0
        for i in range(len(nm_num_list)):
            report_data = report_data + nm_num_list[i]*(10**((element_amount - i - 1)*3))
        return report_data
    
    def get_report_year_month(self, report_nm):
        delimiters = "(", ".", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_one_back(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year - 1
            report_month = 12
        if report_month == 6:
            report_year = report_year
            report_month = 3
        if report_month == 9:
            report_year = report_year
            report_month = 6
        if report_month == 12:
            report_year = report_year
            report_month = 9
        return report_year, report_month
            
    def get_symbol_six(self, analy_kosdaq, i):
        new_symbol = analy_kosdaq.loc[i, 'Symbol']
        if new_symbol != self.OLD_SYMBOL:
            self.OLD_COUNT = 0
            symbol = analy_kosdaq.loc[i, 'Symbol']
            symbol_length = self.digit_length(symbol)
            if symbol_length != 6:
                zero_need = 6 - symbol_length
                self.SYMBOL = '0'*zero_need + str(symbol)
            if symbol_length == 6:
                self.SYMBOL = str(symbol)
            self.OLD_SYMBOL = new_symbol
            return
        if new_symbol == self.OLD_SYMBOL:
            self.OLD_COUNT += 1
        return 
    
    #자릿수 찾기
    def digit_length(self, n):
        ans = 0
        n= int(n)

        while n:
            n //= 10
            ans += 1

        return ans


    def how_report(self, analyse_date, read_csv, write_csv,start,end):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(start,end):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            A_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, final = False)
            FAST_REPORT_LIST = self.get_want_fast_report(A_S_LIST, REPORT_DT)
            #print(FAST_REPORT_LIST)
            FAST_REPORT_NM, FAST_REPORT_DT = self.get_fast_report_date(FAST_REPORT_LIST)
            if not self.check_empty(FAST_REPORT_LIST):
                #print('true')
                #check if report is between real_report
                if REPORT_DT - relativedelta(months = 1) < FAST_REPORT_DT and REPORT_DT > FAST_REPORT_DT:
                    if True:
                    #try:
                        AFTER_REVENUE, AFTER_OPERATING_PROFIT, AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        #print(a_dictionary['REPORT_DATE'])
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    #except:
                        #pass
                    #try:
                        if BACK_REPORT_MONTH != 12:
                            QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                            #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                            a_dictionary['QUARTER_REVENUE'] = QUARTER_REVENUE_R
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = QUARTER_OPERATING_PROFIT_R
                            a_dictionary['QUARTER_CASHFLOW'] = QUARTER_CASHFLOW_R
                            a_dictionary['QUARTER_EARN'] = QUARTER_EARN_R
                            a_dictionary['QUARTER_CONTINUE_EARN'] = CONTINUE_EARN_R
                            a_dictionary['LAST_REVENUE'] = LAST_REVENUE_R
                            a_dictionary['LAST_OPERATING_PROFIT'] = LAST_OPERATING_PROFIT_R
                            a_dictionary['LAST_EARN'] = LAST_EARN_R
                            a_dictionary['LAST_CONTINUE_EARN'] = LAST_CONTINUE_EARN_R
                            a_dictionary['QUARTER_ASSET'] = QUARTER_ASSET_R
                            a_dictionary['QUARTER_LIABILITY'] = QUARTER_LIABILITY_R
                            #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    #except:
                        #pass

                if REPORT_DT + relativedelta(months = 4) > FAST_REPORT_DT and REPORT_DT < FAST_REPORT_DT:
                    if True:
                    #try:
                        AFTER_REVENUE, AFTER_OPERATING_PROFIT, AFTER_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        #print(a_dictionary['REPORT_DATE'])
                        REPORT_YEAR, REPORT_MONTH = self.get_real_report_month(REPORT_YEAR, REPORT_MONTH)
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    #except:
                        #pass
                    #try:
                        if BACK_REPORT_MONTH != 12:
                            QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                            #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                            a_dictionary['QUARTER_REVENUE'] = QUARTER_REVENUE_R
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = QUARTER_OPERATING_PROFIT_R
                            a_dictionary['QUARTER_CASHFLOW'] = QUARTER_CASHFLOW_R
                            a_dictionary['QUARTER_EARN'] = QUARTER_EARN_R
                            a_dictionary['QUARTER_CONTINUE_EARN'] = CONTINUE_EARN_R
                            a_dictionary['LAST_REVENUE'] = LAST_REVENUE_R
                            a_dictionary['LAST_OPERATING_PROFIT'] = LAST_OPERATING_PROFIT_R
                            a_dictionary['LAST_EARN'] = LAST_EARN_R
                            a_dictionary['LAST_CONTINUE_EARN'] = LAST_CONTINUE_EARN_R
                            a_dictionary['QUARTER_ASSET'] = QUARTER_ASSET_R
                            a_dictionary['QUARTER_LIABILITY'] = QUARTER_LIABILITY_R
                            #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    #except:
                        #pass
            else:
                if True:
                #try:
                    QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                    #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                    a_dictionary['QUARTER_REVENUE'] = QUARTER_REVENUE_R
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = QUARTER_OPERATING_PROFIT_R
                    a_dictionary['QUARTER_CASHFLOW'] = QUARTER_CASHFLOW_R
                    a_dictionary['QUARTER_EARN'] = QUARTER_EARN_R
                    a_dictionary['QUARTER_CONTINUE_EARN'] = CONTINUE_EARN_R
                    a_dictionary['LAST_REVENUE'] = LAST_REVENUE_R
                    a_dictionary['LAST_OPERATING_PROFIT'] = LAST_OPERATING_PROFIT_R
                    a_dictionary['LAST_EARN'] = LAST_EARN_R
                    a_dictionary['LAST_CONTINUE_EARN'] = LAST_CONTINUE_EARN_R
                    a_dictionary['QUARTER_ASSET'] = QUARTER_ASSET_R
                    a_dictionary['QUARTER_LIABILITY'] = QUARTER_LIABILITY_R
                    #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                    a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                    time.sleep(1)
                #except:
                    #pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            if True:
            #try:
                LAST_FULL_EARN_R, LAST_FULL_OPERATING_PROFIT_R, LAST_FULL_REVENUE_R, LAST_FULL_CONTINUE_EARN_R = self.get_fs_of_last_full(OLD_REPORT_YEAR, 12)
                a_dictionary['LAST_FULL_REVENUE'] = LAST_FULL_REVENUE_R
                a_dictionary['LAST_FULL_OPERATING_PROFIT'] = LAST_FULL_OPERATING_PROFIT_R
                a_dictionary['LAST_FULL_EARN'] = LAST_FULL_EARN_R
                a_dictionary['LAST_FULL_CONTINUE_EARN'] = LAST_FULL_CONTINUE_EARN_R
                time.sleep(1)
            #except:
                #pass
            #get stock amount
            #try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
            #except:
                #pass
            if True:
                if 'AFTER_REVENUE' in a_dictionary.keys():
                    if 'QUARTER_REVENUE' in a_dictionary.keys():
                        a_dictionary['QUARTER_REVENUE'] = int(a_dictionary['QUARTER_REVENUE']) + int(a_dictionary['AFTER_REVENUE'])
                        
                if 'AFTER_REVENUE' in a_dictionary.keys():
                    if 'QUARTER_REVENUE' not in a_dictionary.keys():
                        a_dictionary['QUARTER_REVENUE'] = int(a_dictionary['AFTER_REVENUE'])
                        
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = int(a_dictionary['QUARTER_OPERATING_PROFIT']) + int(a_dictionary['AFTER_OPERATING_PROFIT'])
                        
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' not in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = int(a_dictionary['AFTER_OPERATING_PROFIT'])
                        
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = int(a_dictionary['QUARTER_EARN']) + int(a_dictionary['AFTER_EARN'])
                
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' not in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = int(a_dictionary['AFTER_EARN'])
                if True:
                #try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EPS'] = int(int(a_dictionary['QUARTER_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
                #except:
                    #pass
                #try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'LAST_EARN' in a_dictionary.keys():
                            a_dictionary['LAST_EPS'] = int(int(a_dictionary['LAST_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
                #except:
                    #pass
                #try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'LAST_FULL_EARN' in a_dictionary.keys():
                            a_dictionary['REAL_LAST_FULL_EPS'] = int(int(a_dictionary['LAST_FULL_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
                #except:
                    #pass
                #try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'QUARTER_ASSET' in a_dictionary.keys():
                            if 'QUARTER_LIABILITY' in a_dictionary.keys():
                                a_dictionary['QUARTER_BPS'] = int((int(a_dictionary['QUARTER_ASSET'])-int(a_dictionary['QUARTER_LIABILITY']))/int(a_dictionary['STOCK_AMOUNT']))
                #except:
                    #pass
                if 'QUARTER_ASSET' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        QUARTER_FULL_EARN = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN']))
                        a_dictionary['QUARTER_ROA'] = round(QUARTER_FULL_EARN / int(a_dictionary['QUARTER_ASSET'])*100, 2)
                
                if 'QUARTER_REVENUE'in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT'] = round(int(a_dictionary['QUARTER_OPERATING_PROFIT'])/int(a_dictionary['QUARTER_REVENUE'])*100, 2)
                if 'LAST_REVENUE'in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_OPERATING_PROFIT'] = round(int(a_dictionary['LAST_OPERATING_PROFIT'])/int(a_dictionary['LAST_REVENUE'])*100, 2)
                if 'QUARTER_REVENUE'in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_EARN'] = round(int(a_dictionary['QUARTER_EARN'])/int(a_dictionary['QUARTER_REVENUE'])*100, 2)
                if 'LAST_REVENUE'in a_dictionary.keys():
                    if 'LAST_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_EARN'] = round(int(a_dictionary['LAST_EARN'])/int(a_dictionary['LAST_REVENUE'])*100, 2)
                if 'QUARTER_EPS' in a_dictionary.keys():
                    a_dictionary['QUARTER_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EPS']))
                '''if 'QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])'''
                
                if 'LAST_EPS' in a_dictionary.keys():
                    a_dictionary['FAKE_LAST_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['LAST_EPS']))   
                '''if 'OLD_QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_EARN'])
                if 'OLD_QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_OPERATING_PROFIT'])
                if 'OLD_QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_REVENUE'])'''
            #try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_CONTINUE_EARN'], a_dictionary['QUARTER_FULL_EPS'], a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT'], a_dictionary['QUARTER_PERCENT_EARN'], a_dictionary['LAST_REVENUE'], a_dictionary['LAST_OPERATING_PROFIT'], \
                a_dictionary['LAST_EARN'], a_dictionary['LAST_CONTINUE_EARN'], a_dictionary['FAKE_LAST_FULL_EPS'], a_dictionary['LAST_PERCENT_OPERATING_PROFIT'], a_dictionary['LAST_PERCENT_EARN'], a_dictionary['LAST_FULL_REVENUE'], a_dictionary['LAST_FULL_OPERATING_PROFIT'], a_dictionary['LAST_FULL_EARN'], a_dictionary['REAL_LAST_FULL_EPS'], a_dictionary['QUARTER_BPS'], a_dictionary['QUARTER_ROA']])
                print(self.OLD_SYMBOL)
                time.sleep(1)
            #except:
                #continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Quarter_Revenue', 'Quarter_Operating_Profit', 'Quarter_Earn', 'Quarter_Continue_Earn', 'Quarter_Full_EPS', 'Quarter_Percent_Operating_Profit', 'Quarter_Percent_Earn', 'Last_Revenue', 'Last_Operating_Profit', 'Last_Earn', \
                                                                    'Last_Continue_Earn', 'Fake_Last_Full_EPS', 'Last_Percent_Operating_Profit', 'Last_Percent_Earn', 'Last_Full_Revenue', 'Last_Full_Operating_Profit', 'Last_Full_Earn', 'Real_Last_Full_EPS', 'Quarter_BPS', 'Quarter_ROA'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
    
    def get_per(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            LAST_YEAR = self.get_last_year_report(REPORT_DATE.strftime("%Y-%m-%d"))
            LAST_STOCK_PRICE = self.get_top_last_year(LAST_YEAR)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #if 'LAST_STOCK_PRICE' in locals():
            LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Last_Earn_Per_Stock'])
            if LAST_EARN_PER_STOCK != 0:
                LAST_PER = int(LAST_STOCK_PRICE/LAST_EARN_PER_STOCK)
            if LAST_EARN_PER_STOCK == 0:
                LAST_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER), str(LAST_STOCK_PRICE), str(LAST_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price', 'Quarter_Per', 'Last_Stock_Price', 'Last_Per'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def how_report_sell(self, analyse_date, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            try:
                FULL_QUARTER_REVENUE, FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                time.sleep(1)
            except:
                pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
            except:
                pass
            if True:
                
                try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = int(a_dictionary['QUARTER_EARN']/a_dictionary['STOCK_AMOUNT'])
                except:
                    pass
            #try:
                if 'QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN_PER_STOCK'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN_PER_STOCK'])
                        a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                    #a_dictionary['PER'] = int(STOCK_PRICE/a_dictionary['QUARTER_EARN_PER_STOCK'])
            #except:
                #a_dictionary['PER'] = 0
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])
                    
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                #pass
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_EARN_PER_STOCK']])
                #print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date_Sell', 'Quarter_Revenue_Sell', 'Quarter_Operating_Profit_Sell', 'Quarter_Earn_Sell', 'Quarter_Earn_Per_Stock_Sell'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def get_per_sell(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I,'Report_Date_Sell']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price_Sell', 'Quarter_Per_Sell'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
#have 2 keys 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788', '6048f2c5a065e754cee104b8c0edd8186f6b3d61', '94331def3e94db8a1aabb84e4257cec0ad3b576b', 'b1fe4037e43a6db43d8ac280d262a05e90bf9171'
akaps = how_per('b1fe4037e43a6db43d8ac280d262a05e90bf9171')
#akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(7).csv')
akaps.how_report('2020-11-30','KOSDAQ.csv', 'A_PER_KOSDAQ_real(2020_6).csv',1,10)
#paka = how_per('84acf17c852e9430843a589191330e0630cc19a7')
#paka.how_report('2020-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ_real(2020_6)(1).csv',200,400)
#akaps = how_per('1eca9bad2afb8becdc5b1714112dd097e73da788')
#akaps.how_report('2020-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ_real(2020_6)(2).csv',400,600)
'''akaps.get_per('AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6).csv')
akaps.how_report_sell('2020-11-30', 'AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6_9).csv')
akaps.get_per_sell('AN_PER_KOSDAQ(2020_6_9).csv', 'AN_PER_KOSDAQ(2020_6_9).csv')

paka = how_per('84acf17c852e9430843a589191330e0630cc19a7')
paka.how_report_sell('2021-03-30', 'AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6_12).csv')
paka.get_per_sell('AN_PER_KOSDAQ(2020_6_12).csv', 'AN_PER_KOSDAQ(2020_6_12).csv')
#akaps.how_report('2020-08-30', 'AN_PER_KOSDAQ_real(2020_6)_test.csv','AN_PER_KOSDAQ_real(2020_6)_test(1).csv')'''

In [ ]:



#real_test




import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import datetime as dt 
from datetime import date
import time
from dateutil.relativedelta import relativedelta
import math
import warnings
import OpenDartReader
from pandas.core.common import SettingWithCopyWarning
import re
from collections.abc import Mapping

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)



class how_per():
    def __init__(self, api_key):
        self.api_key = api_key
        self.OLD_COUNT = 0
        self.OLD_SYMBOL = '가나다'
        self.SYMBOL = '가나다'
        self.TODAY = dt.date.today()
        self.BOOM_PER_LIST = []
        self.dart = OpenDartReader(self.api_key)
        
    def outlier_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q4 = np.percentile(datacolumn , 100)
        except:
            return True
        return Q4
        
    def has_no_fix(self,word):
        if '[기재정정]' in word:
            return True
        else:
            return False

    def get_want_fast_report(self, a_s_list, report_dt):
        a_s_list = a_s_list[a_s_list['report_nm'].str.contains('영업(잠정)실적(공정공시)', regex = False)]
        a_s_list['rcept_dt'] = pd.to_datetime(a_s_list['rcept_dt'], format = '%Y%m%d').dt.date
        a_s_list = a_s_list[a_s_list['rcept_dt'] +  relativedelta(months = 1) > report_dt]
        a_s_list = a_s_list[a_s_list['rcept_dt'] - relativedelta(months = 3) < report_dt]
        a_s_list.reset_index(drop = True, inplace = True)
        return a_s_list

    def get_before_report_time(self, report_year, report_month):
        before_report_year = report_year-1
        before_report_month = report_month
        return before_report_year, before_report_month

    def get_earn_fast_earn(self, fast_report_list):
        if not self.check_lenz(fast_report_list) :
            try:
                REPORT_NO = str(fast_report_list.loc[0, 'rcept_no'])
            except:
                return 
            html_text = self.dart.document(REPORT_NO)
            DF_LIST = pd.read_html(html_text)
            for df_list in DF_LIST:
                '''if df_list[0].str.contains('※ 당해 정보는 확정치가 아닌 잠정치로 결산 결과 등에 따라 변경될 수 있음.
                ※ 동 정보는 확정치가 아닌 잠정치로서 향후 확정치와는 다를 수 있음.'''
                if df_list[0].str.contains('확정치가 아닌 잠정치', regex = False).any():
                    df_list[0] = df_list[0].str.replace(" ","")
                    df_list[1] = df_list[1].str.replace(" ","")
                    #if df_list[2].str.contains('당기이익').any():
                    if df_list[0].str.contains('지배기업', regex = False).any():
                        earn_name = '지배기업소유주지분순이익'
                    elif df_list[0].str.contains('당기순이익', regex = False).any():
                        earn_name = '당기순이익'
                    else:
                        earn_name = None
                        after_earn = 0
                    if df_list[0].str.contains('계속영업', regex = False).any():
                        continue_name = '계속영업당기순이익'
                    df_list.set_index([0,1], inplace = True)
                    after_revenue = df_list.loc[('매출액','당해실적'), 2].item()
                    after_operating_profit = df_list.loc[('영업이익','당해실적'), 2].item()
                    try:
                        after_earn = df_list.loc[(earn_name,'당해실적'), 2].item()
                    except:
                        after_earn = 0
                    try:
                        after_continue_earn = df_list.loc[(continue_name,'당해실적'), 2].item()
                    except:
                        after_continue_earn = 0
                    before_revenue = df_list.loc[('매출액','당해실적'), 5].item()
                    before_operating_profit = df_list.loc[('영업이익','당해실적'), 5].item()
                    try:
                        before_earn = df_list.loc[(earn_name,'당해실적'), 5].item()
                    except:
                        before_earn = 0
                    try:
                        before_continue_earn = df_list.loc[(continue_name,'당해실적'), 5].item()
                    except:
                        before_continue_earn = 0
                    if after_revenue == '-':
                        after_revenue =0
                    if after_operating_profit == '-':
                        after_operating_profit = 0
                    if after_earn == '-':
                        after_earn = 0
                    if after_continue_earn == '-':
                        after_continue_earn = 0
                    if before_revenue == '-':
                        before_revenue =0
                    if before_operating_profit == '-':
                        before_operating_profit = 0
                    if before_earn == '-':
                        before_earn = 0
                    if before_continue_earn == '-':
                        before_continue_earn = 0
                    #print(after_earn, after_operating_profit)
                    return int(after_revenue)*(10**6), int(after_operating_profit)*(10**6), int(after_earn)*(10**6), int(after_continue_earn)*(10**6), int(before_revenue)*(10**6), int(before_operating_profit)*(10**6), int(before_earn)*(10**6), int(before_continue_earn)*(10**6) 
    
    def check_price_data(self, report_date):
        report_stock_date = date(2000,1,1)
        before_report_date = report_date - relativedelta(days = 14)
        now_report_date = report_date + relativedelta(days = 1)
        before_report_date = before_report_date.strftime("%Y-%m-%d")
        now_report_date = now_report_date.strftime("%Y-%m-%d")
        report_stock_price = 0
        df_stock_price = fdr.DataReader(self.SYMBOL, before_report_date, now_report_date)
        
        for i in range(0,14):
            report_stock_date = report_date - relativedelta(days = i)
            report_stock_date = report_stock_date.strftime("%Y-%m-%d")
            try:
                report_stock_price = df_stock_price.loc[report_stock_date, 'Close']
            except:
                continue
            if report_stock_price:
                break
            else:
                continue
        return int(report_stock_price)
    
    def get_top_last_year(self, last_year):
        now_last_year = str(last_year + 1) + '-03-30'
        before_last_year = str(last_year) + '-03-30'
        df_stock_price = fdr.DataReader(self.SYMBOL, before_last_year, now_last_year)
        price_column = df_stock_price[['Close']]
        top_price_last_year = self.outlier_treatment(price_column)
        return int(top_price_last_year)
    
    '''def give_revenue_data(self):
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        else:
            return'''
    
    def get_stock_amount(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        if report_month == 3:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[0])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 6:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[1])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 9:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[2])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 12:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[3])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        else:
            return

        
    def give_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의 소유주', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('귀속될', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        
    def give_operating_profit(self):
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit= self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit = self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return -int(quarter_operating_profit)
        else:
            return
        
    def give_revenue_data(self):
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_last_earn_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의 소유주', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('귀속될', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
    
   
    def give_last_revenue_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_last_operating_profit(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return -int(quarter_operating_profit)
        else:
            return
    
    def give_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return
        
    def give_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return  
        
    def give_continue_earn(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def give_last_continue_earn(self):
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_cashflow(self):
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs-CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 
        
    def check_lenz(self, df):
        return df is None
    
    def check_empty(self, df):
        return df.empty
        
    def get_fs_of_time(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL , str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        if report_month == 6:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        else:
            return 
    
    def give_last_full_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의 소유주', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('귀속될', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return
    
    def give_last_full_revenue_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_last_full_operating_profit(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    '''def give_full_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_Assets')]).any():
            if ('IS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('IS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('CIS', 'ifrs-full_Assets')]).any():
            if ('CIS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('CIS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return
        
    def give_full_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_Liabilities')]).any():
            if ('IS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('IS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('CIS', 'ifrs-full_Liabilities')]).any():
            if ('CIS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('CIS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return
        
    def give_last_continue_earn(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        else:
            return 0'''
        
    def give_last_full_continue_earn(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def get_fs_of_last_full(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        LAST_FULL_EARN = self.give_last_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        LAST_FULL_EARN = self.give_last_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN
            #except:
                #pass
        else:
            return
        
    
    def get_real_report_month(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year
            report_month = 6
        if report_month == 6:
            report_year = report_year
            report_month = 9
        if report_month == 9:
            report_year = report_year
            report_month = 12
        if report_month == 12:
            report_year = report_year + 1
            report_month = 3
        return report_year, report_month
    
    def get_predict_earn_per_stock(self, report_month, earn_per_stock):
        earn_per_stock = int(earn_per_stock)
        
        if report_month == 3:
            earn_per_stock = earn_per_stock*4
        if report_month == 6:
            earn_per_stock = earn_per_stock*2
        if report_month == 9:
            earn_per_stock = earn_per_stock*(4/3)
        if report_month == 12:
            earn_per_stock = earn_per_stock
        
        return earn_per_stock
    
    def get_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_fast_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            #report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            #report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_last_year_report(self, report_date):
        delimiters = "-"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_date)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        last_year = int(nm_num_list[0])-1
        return last_year
    
    def get_only_number(self, report_data):
        delimiters = ","
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_data)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        element_amount = len(nm_num_list)
        report_data = 0
        for i in range(len(nm_num_list)):
            report_data = report_data + nm_num_list[i]*(10**((element_amount - i - 1)*3))
        return report_data
    
    def get_report_year_month(self, report_nm):
        delimiters = "(", ".", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_one_back(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year - 1
            report_month = 12
        if report_month == 6:
            report_year = report_year
            report_month = 3
        if report_month == 9:
            report_year = report_year
            report_month = 6
        if report_month == 12:
            report_year = report_year
            report_month = 9
        return report_year, report_month
            
    def get_symbol_six(self, analy_kosdaq, i):
        new_symbol = analy_kosdaq.loc[i, 'Symbol']
        if new_symbol != self.OLD_SYMBOL:
            self.OLD_COUNT = 0
            symbol = analy_kosdaq.loc[i, 'Symbol']
            symbol_length = self.digit_length(symbol)
            if symbol_length != 6:
                zero_need = 6 - symbol_length
                self.SYMBOL = '0'*zero_need + str(symbol)
            if symbol_length == 6:
                self.SYMBOL = str(symbol)
            self.OLD_SYMBOL = new_symbol
            return
        if new_symbol == self.OLD_SYMBOL:
            self.OLD_COUNT += 1
        return 
    
    #자릿수 찾기
    def digit_length(self, n):
        ans = 0
        n= int(n)

        while n:
            n //= 10
            ans += 1

        return ans


    def how_report(self, analyse_date, read_csv, write_csv,start,end):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(start,end):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            A_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, final = False)
            FAST_REPORT_LIST = self.get_want_fast_report(A_S_LIST, REPORT_DT)
            #print(FAST_REPORT_LIST)
            FAST_REPORT_NM, FAST_REPORT_DT = self.get_fast_report_date(FAST_REPORT_LIST)
            if not self.check_empty(FAST_REPORT_LIST):
                #print('true')
                #check if report is between real_report
                if REPORT_DT - relativedelta(months = 1) < FAST_REPORT_DT and REPORT_DT > FAST_REPORT_DT:
                    #if True:
                    try:
                        AFTER_REVENUE, AFTER_OPERATING_PROFIT, AFTER_EARN, AFTER_CONTINUE_EARN, BEFORE_REVENUE, BEFORE_OPERATING_PROFIT, BEFORE_EARN, BEFORE_CONTINUE_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['AFTER_CONTINUE_EARN'] = AFTER_CONTINUE_EARN
                        a_dictionary['BEFORE_REVENUE'] = BEFORE_REVENUE
                        a_dictionary['BEFORE_OPERATING_PROFIT'] = BEFORE_OPERATING_PROFIT
                        a_dictionary['BEFORE_EARN'] = BEFORE_EARN
                        a_dictionary['BEFORE_CONTINUE_EARN'] = BEFORE_CONTINUE_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        #print(a_dictionary['REPORT_DATE'])
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    except:
                        pass
                    try:
                        if BACK_REPORT_MONTH != 12:
                            QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                            #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                            a_dictionary['QUARTER_REVENUE'] = QUARTER_REVENUE_R
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = QUARTER_OPERATING_PROFIT_R
                            a_dictionary['QUARTER_CASHFLOW'] = QUARTER_CASHFLOW_R
                            a_dictionary['QUARTER_EARN'] = QUARTER_EARN_R
                            a_dictionary['QUARTER_CONTINUE_EARN'] = CONTINUE_EARN_R
                            a_dictionary['LAST_REVENUE'] = LAST_REVENUE_R
                            a_dictionary['LAST_OPERATING_PROFIT'] = LAST_OPERATING_PROFIT_R
                            a_dictionary['LAST_EARN'] = LAST_EARN_R
                            a_dictionary['LAST_CONTINUE_EARN'] = LAST_CONTINUE_EARN_R
                            a_dictionary['QUARTER_ASSET'] = QUARTER_ASSET_R
                            a_dictionary['QUARTER_LIABILITY'] = QUARTER_LIABILITY_R
                            #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    except:
                        pass

                if REPORT_DT + relativedelta(months = 4) > FAST_REPORT_DT and REPORT_DT < FAST_REPORT_DT:
                    #if True:
                    try:
                        AFTER_REVENUE, AFTER_OPERATING_PROFIT, AFTER_EARN, AFTER_CONTINUE_EARN, BEFORE_REVENUE, BEFORE_OPERATING_PROFIT, BEFORE_EARN, BEFORE_CONTINUE_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['AFTER_CONTINUE_EARN'] = AFTER_CONTINUE_EARN
                        a_dictionary['BEFORE_REVENUE'] = BEFORE_REVENUE
                        a_dictionary['BEFORE_OPERATING_PROFIT'] = BEFORE_OPERATING_PROFIT
                        a_dictionary['BEFORE_EARN'] = BEFORE_EARN
                        a_dictionary['BEFORE_CONTINUE_EARN'] = BEFORE_CONTINUE_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        #print(a_dictionary['REPORT_DATE'])
                        REPORT_YEAR, REPORT_MONTH = self.get_real_report_month(REPORT_YEAR, REPORT_MONTH)
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    except:
                        pass
                    try:
                        if BACK_REPORT_MONTH != 12:
                            QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                            #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                            a_dictionary['QUARTER_REVENUE'] = QUARTER_REVENUE_R
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = QUARTER_OPERATING_PROFIT_R
                            a_dictionary['QUARTER_CASHFLOW'] = QUARTER_CASHFLOW_R
                            a_dictionary['QUARTER_EARN'] = QUARTER_EARN_R
                            a_dictionary['QUARTER_CONTINUE_EARN'] = CONTINUE_EARN_R
                            a_dictionary['LAST_REVENUE'] = LAST_REVENUE_R
                            a_dictionary['LAST_OPERATING_PROFIT'] = LAST_OPERATING_PROFIT_R
                            a_dictionary['LAST_EARN'] = LAST_EARN_R
                            a_dictionary['LAST_CONTINUE_EARN'] = LAST_CONTINUE_EARN_R
                            a_dictionary['QUARTER_ASSET'] = QUARTER_ASSET_R
                            a_dictionary['QUARTER_LIABILITY'] = QUARTER_LIABILITY_R
                            #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    except:
                        pass
            else:
                #if True:
                try:
                    QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                    #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                    a_dictionary['QUARTER_REVENUE'] = QUARTER_REVENUE_R
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = QUARTER_OPERATING_PROFIT_R
                    a_dictionary['QUARTER_CASHFLOW'] = QUARTER_CASHFLOW_R
                    a_dictionary['QUARTER_EARN'] = QUARTER_EARN_R
                    a_dictionary['QUARTER_CONTINUE_EARN'] = CONTINUE_EARN_R
                    a_dictionary['LAST_REVENUE'] = LAST_REVENUE_R
                    a_dictionary['LAST_OPERATING_PROFIT'] = LAST_OPERATING_PROFIT_R
                    a_dictionary['LAST_EARN'] = LAST_EARN_R
                    a_dictionary['LAST_CONTINUE_EARN'] = LAST_CONTINUE_EARN_R
                    a_dictionary['QUARTER_ASSET'] = QUARTER_ASSET_R
                    a_dictionary['QUARTER_LIABILITY'] = QUARTER_LIABILITY_R
                    #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                    a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                    time.sleep(1)
                except:
                    pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            try:
                LAST_FULL_EARN_R, LAST_FULL_OPERATING_PROFIT_R, LAST_FULL_REVENUE_R, LAST_FULL_CONTINUE_EARN_R = self.get_fs_of_last_full(OLD_REPORT_YEAR, 12)
                a_dictionary['LAST_FULL_REVENUE'] = LAST_FULL_REVENUE_R
                a_dictionary['LAST_FULL_OPERATING_PROFIT'] = LAST_FULL_OPERATING_PROFIT_R
                a_dictionary['LAST_FULL_EARN'] = LAST_FULL_EARN_R
                a_dictionary['LAST_FULL_CONTINUE_EARN'] = LAST_FULL_CONTINUE_EARN_R
                time.sleep(1)
            except:
                pass
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
                time.sleep(1)
            except:
                pass
            #if True:
            try:
                if 'AFTER_REVENUE' in a_dictionary.keys():
                    if 'QUARTER_REVENUE' in a_dictionary.keys():
                        a_dictionary['QUARTER_REVENUE'] = int(a_dictionary['QUARTER_REVENUE']) + int(a_dictionary['AFTER_REVENUE'])
               
                if 'AFTER_REVENUE' in a_dictionary.keys():
                    if 'QUARTER_REVENUE' not in a_dictionary.keys():
                        a_dictionary['QUARTER_REVENUE'] = int(a_dictionary['AFTER_REVENUE'])
            except:
                pass
            try:
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = int(a_dictionary['QUARTER_OPERATING_PROFIT']) + int(a_dictionary['AFTER_OPERATING_PROFIT'])
            
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' not in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = int(a_dictionary['AFTER_OPERATING_PROFIT'])
            except:
                pass
            try:
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = int(a_dictionary['QUARTER_EARN']) + int(a_dictionary['AFTER_EARN'])
            
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' not in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = int(a_dictionary['AFTER_EARN'])
            except:
                pass
            try:
                if 'AFTER_CONTINUE_EARN' in a_dictionary.keys():
                    if 'QUARTER_CONTINUE_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_CONTINUE_EARN'] = int(a_dictionary['QUARTER_CONTINUE_EARN']) + int(a_dictionary['AFTER_CONTINUE_EARN'])
            
                if 'AFTER_CONTINUE_EARN' in a_dictionary.keys():
                    if 'QUARTER_CONTINUE_EARN' not in a_dictionary.keys():
                        a_dictionary['QUARTER_CONTINUE_EARN'] = int(a_dictionary['AFTER_CONTINUE_EARN'])
            except:
                pass
            try:
                if 'BEFORE_REVENUE' in a_dictionary.keys():
                    if 'LAST_REVENUE' in a_dictionary.keys():
                        a_dictionary['LAST_REVENUE'] = int(a_dictionary['LAST_REVENUE']) + int(a_dictionary['BEFORE_REVENUE'])

                if 'BEFORE_REVENUE' in a_dictionary.keys():
                    if 'LAST_REVENUE' not in a_dictionary.keys():
                        a_dictionary['LAST_REVENUE'] = int(a_dictionary['BEFORE_REVENUE'])
            except:
                pass
            try:
                if 'BEFORE_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['LAST_OPERATING_PROFIT'] = int(a_dictionary['LAST_OPERATING_PROFIT']) + int(a_dictionary['BEFORE_OPERATING_PROFIT'])
        
                if 'BEFORE_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' not in a_dictionary.keys():
                        a_dictionary['LAST_OPERATING_PROFIT'] = int(a_dictionary['BEFORE_OPERATING_PROFIT'])
            except:
                pass
            try:
                if 'BEFORE_EARN' in a_dictionary.keys():
                    if 'LAST_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_EARN'] = int(a_dictionary['LAST_EARN']) + int(a_dictionary['BEFORE_EARN'])
                        
                if 'BEFORE_EARN' in a_dictionary.keys():
                    if 'LAST_EARN' not in a_dictionary.keys():
                        a_dictionary['LAST_EARN'] = int(a_dictionary['BEFORE_EARN'])
            except:
                pass
            try:
                if 'BEFORE_CONTINUE_EARN' in a_dictionary.keys():
                    if 'LAST_CONTINUE_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_CONTINUE_EARN'] = int(a_dictionary['LAST_CONTINUE_EARN']) + int(a_dictionary['BEFORE_CONTINUE_EARN'])

                if 'BEFORE_CONTINUE_EARN' in a_dictionary.keys():
                    if 'LAST_CONTINUE_EARN' not in a_dictionary.keys():
                        a_dictionary['LAST_CONTINUE_EARN'] = int(a_dictionary['BEFORE_CONTINUE_EARN'])
            except:
                pass
            #if True:
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EPS'] = int(int(a_dictionary['QUARTER_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_EPS'] = int(int(a_dictionary['LAST_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['REAL_LAST_FULL_EPS'] = int(int(a_dictionary['LAST_FULL_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'QUARTER_ASSET' in a_dictionary.keys():
                        if 'QUARTER_LIABILITY' in a_dictionary.keys():
                            a_dictionary['QUARTER_BPS'] = int((int(a_dictionary['QUARTER_ASSET'])-int(a_dictionary['QUARTER_LIABILITY']))/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'QUARTER_ASSET' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        QUARTER_FULL_EARN = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN']))
                        a_dictionary['QUARTER_ROA'] = round(QUARTER_FULL_EARN / int(a_dictionary['QUARTER_ASSET'])*100, 2)
            except:
                pass
            try:
                if 'QUARTER_REVENUE'in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT'] = round(int(a_dictionary['QUARTER_OPERATING_PROFIT'])/int(a_dictionary['QUARTER_REVENUE'])*100, 2)
                if 'LAST_REVENUE'in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_OPERATING_PROFIT'] = round(int(a_dictionary['LAST_OPERATING_PROFIT'])/int(a_dictionary['LAST_REVENUE'])*100, 2)
                if 'QUARTER_REVENUE'in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_EARN'] = round(int(a_dictionary['QUARTER_EARN'])/int(a_dictionary['QUARTER_REVENUE'])*100, 2)
                if 'LAST_REVENUE'in a_dictionary.keys():
                    if 'LAST_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_EARN'] = round(int(a_dictionary['LAST_EARN'])/int(a_dictionary['LAST_REVENUE'])*100, 2)
            except:
                pass
            try:
                if 'QUARTER_EPS' in a_dictionary.keys():
                    a_dictionary['QUARTER_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EPS']))
                '''if 'QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])'''
            except:
                pass
            try:
                if 'LAST_EPS' in a_dictionary.keys():
                    a_dictionary['FAKE_LAST_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['LAST_EPS']))   
                '''if 'OLD_QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_EARN'])
                if 'OLD_QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_OPERATING_PROFIT'])
                if 'OLD_QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_REVENUE'])'''
            except:
                pass
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_CONTINUE_EARN'], a_dictionary['QUARTER_FULL_EPS'], a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT'], a_dictionary['QUARTER_PERCENT_EARN'], a_dictionary['LAST_REVENUE'], a_dictionary['LAST_OPERATING_PROFIT'], \
                a_dictionary['LAST_EARN'], a_dictionary['LAST_CONTINUE_EARN'], a_dictionary['FAKE_LAST_FULL_EPS'], a_dictionary['LAST_PERCENT_OPERATING_PROFIT'], a_dictionary['LAST_PERCENT_EARN'], a_dictionary['LAST_FULL_REVENUE'], a_dictionary['LAST_FULL_OPERATING_PROFIT'], a_dictionary['LAST_FULL_EARN'], a_dictionary['REAL_LAST_FULL_EPS'], a_dictionary['QUARTER_BPS'], a_dictionary['QUARTER_ROA']])
                print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Quarter_Revenue', 'Quarter_Operating_Profit', 'Quarter_Earn', 'Quarter_Continue_Earn', 'Quarter_Full_EPS', 'Quarter_Percent_Operating_Profit', 'Quarter_Percent_Earn', 'Last_Revenue', 'Last_Operating_Profit', 'Last_Earn', \
                                                                    'Last_Continue_Earn', 'Fake_Last_Full_EPS', 'Last_Percent_Operating_Profit', 'Last_Percent_Earn', 'Last_Full_Revenue', 'Last_Full_Operating_Profit', 'Last_Full_Earn', 'Real_Last_Full_EPS', 'Quarter_BPS', 'Quarter_ROA'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
    
    def get_per(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            LAST_YEAR = self.get_last_year_report(REPORT_DATE.strftime("%Y-%m-%d"))
            LAST_STOCK_PRICE = self.get_top_last_year(LAST_YEAR)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #if 'LAST_STOCK_PRICE' in locals():
            LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Last_Earn_Per_Stock'])
            if LAST_EARN_PER_STOCK != 0:
                LAST_PER = int(LAST_STOCK_PRICE/LAST_EARN_PER_STOCK)
            if LAST_EARN_PER_STOCK == 0:
                LAST_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER), str(LAST_STOCK_PRICE), str(LAST_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price', 'Quarter_Per', 'Last_Stock_Price', 'Last_Per'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def how_report_sell(self, analyse_date, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            try:
                FULL_QUARTER_REVENUE, FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                time.sleep(1)
            except:
                pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
            except:
                pass
            #if True:
                
                try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = int(a_dictionary['QUARTER_EARN']/a_dictionary['STOCK_AMOUNT'])
                except:
                    pass
            #try:
                if 'QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN_PER_STOCK'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN_PER_STOCK'])
                        a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                    #a_dictionary['PER'] = int(STOCK_PRICE/a_dictionary['QUARTER_EARN_PER_STOCK'])
            #except:
                #a_dictionary['PER'] = 0
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])
                    
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                #pass
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_EARN_PER_STOCK']])
                #print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date_Sell', 'Quarter_Revenue_Sell', 'Quarter_Operating_Profit_Sell', 'Quarter_Earn_Sell', 'Quarter_Earn_Per_Stock_Sell'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def get_per_sell(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I,'Report_Date_Sell']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price_Sell', 'Quarter_Per_Sell'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
#have 2 keys 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788', '6048f2c5a065e754cee104b8c0edd8186f6b3d61', '94331def3e94db8a1aabb84e4257cec0ad3b576b', 'b1fe4037e43a6db43d8ac280d262a05e90bf9171'
akaps = how_per('94331def3e94db8a1aabb84e4257cec0ad3b576b')
#akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(7).csv')
akaps.how_report('2020-11-30','KOSDAQ.csv', 'A_PER_KOSDAQ_real(2020_9).csv',1,400)
paka = how_per('84acf17c852e9430843a589191330e0630cc19a7')
paka.how_report('2020-11-30','KOSDAQ.csv', 'A_PER_KOSDAQ_real(2020_6)(1).csv',400,800)
akaps = how_per('1eca9bad2afb8becdc5b1714112dd097e73da788')
akaps.how_report('2020-11-30','KOSDAQ.csv', 'A_PER_KOSDAQ_real(2020_6)(2).csv',800,1270)
'''akaps.get_per('AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6).csv')
akaps.how_report_sell('2020-11-30', 'AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6_9).csv')
akaps.get_per_sell('AN_PER_KOSDAQ(2020_6_9).csv', 'AN_PER_KOSDAQ(2020_6_9).csv')

paka = how_per('84acf17c852e9430843a589191330e0630cc19a7')
paka.how_report_sell('2021-03-30', 'AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6_12).csv')
paka.get_per_sell('AN_PER_KOSDAQ(2020_6_12).csv', 'AN_PER_KOSDAQ(2020_6_12).csv')
#akaps.how_report('2020-08-30', 'AN_PER_KOSDAQ_real(2020_6)_test.csv','AN_PER_KOSDAQ_real(2020_6)_test(1).csv')'''

In [ ]:



#real_test




import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import datetime as dt 
from datetime import date
import time
from dateutil.relativedelta import relativedelta
import math
import warnings
import OpenDartReader
from pandas.core.common import SettingWithCopyWarning
import re
from collections.abc import Mapping

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)



class how_per():
    def __init__(self, api_key):
        self.api_key = api_key
        self.OLD_COUNT = 0
        self.OLD_SYMBOL = '가나다'
        self.SYMBOL = '가나다'
        self.TODAY = dt.date.today()
        self.BOOM_PER_LIST = []
        self.dart = OpenDartReader(self.api_key)
        
    def outlier_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q4 = np.percentile(datacolumn , 100)
        except:
            return True
        return Q4
        
    def has_no_fix(self,word):
        if '[기재정정]' in word:
            return True
        else:
            return False

    def get_want_fast_report(self, a_s_list, report_dt):
        a_s_list = a_s_list[a_s_list['report_nm'].str.contains('영업(잠정)실적(공정공시)', regex = False)]
        a_s_list['rcept_dt'] = pd.to_datetime(a_s_list['rcept_dt'], format = '%Y%m%d').dt.date
        a_s_list = a_s_list[a_s_list['rcept_dt'] +  relativedelta(months = 1) > report_dt]
        a_s_list = a_s_list[a_s_list['rcept_dt'] - relativedelta(months = 3) < report_dt]
        a_s_list.reset_index(drop = True, inplace = True)
        return a_s_list

    def get_before_report_time(self, report_year, report_month):
        before_report_year = report_year-1
        before_report_month = report_month
        return before_report_year, before_report_month

    def get_earn_fast_earn(self, fast_report_list):
        if not self.check_lenz(fast_report_list) :
            try:
                REPORT_NO = str(fast_report_list.loc[0, 'rcept_no'])
            except:
                return 
            html_text = self.dart.document(REPORT_NO)
            DF_LIST = pd.read_html(html_text)
            for df_list in DF_LIST:
                '''if df_list[0].str.contains('※ 당해 정보는 확정치가 아닌 잠정치로 결산 결과 등에 따라 변경될 수 있음.
                ※ 동 정보는 확정치가 아닌 잠정치로서 향후 확정치와는 다를 수 있음.'''
                if df_list[0].str.contains('확정치가 아닌 잠정치', regex = False).any():
                    df_list[0] = df_list[0].str.replace(" ","")
                    df_list[1] = df_list[1].str.replace(" ","")
                    #if df_list[2].str.contains('당기이익').any():
                    if df_list[0].str.contains('지배기업', regex = False).any():
                        earn_name = '지배기업소유주지분순이익'
                    elif df_list[0].str.contains('당기순이익', regex = False).any():
                        earn_name = '당기순이익'
                    else:
                        earn_name = None
                        after_earn = 0
                    if df_list[0].str.contains('계속영업', regex = False).any():
                        continue_name = '계속영업당기순이익'
                    df_list.set_index([0,1], inplace = True)
                    after_revenue = df_list.loc[('매출액','당해실적'), 2].item()
                    after_operating_profit = df_list.loc[('영업이익','당해실적'), 2].item()
                    try:
                        after_earn = df_list.loc[(earn_name,'당해실적'), 2].item()
                    except:
                        after_earn = 0
                    try:
                        after_continue_earn = df_list.loc[(continue_name,'당해실적'), 2].item()
                    except:
                        after_continue_earn = 0
                    before_revenue = df_list.loc[('매출액','당해실적'), 5].item()
                    before_operating_profit = df_list.loc[('영업이익','당해실적'), 5].item()
                    try:
                        before_earn = df_list.loc[(earn_name,'당해실적'), 5].item()
                    except:
                        before_earn = 0
                    try:
                        before_continue_earn = df_list.loc[(continue_name,'당해실적'), 5].item()
                    except:
                        before_continue_earn = 0
                    if after_revenue == '-':
                        after_revenue =0
                    if after_operating_profit == '-':
                        after_operating_profit = 0
                    if after_earn == '-':
                        after_earn = 0
                    if after_continue_earn == '-':
                        after_continue_earn = 0
                    if before_revenue == '-':
                        before_revenue =0
                    if before_operating_profit == '-':
                        before_operating_profit = 0
                    if before_earn == '-':
                        before_earn = 0
                    if before_continue_earn == '-':
                        before_continue_earn = 0
                    #print(after_earn, after_operating_profit)
                    return int(after_revenue)*(10**6), int(after_operating_profit)*(10**6), int(after_earn)*(10**6), int(after_continue_earn)*(10**6), int(before_revenue)*(10**6), int(before_operating_profit)*(10**6), int(before_earn)*(10**6), int(before_continue_earn)*(10**6) 
    
    def check_price_data(self, report_date):
        report_stock_date = date(2000,1,1)
        before_report_date = report_date - relativedelta(days = 14)
        now_report_date = report_date + relativedelta(days = 1)
        before_report_date = before_report_date.strftime("%Y-%m-%d")
        now_report_date = now_report_date.strftime("%Y-%m-%d")
        report_stock_price = 0
        df_stock_price = fdr.DataReader(self.SYMBOL, before_report_date, now_report_date)
        
        for i in range(0,14):
            report_stock_date = report_date - relativedelta(days = i)
            report_stock_date = report_stock_date.strftime("%Y-%m-%d")
            try:
                report_stock_price = df_stock_price.loc[report_stock_date, 'Close']
            except:
                continue
            if report_stock_price:
                break
            else:
                continue
        return int(report_stock_price)
    
    def get_top_last_year(self, last_year):
        now_last_year = str(last_year + 1) + '-03-30'
        before_last_year = str(last_year) + '-03-30'
        df_stock_price = fdr.DataReader(self.SYMBOL, before_last_year, now_last_year)
        price_column = df_stock_price[['Close']]
        top_price_last_year = self.outlier_treatment(price_column)
        return int(top_price_last_year)
    
    '''def give_revenue_data(self):
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        else:
            return'''
    
    def get_stock_amount(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        if report_month == 3:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[0])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 6:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[1])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 9:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[2])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 12:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[3])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        else:
            return

        
    def give_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        self.report['account_nm'] = self.report['account_nm'].str.replace(" ","")
        if self.report['account_nm'].str.contains('지배기업의소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의소유주', regex = False)]
                    report = report[report['account_nm'].str.contains('귀속될', regex = False)]
                    report = report[report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        
    def give_operating_profit(self):
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit= self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit = self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return -int(quarter_operating_profit)
        else:
            return
        
    def give_revenue_data(self):
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_last_earn_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의 소유주', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('귀속될', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
    
   
    def give_last_revenue_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_last_operating_profit(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return -int(quarter_operating_profit)
        else:
            return
    
    def give_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return
        
    def give_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return  
        
    def give_continue_earn(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def give_last_continue_earn(self):
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_cashflow(self):
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs-CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 
        
    def check_lenz(self, df):
        return df is None
    
    def check_empty(self, df):
        return df.empty
        
    def get_fs_of_time(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL , str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        if report_month == 6:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        else:
            return 
    
    def give_last_full_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        self.report['account_nm'] = self.report['account_nm'].str.replace(" ","")
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의소유주', regex = False)]
                    report = report[report['account_nm'].str.contains('귀속될', regex = False)]
                    report = report[report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return
    
    def give_last_full_revenue_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_last_full_operating_profit(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    '''def give_full_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_Assets')]).any():
            if ('IS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('IS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('CIS', 'ifrs-full_Assets')]).any():
            if ('CIS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('CIS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return
        
    def give_full_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_Liabilities')]).any():
            if ('IS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('IS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('CIS', 'ifrs-full_Liabilities')]).any():
            if ('CIS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('CIS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return
        
    def give_last_continue_earn(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        else:
            return 0'''
        
    def give_last_full_continue_earn(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def get_fs_of_last_full(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        LAST_FULL_EARN = self.give_last_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        LAST_FULL_EARN = self.give_last_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN
            #except:
                #pass
        else:
            return
        
    
    def get_real_report_month(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year
            report_month = 6
        if report_month == 6:
            report_year = report_year
            report_month = 9
        if report_month == 9:
            report_year = report_year
            report_month = 12
        if report_month == 12:
            report_year = report_year + 1
            report_month = 3
        return report_year, report_month
    
    def get_predict_earn_per_stock(self, report_month, earn_per_stock):
        earn_per_stock = int(earn_per_stock)
        
        if report_month == 3:
            earn_per_stock = earn_per_stock*4
        if report_month == 6:
            earn_per_stock = earn_per_stock*2
        if report_month == 9:
            earn_per_stock = earn_per_stock*(4/3)
        if report_month == 12:
            earn_per_stock = earn_per_stock
        
        return earn_per_stock
    
    def get_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_fast_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            #report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            #report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_last_year_report(self, report_date):
        delimiters = "-"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_date)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        last_year = int(nm_num_list[0])-1
        return last_year
    
    def get_only_number(self, report_data):
        delimiters = ","
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_data)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        element_amount = len(nm_num_list)
        report_data = 0
        for i in range(len(nm_num_list)):
            report_data = report_data + nm_num_list[i]*(10**((element_amount - i - 1)*3))
        return report_data
    
    def get_report_year_month(self, report_nm):
        delimiters = "(", ".", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_one_back(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year - 1
            report_month = 12
        if report_month == 6:
            report_year = report_year
            report_month = 3
        if report_month == 9:
            report_year = report_year
            report_month = 6
        if report_month == 12:
            report_year = report_year
            report_month = 9
        return report_year, report_month
            
    def get_symbol_six(self, analy_kosdaq, i):
        new_symbol = analy_kosdaq.loc[i, 'Symbol']
        if new_symbol != self.OLD_SYMBOL:
            self.OLD_COUNT = 0
            symbol = analy_kosdaq.loc[i, 'Symbol']
            symbol_length = self.digit_length(symbol)
            if symbol_length != 6:
                zero_need = 6 - symbol_length
                self.SYMBOL = '0'*zero_need + str(symbol)
            if symbol_length == 6:
                self.SYMBOL = str(symbol)
            self.OLD_SYMBOL = new_symbol
            return
        if new_symbol == self.OLD_SYMBOL:
            self.OLD_COUNT += 1
        return 
    
    #자릿수 찾기
    def digit_length(self, n):
        ans = 0
        n= int(n)

        while n:
            n //= 10
            ans += 1

        return ans


    def how_report(self, analyse_date, read_csv, write_csv,start,end):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(start,end):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            A_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, final = False)
            FAST_REPORT_LIST = self.get_want_fast_report(A_S_LIST, REPORT_DT)
            #print(FAST_REPORT_LIST)
            FAST_REPORT_NM, FAST_REPORT_DT = self.get_fast_report_date(FAST_REPORT_LIST)
            if not self.check_empty(FAST_REPORT_LIST):
                #print('true')
                #check if report is between real_report
                if REPORT_DT - relativedelta(months = 1) < FAST_REPORT_DT and REPORT_DT > FAST_REPORT_DT:
                    if True:
                    #try:
                        AFTER_REVENUE, AFTER_OPERATING_PROFIT, AFTER_EARN, AFTER_CONTINUE_EARN, BEFORE_REVENUE, BEFORE_OPERATING_PROFIT, BEFORE_EARN, BEFORE_CONTINUE_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['AFTER_CONTINUE_EARN'] = AFTER_CONTINUE_EARN
                        a_dictionary['BEFORE_REVENUE'] = BEFORE_REVENUE
                        a_dictionary['BEFORE_OPERATING_PROFIT'] = BEFORE_OPERATING_PROFIT
                        a_dictionary['BEFORE_EARN'] = BEFORE_EARN
                        a_dictionary['BEFORE_CONTINUE_EARN'] = BEFORE_CONTINUE_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        #print(a_dictionary['REPORT_DATE'])
                        time.sleep(1)
                    #except:
                        #pass
                    #try:
                        if REPORT_MONTH != 12:
                            QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                            #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                            #print(QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R)
                            a_dictionary['QUARTER_REVENUE'] = QUARTER_REVENUE_R
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = QUARTER_OPERATING_PROFIT_R
                            a_dictionary['QUARTER_CASHFLOW'] = QUARTER_CASHFLOW_R
                            a_dictionary['QUARTER_EARN'] = QUARTER_EARN_R
                            a_dictionary['QUARTER_CONTINUE_EARN'] = CONTINUE_EARN_R
                            a_dictionary['LAST_REVENUE'] = LAST_REVENUE_R
                            a_dictionary['LAST_OPERATING_PROFIT'] = LAST_OPERATING_PROFIT_R
                            a_dictionary['LAST_EARN'] = LAST_EARN_R
                            a_dictionary['LAST_CONTINUE_EARN'] = LAST_CONTINUE_EARN_R
                            a_dictionary['QUARTER_ASSET'] = QUARTER_ASSET_R
                            a_dictionary['QUARTER_LIABILITY'] = QUARTER_LIABILITY_R
                            #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    #except:
                        #pass

                if REPORT_DT + relativedelta(months = 4) > FAST_REPORT_DT and REPORT_DT < FAST_REPORT_DT:
                    if True:
                    #try:
                        AFTER_REVENUE, AFTER_OPERATING_PROFIT, AFTER_EARN, AFTER_CONTINUE_EARN, BEFORE_REVENUE, BEFORE_OPERATING_PROFIT, BEFORE_EARN, BEFORE_CONTINUE_EARN = self.get_earn_fast_earn(FAST_REPORT_LIST)
                        #REPORT MONTH 되돌리기 YEAR도
                        a_dictionary['AFTER_REVENUE'] = AFTER_REVENUE
                        a_dictionary['AFTER_OPERATING_PROFIT'] = AFTER_OPERATING_PROFIT
                        a_dictionary['AFTER_EARN'] = AFTER_EARN
                        a_dictionary['AFTER_CONTINUE_EARN'] = AFTER_CONTINUE_EARN
                        a_dictionary['BEFORE_REVENUE'] = BEFORE_REVENUE
                        a_dictionary['BEFORE_OPERATING_PROFIT'] = BEFORE_OPERATING_PROFIT
                        a_dictionary['BEFORE_EARN'] = BEFORE_EARN
                        a_dictionary['BEFORE_CONTINUE_EARN'] = BEFORE_CONTINUE_EARN
                        a_dictionary['REPORT_DATE'] = FAST_REPORT_DT.strftime("%Y-%m-%d")
                        #print(a_dictionary['REPORT_DATE'])
                        REPORT_YEAR, REPORT_MONTH = self.get_real_report_month(REPORT_YEAR, REPORT_MONTH)
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    #except:
                        #pass
                    #try:
                        if BACK_REPORT_MONTH != 12:
                            QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                            #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                            a_dictionary['QUARTER_REVENUE'] = QUARTER_REVENUE_R
                            a_dictionary['QUARTER_OPERATING_PROFIT'] = QUARTER_OPERATING_PROFIT_R
                            a_dictionary['QUARTER_CASHFLOW'] = QUARTER_CASHFLOW_R
                            a_dictionary['QUARTER_EARN'] = QUARTER_EARN_R
                            a_dictionary['QUARTER_CONTINUE_EARN'] = CONTINUE_EARN_R
                            a_dictionary['LAST_REVENUE'] = LAST_REVENUE_R
                            a_dictionary['LAST_OPERATING_PROFIT'] = LAST_OPERATING_PROFIT_R
                            a_dictionary['LAST_EARN'] = LAST_EARN_R
                            a_dictionary['LAST_CONTINUE_EARN'] = LAST_CONTINUE_EARN_R
                            a_dictionary['QUARTER_ASSET'] = QUARTER_ASSET_R
                            a_dictionary['QUARTER_LIABILITY'] = QUARTER_LIABILITY_R
                            #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                            time.sleep(1)
                    #except:
                        #pass
            else:
                if True:
                #try:
                    QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                    #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                    a_dictionary['QUARTER_REVENUE'] = QUARTER_REVENUE_R
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = QUARTER_OPERATING_PROFIT_R
                    a_dictionary['QUARTER_CASHFLOW'] = QUARTER_CASHFLOW_R
                    a_dictionary['QUARTER_EARN'] = QUARTER_EARN_R
                    a_dictionary['QUARTER_CONTINUE_EARN'] = CONTINUE_EARN_R
                    a_dictionary['LAST_REVENUE'] = LAST_REVENUE_R
                    a_dictionary['LAST_OPERATING_PROFIT'] = LAST_OPERATING_PROFIT_R
                    a_dictionary['LAST_EARN'] = LAST_EARN_R
                    a_dictionary['LAST_CONTINUE_EARN'] = LAST_CONTINUE_EARN_R
                    a_dictionary['QUARTER_ASSET'] = QUARTER_ASSET_R
                    a_dictionary['QUARTER_LIABILITY'] = QUARTER_LIABILITY_R
                    #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                    a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                    time.sleep(1)
                #except:
                    #pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            if True:
            #try:
                LAST_FULL_EARN_R, LAST_FULL_OPERATING_PROFIT_R, LAST_FULL_REVENUE_R, LAST_FULL_CONTINUE_EARN_R = self.get_fs_of_last_full(OLD_REPORT_YEAR, 12)
                a_dictionary['LAST_FULL_REVENUE'] = LAST_FULL_REVENUE_R
                a_dictionary['LAST_FULL_OPERATING_PROFIT'] = LAST_FULL_OPERATING_PROFIT_R
                a_dictionary['LAST_FULL_EARN'] = LAST_FULL_EARN_R
                a_dictionary['LAST_FULL_CONTINUE_EARN'] = LAST_FULL_CONTINUE_EARN_R
                time.sleep(1)
            #except:
                #pass
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
                time.sleep(1)
            except:
                pass
            if True:
            #try:
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_CASHFLOW' in a_dictionary.keys():
                        a_dictionary['QUARTER_CASHFLOW'] = int(a_dictionary['QUARTER_CASHFLOW']) + int(a_dictionary['AFTER_OPERATING_PROFIT'])
               
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_CASHFLOW' not in a_dictionary.keys():
                        a_dictionary['QUARTER_CASHFLOW'] = int(a_dictionary['AFTER_OPERATING_PROFIT'])
            #except:
                #pass
            #try:
                if 'AFTER_REVENUE' in a_dictionary.keys():
                    if 'QUARTER_REVENUE' in a_dictionary.keys():
                        a_dictionary['QUARTER_REVENUE'] = int(a_dictionary['QUARTER_REVENUE']) + int(a_dictionary['AFTER_REVENUE'])
               
                if 'AFTER_REVENUE' in a_dictionary.keys():
                    if 'QUARTER_REVENUE' not in a_dictionary.keys():
                        a_dictionary['QUARTER_REVENUE'] = int(a_dictionary['AFTER_REVENUE'])
            #except:
                #pass
            #try:
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = int(a_dictionary['QUARTER_OPERATING_PROFIT']) + int(a_dictionary['AFTER_OPERATING_PROFIT'])
            
                if 'AFTER_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' not in a_dictionary.keys():
                        a_dictionary['QUARTER_OPERATING_PROFIT'] = int(a_dictionary['AFTER_OPERATING_PROFIT'])
            #except:
                #pass
            #try:
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = int(a_dictionary['QUARTER_EARN']) + int(a_dictionary['AFTER_EARN'])
            
                if 'AFTER_EARN' in a_dictionary.keys():
                    if 'QUARTER_EARN' not in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN'] = int(a_dictionary['AFTER_EARN'])
            #except:
                #pass
            #try:
                if 'AFTER_CONTINUE_EARN' in a_dictionary.keys():
                    if 'QUARTER_CONTINUE_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_CONTINUE_EARN'] = int(a_dictionary['QUARTER_CONTINUE_EARN']) + int(a_dictionary['AFTER_CONTINUE_EARN'])
            
                if 'AFTER_CONTINUE_EARN' in a_dictionary.keys():
                    if 'QUARTER_CONTINUE_EARN' not in a_dictionary.keys():
                        a_dictionary['QUARTER_CONTINUE_EARN'] = int(a_dictionary['AFTER_CONTINUE_EARN'])
            #except:
                #pass
            #try:
                if 'BEFORE_REVENUE' in a_dictionary.keys():
                    if 'LAST_REVENUE' in a_dictionary.keys():
                        a_dictionary['LAST_REVENUE'] = int(a_dictionary['LAST_REVENUE']) + int(a_dictionary['BEFORE_REVENUE'])

                if 'BEFORE_REVENUE' in a_dictionary.keys():
                    if 'LAST_REVENUE' not in a_dictionary.keys():
                        a_dictionary['LAST_REVENUE'] = int(a_dictionary['BEFORE_REVENUE'])
            #except:
                #pass
            #try:
                if 'BEFORE_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['LAST_OPERATING_PROFIT'] = int(a_dictionary['LAST_OPERATING_PROFIT']) + int(a_dictionary['BEFORE_OPERATING_PROFIT'])
        
                if 'BEFORE_OPERATING_PROFIT' in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' not in a_dictionary.keys():
                        a_dictionary['LAST_OPERATING_PROFIT'] = int(a_dictionary['BEFORE_OPERATING_PROFIT'])
            #except:
                #pass
            #try:
                if 'BEFORE_EARN' in a_dictionary.keys():
                    if 'LAST_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_EARN'] = int(a_dictionary['LAST_EARN']) + int(a_dictionary['BEFORE_EARN'])
                        
                if 'BEFORE_EARN' in a_dictionary.keys():
                    if 'LAST_EARN' not in a_dictionary.keys():
                        a_dictionary['LAST_EARN'] = int(a_dictionary['BEFORE_EARN'])
            #except:
                #pass
            #try:
                if 'BEFORE_CONTINUE_EARN' in a_dictionary.keys():
                    if 'LAST_CONTINUE_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_CONTINUE_EARN'] = int(a_dictionary['LAST_CONTINUE_EARN']) + int(a_dictionary['BEFORE_CONTINUE_EARN'])

                if 'BEFORE_CONTINUE_EARN' in a_dictionary.keys():
                    if 'LAST_CONTINUE_EARN' not in a_dictionary.keys():
                        a_dictionary['LAST_CONTINUE_EARN'] = int(a_dictionary['BEFORE_CONTINUE_EARN'])
            #except:
                #pass
            if True:
            #try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EPS'] = int(int(a_dictionary['QUARTER_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
            #except:
                #pass
            #try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_EPS'] = int(int(a_dictionary['LAST_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
            #except:
                #pass
            #try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['REAL_LAST_FULL_EPS'] = int(int(a_dictionary['LAST_FULL_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
            #except:
                #pass
            #try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'QUARTER_ASSET' in a_dictionary.keys():
                        if 'QUARTER_LIABILITY' in a_dictionary.keys():
                            a_dictionary['QUARTER_BPS'] = int((int(a_dictionary['QUARTER_ASSET'])-int(a_dictionary['QUARTER_LIABILITY']))/int(a_dictionary['STOCK_AMOUNT']))
            #except:
                #pass
            #try:
                if 'QUARTER_ASSET' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        QUARTER_FULL_EARN = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN']))
                        a_dictionary['QUARTER_ROA'] = round(QUARTER_FULL_EARN / int(a_dictionary['QUARTER_ASSET'])*100, 2)
            #except:
                #pass
            #try:
                if 'QUARTER_REVENUE'in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT'] = round(int(a_dictionary['QUARTER_OPERATING_PROFIT'])/int(a_dictionary['QUARTER_REVENUE'])*100, 2)
                if 'LAST_REVENUE'in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_OPERATING_PROFIT'] = round(int(a_dictionary['LAST_OPERATING_PROFIT'])/int(a_dictionary['LAST_REVENUE'])*100, 2)
                if 'QUARTER_REVENUE'in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_EARN'] = round(int(a_dictionary['QUARTER_EARN'])/int(a_dictionary['QUARTER_REVENUE'])*100, 2)
                if 'LAST_REVENUE'in a_dictionary.keys():
                    if 'LAST_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_EARN'] = round(int(a_dictionary['LAST_EARN'])/int(a_dictionary['LAST_REVENUE'])*100, 2)
            #except:
                #pass
            try:
                if 'QUARTER_EPS' in a_dictionary.keys():
                    a_dictionary['QUARTER_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EPS']))
                '''if 'QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])'''
            except:
                pass
            try:
                if 'LAST_EPS' in a_dictionary.keys():
                    a_dictionary['FAKE_LAST_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['LAST_EPS']))   
                '''if 'OLD_QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_EARN'])
                if 'OLD_QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_OPERATING_PROFIT'])
                if 'OLD_QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_REVENUE'])'''
            except:
                pass
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_CONTINUE_EARN'], a_dictionary['QUARTER_CASHFLOW'], a_dictionary['QUARTER_FULL_EPS'], a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT'], a_dictionary['QUARTER_PERCENT_EARN'], a_dictionary['LAST_REVENUE'], a_dictionary['LAST_OPERATING_PROFIT'], \
                a_dictionary['LAST_EARN'], a_dictionary['LAST_CONTINUE_EARN'], a_dictionary['FAKE_LAST_FULL_EPS'], a_dictionary['LAST_PERCENT_OPERATING_PROFIT'], a_dictionary['LAST_PERCENT_EARN'], a_dictionary['LAST_FULL_REVENUE'], a_dictionary['LAST_FULL_OPERATING_PROFIT'], a_dictionary['LAST_FULL_EARN'], a_dictionary['REAL_LAST_FULL_EPS'], a_dictionary['QUARTER_BPS'], a_dictionary['QUARTER_ROA']])
                print(self.OLD_SYMBOL)
                #print(self.BOOM_PER_LIST)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Quarter_Revenue', 'Quarter_Operating_Profit', 'Quarter_Earn', 'Quarter_Continue_Earn', 'Quarter_Cashflow', 'Quarter_Full_EPS', 'Quarter_Percent_Operating_Profit', 'Quarter_Percent_Earn', 'Last_Revenue', 'Last_Operating_Profit', 'Last_Earn', \
                                                                    'Last_Continue_Earn', 'Fake_Last_Full_EPS', 'Last_Percent_Operating_Profit', 'Last_Percent_Earn', 'Last_Full_Revenue', 'Last_Full_Operating_Profit', 'Last_Full_Earn', 'Real_Last_Full_EPS', 'Quarter_BPS', 'Quarter_ROA'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
    
    def get_per(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            LAST_YEAR = self.get_last_year_report(REPORT_DATE.strftime("%Y-%m-%d"))
            LAST_STOCK_PRICE = self.get_top_last_year(LAST_YEAR)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #if 'LAST_STOCK_PRICE' in locals():
            LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Last_Earn_Per_Stock'])
            if LAST_EARN_PER_STOCK != 0:
                LAST_PER = int(LAST_STOCK_PRICE/LAST_EARN_PER_STOCK)
            if LAST_EARN_PER_STOCK == 0:
                LAST_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER), str(LAST_STOCK_PRICE), str(LAST_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price', 'Quarter_Per', 'Last_Stock_Price', 'Last_Per'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def how_report_sell(self, analyse_date, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            try:
                FULL_QUARTER_REVENUE, FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                time.sleep(1)
            except:
                pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
            except:
                pass
            #if True:
                
                try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = int(a_dictionary['QUARTER_EARN']/a_dictionary['STOCK_AMOUNT'])
                except:
                    pass
            #try:
                if 'QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN_PER_STOCK'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN_PER_STOCK'])
                        a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                    #a_dictionary['PER'] = int(STOCK_PRICE/a_dictionary['QUARTER_EARN_PER_STOCK'])
            #except:
                #a_dictionary['PER'] = 0
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])
                    
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                #pass
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_EARN_PER_STOCK']])
                print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date_Sell', 'Quarter_Revenue_Sell', 'Quarter_Operating_Profit_Sell', 'Quarter_Earn_Sell', 'Quarter_Earn_Per_Stock_Sell'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def get_per_sell(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I,'Report_Date_Sell']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price_Sell', 'Quarter_Per_Sell'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
#have 2 keys 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788', '6048f2c5a065e754cee104b8c0edd8186f6b3d61', '94331def3e94db8a1aabb84e4257cec0ad3b576b', 'b1fe4037e43a6db43d8ac280d262a05e90bf9171'
akaps = how_per('94331def3e94db8a1aabb84e4257cec0ad3b576b')
#akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(7).csv')
akaps.how_report('2020-11-30','A_PER_KOSDAQ_TEST(2020_9).csv', 'A_PER_KOSDAQ_TEST(2020_9)(1).csv',52,53)
'''paka = how_per('84acf17c852e9430843a589191330e0630cc19a7')
paka.how_report('2020-11-30','KOSDAQ.csv', 'A_PER_KOSDAQ_real(2020_6)(1).csv',400,800)
akaps = how_per('1eca9bad2afb8becdc5b1714112dd097e73da788')
akaps.how_report('2020-11-30','KOSDAQ.csv', 'A_PER_KOSDAQ_real(2020_6)(2).csv',800,1270)'''
'''akaps.get_per('AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6).csv')
akaps.how_report_sell('2020-11-30', 'AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6_9).csv')
akaps.get_per_sell('AN_PER_KOSDAQ(2020_6_9).csv', 'AN_PER_KOSDAQ(2020_6_9).csv')

paka = how_per('84acf17c852e9430843a589191330e0630cc19a7')
paka.how_report_sell('2021-03-30', 'AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6_12).csv')
paka.get_per_sell('AN_PER_KOSDAQ(2020_6_12).csv', 'AN_PER_KOSDAQ(2020_6_12).csv')
#akaps.how_report('2020-08-30', 'AN_PER_KOSDAQ_real(2020_6)_test.csv','AN_PER_KOSDAQ_real(2020_6)_test(1).csv')'''

In [ ]:



#real_test




import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import datetime as dt 
from datetime import date
import time
from dateutil.relativedelta import relativedelta
import math
import warnings
import OpenDartReader
from pandas.core.common import SettingWithCopyWarning
import re
from collections.abc import Mapping

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)



class how_per():
    def __init__(self, api_key):
        self.api_key = api_key
        self.OLD_COUNT = 0
        self.OLD_SYMBOL = '가나다'
        self.SYMBOL = '가나다'
        self.TODAY = dt.date.today()
        self.BOOM_PER_LIST = []
        self.dart = OpenDartReader(self.api_key)
        
    def outlier_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q4 = np.percentile(datacolumn , 100)
        except:
            return True
        return Q4
    
    def big_small_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q1, Q2= np.percentile(datacolumn ,[10,90])
        except:
            return True
        return Q1, Q2
    
    def has_no_fix(self,word):
        if '[기재정정]' in word:
            return True
        else:
            return False

    def get_want_fast_report(self, a_s_list, report_dt):
        a_s_list = a_s_list[a_s_list['report_nm'].str.contains('영업(잠정)실적(공정공시)', regex = False)]
        a_s_list['rcept_dt'] = pd.to_datetime(a_s_list['rcept_dt'], format = '%Y%m%d').dt.date
        a_s_list = a_s_list[a_s_list['rcept_dt'] +  relativedelta(months = 1) > report_dt]
        a_s_list = a_s_list[a_s_list['rcept_dt'] - relativedelta(months = 3) < report_dt]
        a_s_list.reset_index(drop = True, inplace = True)
        return a_s_list

    def get_before_report_time(self, report_year, report_month):
        before_report_year = report_year-1
        before_report_month = report_month
        return before_report_year, before_report_month
    
    def check_report_month_price_data(self, report_date):
        report_plus_date = report_date + relativedelta(days = 10)
        report_minus_date = report_date - relativedelta(days = 10)
        report_plus_date = report_plus_date.strftime("%Y-%m-%d")
        report_minus_date = report_minus_date.strftime("%Y-%m-%d")
        df_stock_price = fdr.DataReader(self.SYMBOL, report_minus_date, report_plus_date)
        close_price_column = df_stock_price[['Close']]
        low_price, high_price = self.low_high_treatment(close_price_column)
        percent_change = round((int(high_price)-int(low_price))/int(low_price)*100,2)
        return percent_change
    
    def check_price_data(self, report_date):
        report_stock_date = date(2000,1,1)
        before_report_date = report_date - relativedelta(days = 14)
        now_report_date = report_date + relativedelta(days = 1)
        before_report_date = before_report_date.strftime("%Y-%m-%d")
        now_report_date = now_report_date.strftime("%Y-%m-%d")
        report_stock_price = 0
        df_stock_price = fdr.DataReader(self.SYMBOL, before_report_date, now_report_date)
        
        for i in range(0,14):
            report_stock_date = report_date - relativedelta(days = i)
            report_stock_date = report_stock_date.strftime("%Y-%m-%d")
            try:
                report_stock_price = df_stock_price.loc[report_stock_date, 'Close']
            except:
                continue
            if report_stock_price:
                break
            else:
                continue
        return int(report_stock_price)
    
    def get_top_last_year(self, last_year):
        now_last_year = str(last_year + 1) + '-03-30'
        before_last_year = str(last_year) + '-03-30'
        df_stock_price = fdr.DataReader(self.SYMBOL, before_last_year, now_last_year)
        price_column = df_stock_price[['Close']]
        top_price_last_year = self.outlier_treatment(price_column)
        return int(top_price_last_year)
    
    '''def give_revenue_data(self):
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        else:
            return'''
    
    def get_stock_amount(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        if report_month == 3:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[0])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 6:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[1])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 9:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[2])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 12:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[3])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        else:
            return

        
    def give_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        self.report['account_nm'] = self.report['account_nm'].str.replace(" ","")
        if self.report['account_nm'].str.contains('지배기업의소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의소유주', regex = False)]
                    report = report[report['account_nm'].str.contains('귀속될', regex = False)]
                    report = report[report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        
    def give_operating_profit(self):
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit= self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit = self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return -int(quarter_operating_profit)
        else:
            return
        
    def give_revenue_data(self):
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_last_earn_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의 소유주', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('귀속될', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
    
   
    def give_last_revenue_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_last_operating_profit(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return -int(quarter_operating_profit)
        else:
            return
    
    def give_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return
        
    def give_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return 
        
    def give_continue_earn(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def give_last_continue_earn(self):
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_cashflow(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs-CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 
    
    def give_last_cashflow(self):
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs-CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-CashFlowsFromUsedInOperatingActivities'), 'frmtrm_q_amount']
                return int(quarter_earn)
        else:
            return 
        
    def check_lenz(self, df):
        return df is None
    
    def check_empty(self, df):
        return df.empty
        
    def get_fs_of_time(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL , str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        if report_month == 6:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT,LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT,LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT,LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT,LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT,LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY
            #except:
                #pass
        else:
            return 
    
    def give_last_full_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        self.report['account_nm'] = self.report['account_nm'].str.replace(" ","")
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의소유주', regex = False)]
                    report = report[report['account_nm'].str.contains('귀속될', regex = False)]
                    report = report[report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return
    
    def give_last_full_revenue_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return
    
    def give_last_full_operating_profit(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return
    
     
    
    def give_last_full_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return
        
    def give_last_full_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return  
        
    def give_last_full_continue_earn(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_last_full_cashflow(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs-CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 
        
    def get_fs_of_last_full(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        LAST_FULL_EARN = self.give_last_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        LAST_FULL_CASHFLOW = self.give_last_full_cashflow()
                        LAST_FULL_ASSET = self.give_last_full_asset()
                        LAST_FULL_LIABILITY = self.give_last_full_liability()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_CASHFLOW, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN, LAST_FULL_ASSET, LAST_FULL_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        LAST_FULL_EARN = self.give_last_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        LAST_FULL_CASHFLOW = self.give_last_full_cashflow()
                        LAST_FULL_ASSET = self.give_last_full_asset()
                        LAST_FULL_LIABILITY = self.give_last_full_liability()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_CASHFLOW, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN, LAST_FULL_ASSET, LAST_FULL_LIABILITY
            #except:
                #pass
        else:
            return
        
    
    def get_real_report_month(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year
            report_month = 6
        if report_month == 6:
            report_year = report_year
            report_month = 9
        if report_month == 9:
            report_year = report_year
            report_month = 12
        if report_month == 12:
            report_year = report_year + 1
            report_month = 3
        return report_year, report_month
    
    def get_predict_earn_per_stock(self, report_month, earn_per_stock):
        earn_per_stock = int(earn_per_stock)
        
        if report_month == 3:
            earn_per_stock = earn_per_stock*4
        if report_month == 6:
            earn_per_stock = earn_per_stock*2
        if report_month == 9:
            earn_per_stock = earn_per_stock*(4/3)
        if report_month == 12:
            earn_per_stock = earn_per_stock
        
        return earn_per_stock
    
    def get_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_fast_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            #report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            #report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_last_year_report(self, report_date):
        delimiters = "-"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_date)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        last_year = int(nm_num_list[0])-1
        return last_year
    
    def get_only_number(self, report_data):
        delimiters = ","
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_data)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        element_amount = len(nm_num_list)
        report_data = 0
        for i in range(len(nm_num_list)):
            report_data = report_data + nm_num_list[i]*(10**((element_amount - i - 1)*3))
        return report_data
    
    def get_report_year_month(self, report_nm):
        delimiters = "(", ".", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_one_back(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year - 1
            report_month = 12
        if report_month == 6:
            report_year = report_year
            report_month = 3
        if report_month == 9:
            report_year = report_year
            report_month = 6
        if report_month == 12:
            report_year = report_year
            report_month = 9
        return report_year, report_month
            
    def get_symbol_six(self, analy_kosdaq, i):
        new_symbol = analy_kosdaq.loc[i, 'Symbol']
        if new_symbol != self.OLD_SYMBOL:
            self.OLD_COUNT = 0
            symbol = analy_kosdaq.loc[i, 'Symbol']
            symbol_length = self.digit_length(symbol)
            if symbol_length != 6:
                zero_need = 6 - symbol_length
                self.SYMBOL = '0'*zero_need + str(symbol)
            if symbol_length == 6:
                self.SYMBOL = str(symbol)
            self.OLD_SYMBOL = new_symbol
            return
        if new_symbol == self.OLD_SYMBOL:
            self.OLD_COUNT += 1
        return 
    
    #자릿수 찾기
    def digit_length(self, n):
        ans = 0
        n= int(n)

        while n:
            n //= 10
            ans += 1

        return ans


    def how_report(self, analyse_date, read_csv, write_csv,start,end):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(start,end):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            #if True:
            try:
                QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                a_dictionary['QUARTER_REVENUE'] = QUARTER_REVENUE_R
                a_dictionary['QUARTER_OPERATING_PROFIT'] = QUARTER_OPERATING_PROFIT_R
                a_dictionary['QUARTER_CASHFLOW'] = QUARTER_CASHFLOW_R
                a_dictionary['QUARTER_EARN'] = QUARTER_EARN_R
                a_dictionary['QUARTER_CONTINUE_EARN'] = CONTINUE_EARN_R
                a_dictionary['LAST_REVENUE'] = LAST_REVENUE_R
                a_dictionary['LAST_OPERATING_PROFIT'] = LAST_OPERATING_PROFIT_R
                a_dictionary['LAST_CASHFLOW'] = LAST_CASHFLOW_R
                a_dictionary['LAST_EARN'] = LAST_EARN_R
                a_dictionary['LAST_CONTINUE_EARN'] = LAST_CONTINUE_EARN_R
                a_dictionary['QUARTER_ASSET'] = QUARTER_ASSET_R
                a_dictionary['QUARTER_LIABILITY'] = QUARTER_LIABILITY_R
                #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                time.sleep(1)
            except:
                pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            try:
                LAST_FULL_EARN_R, LAST_FULL_OPERATING_PROFIT_R, LAST_FULL_CASHFLOW_R, LAST_FULL_REVENUE_R, LAST_FULL_CONTINUE_EARN_R, LAST_FULL_ASSET_R, LAST_FULL_LIABILITY_R = self.get_fs_of_last_full(OLD_REPORT_YEAR, 12)
                a_dictionary['LAST_FULL_REVENUE'] = LAST_FULL_REVENUE_R
                a_dictionary['LAST_FULL_OPERATING_PROFIT'] = LAST_FULL_OPERATING_PROFIT_R
                a_dictionary['LAST_FULL_CASHFLOW'] = LAST_FULL_CASHFLOW_R
                a_dictionary['LAST_FULL_EARN'] = LAST_FULL_EARN_R
                a_dictionary['LAST_FULL_CONTINUE_EARN'] = LAST_FULL_CONTINUE_EARN_R
                a_dictionary['LAST_FULL_ASSET'] = LAST_FULL_ASSET_R
                a_dictionary['LAST_FULL_LIABILITY'] = LAST_FULL_LIABILITY_R
                time.sleep(1)
            except:
                pass
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
                time.sleep(1)
            except:
                pass
           
            #if True:
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EPS'] = int(int(a_dictionary['QUARTER_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_EPS'] = int(int(a_dictionary['LAST_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['REAL_LAST_FULL_EPS'] = int(int(a_dictionary['LAST_FULL_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'QUARTER_ASSET' in a_dictionary.keys():
                        if 'QUARTER_LIABILITY' in a_dictionary.keys():
                            a_dictionary['QUARTER_BPS'] = int((int(a_dictionary['QUARTER_ASSET'])-int(a_dictionary['QUARTER_LIABILITY']))/int(a_dictionary['STOCK_AMOUNT']))
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_FULL_ASSET' in a_dictionary.keys():
                        if 'LAST_FULL_LIABILITY' in a_dictionary.keys():
                            a_dictionary['LAST_FULL_BPS'] = int((int(a_dictionary['LAST_FULL_ASSET'])-int(a_dictionary['LAST_FULL_LIABILITY']))/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'QUARTER_ASSET' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        QUARTER_FULL_EARN = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN']))
                        a_dictionary['QUARTER_ROA'] = round(QUARTER_FULL_EARN / int(a_dictionary['QUARTER_ASSET'])*100, 2)
                if 'LAST_FULL_ASSET' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_ROA'] = round(int(a_dictionary['LAST_FULL_EARN']) / int(a_dictionary['LAST_FULL_ASSET'])*100, 2)
            except:
                pass
            try:
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT'] = round(int(a_dictionary['QUARTER_OPERATING_PROFIT'])/int(a_dictionary['QUARTER_REVENUE'])*100, 2)
                if 'LAST_REVENUE' in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_OPERATING_PROFIT'] = round(int(a_dictionary['LAST_OPERATING_PROFIT'])/int(a_dictionary['LAST_REVENUE'])*100, 2)
                if 'LAST_FULL_REVENUE' in a_dictionary.keys():
                    if 'LAST_FULL_OPERATING_PROFIT' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_PERCENT_OPERATING_PROFIT'] = round(int(a_dictionary['LAST_FULL_OPERATING_PROFIT'])/int(a_dictionary['LAST_FULL_REVENUE'])*100, 2)
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_EARN'] = round(int(a_dictionary['QUARTER_EARN'])/int(a_dictionary['QUARTER_REVENUE'])*100, 2)
                if 'LAST_REVENUE' in a_dictionary.keys():
                    if 'LAST_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_EARN'] = round(int(a_dictionary['LAST_EARN'])/int(a_dictionary['LAST_REVENUE'])*100, 2)
                if 'LAST_FULL_REVENUE' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_PERCENT_EARN'] = round(int(a_dictionary['LAST_FULL_EARN'])/int(a_dictionary['LAST_FULL_REVENUE'])*100, 2)
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    if 'QUARTER_CASHFLOW' in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_CASHFLOW'] = round(int(a_dictionary['QUARTER_CASHFLOW'])/int(a_dictionary['QUARTER_REVENUE'])*100, 2)
                if 'LAST_REVENUE' in a_dictionary.keys():
                    if 'LAST_CASHFLOW' in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_CASHFLOW'] = round(int(a_dictionary['LAST_CASHFLOW'])/int(a_dictionary['LAST_REVENUE'])*100, 2)
                if 'LAST_FULL_REVENUE' in a_dictionary.keys():
                    if 'LAST_FULL_CASHFLOW' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_PERCENT_CASHFLOW'] = round(int(a_dictionary['LAST_FULL_CASHFLOW'])/int(a_dictionary['LAST_FULL_REVENUE'])*100, 2)
            except:
                pass
            try:
                if 'QUARTER_EPS' in a_dictionary.keys():
                    a_dictionary['QUARTER_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EPS']))
                '''if 'QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])'''
            except:
                pass
            try:
                if 'LAST_EPS' in a_dictionary.keys():
                    a_dictionary['FAKE_LAST_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['LAST_EPS']))   
                '''if 'OLD_QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_EARN'])
                if 'OLD_QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_OPERATING_PROFIT'])
                if 'OLD_QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_REVENUE'])'''
            except:
                pass
            #if True:
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_CASHFLOW'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_CONTINUE_EARN'], a_dictionary['QUARTER_FULL_EPS'], a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT'], a_dictionary['QUARTER_PERCENT_CASHFLOW'], a_dictionary['QUARTER_PERCENT_EARN'], a_dictionary['LAST_REVENUE'], a_dictionary['LAST_OPERATING_PROFIT'], a_dictionary['LAST_CASHFLOW'], \
                a_dictionary['LAST_EARN'], a_dictionary['LAST_CONTINUE_EARN'], a_dictionary['FAKE_LAST_FULL_EPS'], a_dictionary['LAST_PERCENT_OPERATING_PROFIT'], a_dictionary['LAST_PERCENT_CASHFLOW'], a_dictionary['LAST_PERCENT_EARN'], a_dictionary['LAST_FULL_REVENUE'], a_dictionary['LAST_FULL_OPERATING_PROFIT'], a_dictionary['LAST_FULL_CASHFLOW'], a_dictionary['LAST_FULL_EARN'], a_dictionary['REAL_LAST_FULL_EPS'], a_dictionary['QUARTER_BPS'],a_dictionary['LAST_FULL_BPS'], a_dictionary['QUARTER_ROA'], a_dictionary['LAST_FULL_ROA']])
                print(self.OLD_SYMBOL)
                #print(self.BOOM_PER_LIST)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Quarter_Revenue', 'Quarter_Operating_Profit', 'Quarter_Cashflow', 'Quarter_Earn', 'Quarter_Continue_Earn', 'Quarter_Full_EPS', 'Quarter_Percent_Operating_Profit','Quarter_Percent_Cashflow', 'Quarter_Percent_Earn', 'Last_Revenue', 'Last_Operating_Profit', 'Last_Cashflow', 'Last_Earn', \
                                                                    'Last_Continue_Earn', 'Fake_Last_Full_EPS', 'Last_Percent_Operating_Profit', 'Last_Percent_Cashflow', 'Last_Percent_Earn', 'Last_Full_Revenue', 'Last_Full_Operating_Profit', 'Last_Full_Cashflow', 'Last_Full_Earn', 'Real_Last_Full_EPS', 'Quarter_BPS', 'Last_Full_BPS','Quarter_ROA','Last_Full_ROA'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
    
    def get_per(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            LAST_YEAR = self.get_last_year_report(REPORT_DATE.strftime("%Y-%m-%d"))
            LAST_STOCK_PRICE = self.get_top_last_year(LAST_YEAR)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Full_EPS'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            FAKE_LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Fake_Last_Full_EPS'])
            if FAKE_LAST_EARN_PER_STOCK != 0:
                FAKE_LAST_PER = int(LAST_STOCK_PRICE/FAKE_LAST_EARN_PER_STOCK)
            if FAKE_LAST_EARN_PER_STOCK == 0:
                FAKE_LAST_PER = 0
            #if 'LAST_STOCK_PRICE' in locals():
            REAL_LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Real_Last_Full_EPS'])
            if REAL_LAST_EARN_PER_STOCK != 0:
                REAL_LAST_PER = int(LAST_STOCK_PRICE/REAL_LAST_EARN_PER_STOCK)
            if REAL_LAST_EARN_PER_STOCK == 0:
                REAL_LAST_PER = 0
            
            PRICE_PERCENT_CHANGE = self.check_report_month_price_data(REPORT_DATE)
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER), str(LAST_STOCK_PRICE), str(FAKE_LAST_PER), str(REAL_LAST_PER), str(PRICE_PERCENT_CHANGE)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price', 'Quarter_Per', 'Last_Stock_Price', 'Fake_Last_Per', 'Real_Last_Per', 'Price_Percent_Change'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def how_report_sell(self, analyse_date, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            try:
                FULL_QUARTER_REVENUE, FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                time.sleep(1)
            except:
                pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
                print(STOCK_AMOUNT)
            except:
                pass
            #if True:
                
                try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = int(a_dictionary['QUARTER_EARN']/a_dictionary['STOCK_AMOUNT'])
                except:
                    pass
            #try:
                if 'QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN_PER_STOCK'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN_PER_STOCK'])
                        a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                    #a_dictionary['PER'] = int(STOCK_PRICE/a_dictionary['QUARTER_EARN_PER_STOCK'])
            #except:
                #a_dictionary['PER'] = 0
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])
                    
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                #pass
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_EARN_PER_STOCK']])
                print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date_Sell', 'Quarter_Revenue_Sell', 'Quarter_Operating_Profit_Sell', 'Quarter_Earn_Sell', 'Quarter_Earn_Per_Stock_Sell'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def get_per_sell(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I,'Report_Date_Sell']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price_Sell', 'Quarter_Per_Sell'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
#have 2 keys 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788', '6048f2c5a065e754cee104b8c0edd8186f6b3d61', '94331def3e94db8a1aabb84e4257cec0ad3b576b', 'b1fe4037e43a6db43d8ac280d262a05e90bf9171'
akaps = how_per('d088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9')
#akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(7).csv')
akaps.how_report('2020-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(2020_6).csv',1,1270)
'''paka = how_per('84acf17c852e9430843a589191330e0630cc19a7')
paka.how_report('2020-11-30','KOSDAQ.csv', 'A_PER_KOSDAQ_real(2020_6)(1).csv',400,800)
akaps = how_per('1eca9bad2afb8becdc5b1714112dd097e73da788')
akaps.how_report('2020-11-30','KOSDAQ.csv', 'A_PER_KOSDAQ_real(2020_6)(2).csv',800,1270)'''
'''akaps.get_per('AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6).csv')
akaps.how_report_sell('2020-11-30', 'AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6_9).csv')
akaps.get_per_sell('AN_PER_KOSDAQ(2020_6_9).csv', 'AN_PER_KOSDAQ(2020_6_9).csv')

paka = how_per('84acf17c852e9430843a589191330e0630cc19a7')
paka.how_report_sell('2021-03-30', 'AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6_12).csv')
paka.get_per_sell('AN_PER_KOSDAQ(2020_6_12).csv', 'AN_PER_KOSDAQ(2020_6_12).csv')
#akaps.how_report('2020-08-30', 'AN_PER_KOSDAQ_real(2020_6)_test.csv','AN_PER_KOSDAQ_real(2020_6)_test(1).csv')'''

In [7]:



#real__monthly_test




import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import datetime as dt 
from datetime import date
import time
from dateutil.relativedelta import relativedelta
import math
import warnings
import OpenDartReader
from pandas.core.common import SettingWithCopyWarning
import re
from collections.abc import Mapping

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)



class how_per():
    def __init__(self, api_key):
        self.api_key = api_key
        self.OLD_COUNT = 0
        self.OLD_SYMBOL = '가나다'
        self.SYMBOL = '가나다'
        self.TODAY = dt.date.today()
        self.BOOM_PER_LIST = []
        self.dart = OpenDartReader(self.api_key)
        
    def outlier_treatment(self, datacolumn):
        sorted(datacolumn)
        if True:
        #try:
            Q4 = np.percentile(datacolumn , 100)
            Q4_Date = datacolumn.index[datacolumn['Close'] == Q4].tolist()
            Q4_Date = Q4_Date[-1].date()
        #except:
            #return True
        return Q4, Q4_Date
    
    def low_high_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q1, Q2= np.percentile(datacolumn ,[10,100])
        except:
            return True
        return Q1, Q2
    
    def big_small_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q1, Q2= np.percentile(datacolumn ,[10,90])
        except:
            return True
        return Q1, Q2
    
    def has_no_fix(self,word):
        if '[기재정정]' in word:
            return True
        else:
            return False

    def get_want_fast_report(self, a_s_list, report_dt):
        a_s_list = a_s_list[a_s_list['report_nm'].str.contains('영업(잠정)실적(공정공시)', regex = False)]
        a_s_list['rcept_dt'] = pd.to_datetime(a_s_list['rcept_dt'], format = '%Y%m%d').dt.date
        a_s_list = a_s_list[a_s_list['rcept_dt'] +  relativedelta(months = 1) > report_dt]
        a_s_list = a_s_list[a_s_list['rcept_dt'] - relativedelta(months = 3) < report_dt]
        a_s_list.reset_index(drop = True, inplace = True)
        return a_s_list

    def get_before_report_time(self, report_year, report_month):
        before_report_year = report_year-1
        before_report_month = report_month
        return before_report_year, before_report_month
    
    def check_report_month_price_data(self, report_date):
        #report_plus_date = report_date + relativedelta(days = 10)
        report_minus_date = report_date - relativedelta(days = 15)
        #report_plus_date = report_plus_date.strftime("%Y-%m-%d")
        report_date = report_date.strftime("%Y-%m-%d")
        report_minus_date = report_minus_date.strftime("%Y-%m-%d")
        df_stock_price = fdr.DataReader(self.SYMBOL, report_minus_date, report_date)
        close_price_column = df_stock_price[['Close']]
        low_price, high_price = self.low_high_treatment(close_price_column)
        percent_change = round((int(high_price)-int(low_price))/int(low_price)*100,2)
        return percent_change
    
    def check_price_data(self, report_date):
        report_stock_date = date(2000,1,1)
        before_report_date = report_date - relativedelta(days = 14)
        now_report_date = report_date + relativedelta(days = 1)
        before_report_date = before_report_date.strftime("%Y-%m-%d")
        now_report_date = now_report_date.strftime("%Y-%m-%d")
        report_stock_price = 0
        df_stock_price = fdr.DataReader(self.SYMBOL, before_report_date, now_report_date)
        
        for i in range(0,14):
            report_stock_date = report_date - relativedelta(days = i)
            report_stock_date = report_stock_date.strftime("%Y-%m-%d")
            try:
                report_stock_price = df_stock_price.loc[report_stock_date, 'Close']
            except:
                continue
            if report_stock_price:
                break
            else:
                continue
        return int(report_stock_price)
    
    def get_top_last_year(self, last_report_year, recent_report_year):
        last_report_year = last_report_year.strftime("%Y-%m-%d")
        recent_report_year = recent_report_year.strftime("%Y-%m-%d")
        df_stock_price = fdr.DataReader(self.SYMBOL, last_report_year, recent_report_year)
        price_column = df_stock_price[['Close']]
        top_price_last_year, top_price_date = self.outlier_treatment(price_column)
        return int(top_price_last_year), top_price_date
    
    def get_top_bottom_percent_change(self, last_report_year, recent_report_year):
        last_report_year = last_report_year.strftime("%Y-%m-%d")
        recent_report_year = recent_report_year.strftime("%Y-%m-%d")
        df_stock_price = fdr.DataReader(self.SYMBOL, last_report_year, recent_report_year)
        price_column = df_stock_price[['Close']]
        print(price_column)
        bottom_price_last_year, top_price_last_year = self.low_high_treatment(price_column)
        percent_change = round((int(top_price_last_year)-int(bottom_price_last_year))/int(bottom_price_last_year)*100, 2)
        return percent_change
    
    '''def give_revenue_data(self):
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        else:
            return'''
    
    def get_stock_amount(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        if report_month == 3:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[0])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 6:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[1])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 9:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[2])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 12:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[3])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        else:
            return

        
    def give_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        self.report['account_nm'] = self.report['account_nm'].str.replace(" ","")
        if self.report['account_nm'].str.contains('지배기업의소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의소유주', regex = False)]
                    report = report[report['account_nm'].str.contains('귀속될', regex = False)]
                    report = report[report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        
    def give_operating_profit(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit= self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit = self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return -int(quarter_operating_profit)
        else:
            return 0
        
    def give_revenue_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs_GrossProfit'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs_GrossProfit'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
    def give_last_earn_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의 소유주', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('귀속될', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
    
   
    def give_last_revenue_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs_GrossProfit')]).any():
            if ('IS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_GrossProfit'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_GrossProfit')]).any():
            if ('CIS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_GrossProfit'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                return int(quarter_operating_profit)
        else:
            return 0 
    
    def give_last_operating_profit(self):
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                return -int(quarter_operating_profit)
        else:
            return 0 
    
    def give_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('BS', 'ifrs_Assets')]).any():
            if ('BS', 'ifrs_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return 0 
    
    def give_last_asset(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'frmtrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('BS', 'ifrs_Assets')]).any():
            if ('BS', 'ifrs_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs_Assets'), 'frmtrm_amount']
                return int(quarter_asset)
        else:
            return 0 
        
    def give_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('BS', 'ifrs_Liabilities')]).any():
            if ('BS', 'ifrs_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return 0 
        
    def give_last_liability(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'frmtrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('BS', 'ifrs_Liabilities')]).any():
            if ('BS', 'ifrs_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs_Liabilities'), 'frmtrm_amount']
                return int(quarter_liability)
        else:
            return 0 
        
    def give_continue_earn(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def give_last_continue_earn(self):
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_cashflow(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0 
    
    def give_last_cashflow(self):
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_q_amount']
                return int(quarter_earn)
        else:
            return 0 
        
    def check_lenz(self, df):
        return df is None
    
    def check_empty(self, df):
        return df.empty
        
    def get_fs_of_time(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL , str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
        if report_month == 6:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
        else:
            return 
    
    def give_add_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        self.report['account_nm'] = self.report['account_nm'].str.replace(" ","")
        if self.report['account_nm'].str.contains('지배기업의소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의소유주', regex = False)]
                    report = report[report['account_nm'].str.contains('귀속될', regex = False)]
                    report = report[report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        
    def give_add_operating_profit(self):
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit= self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit = self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return -int(quarter_operating_profit)
        else:
            return 0
        
    def give_add_revenue_data(self):
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
    def give_last_add_earn_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의 소유주', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('귀속될', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
    
   
    def give_last_add_revenue_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs_GrossProfit')]).any():
            if ('IS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_GrossProfit')]).any():
            if ('CIS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
    def give_last_add_operating_profit(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return -int(quarter_operating_profit)
        else:
            return 0
    
    def give_add_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('BS', 'ifrs_Assets')]).any():
            if ('BS', 'ifrs_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return 0
    
    def give_last_add_asset(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'frmtrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('BS', 'ifrs_Assets')]).any():
            if ('BS', 'ifrs_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs_Assets'), 'frmtrm_amount']
                return int(quarter_asset)
        else:
            return 0
        
    def give_add_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('BS', 'ifrs_Liabilities')]).any():
            if ('BS', 'ifrs_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return 0
        
    def give_last_add_liability(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'frmtrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('BS', 'ifrs_Liabilities')]).any():
            if ('BS', 'ifrs_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs_Liabilities'), 'frmtrm_amount']
                return int(quarter_liability)
        else:
            return 0
        
    def give_add_continue_earn(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def give_last_add_continue_earn(self):
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_add_cashflow(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0, 0
    
    def give_last_add_cashflow(self):
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_q_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def get_fs_of_add_time(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_add_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_add_operating_profit()
                        QUARTER_CASHFLOW = self.give_add_cashflow()
                        QUARTER_REVENUE = self.give_add_revenue_data()
                        CONTINUE_EARN = self.give_add_continue_earn()
                        QUARTER_ASSET = self.give_add_asset()
                        QUARTER_LIABILITY = self.give_add_liability()
                        LAST_EARN = self.give_last_add_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_add_operating_profit()
                        LAST_CASHFLOW = self.give_last_add_cashflow()
                        LAST_REVENUE = self.give_last_add_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_add_continue_earn()
                        LAST_ASSET = self.give_last_add_asset()
                        LAST_LIABILITY = self.give_last_add_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_add_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_add_operating_profit()
                        QUARTER_CASHFLOW = self.give_add_cashflow()
                        QUARTER_REVENUE = self.give_add_revenue_data()
                        CONTINUE_EARN = self.give_add_continue_earn()
                        QUARTER_ASSET = self.give_add_asset()
                        QUARTER_LIABILITY = self.give_add_liability()
                        LAST_EARN = self.give_last_add_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_add_operating_profit()
                        LAST_CASHFLOW = self.give_last_add_cashflow()
                        LAST_REVENUE = self.give_last_add_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_add_continue_earn()
                        LAST_ASSET = self.give_last_add_asset()
                        LAST_LIABILITY = self.give_last_add_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
        else:
            return 
        
    def give_last_full_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        self.report['account_nm'] = self.report['account_nm'].str.replace(" ","")
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의소유주', regex = False)]
                    report = report[report['account_nm'].str.contains('귀속될', regex = False)]
                    report = report[report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_last_full_revenue_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs_GrossProfit')]).any():
            if ('IS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_GrossProfit')]).any():
            if ('CIS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
    def give_last_full_operating_profit(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
     
    
    def give_last_full_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('BS', 'ifrs_Assets')]).any():
            if ('BS', 'ifrs_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return
        
    def give_last_full_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('BS', 'ifrs_Liabilities')]).any():
            if ('BS', 'ifrs_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return 0 
        
    def give_last_full_continue_earn(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_last_full_cashflow(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_last_add_full_earn_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        self.report['account_nm'] = self.report['account_nm'].str.replace(" ","")
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의소유주', regex = False)]
                    report = report[report['account_nm'].str.contains('귀속될', regex = False)]
                    report = report[report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_last_add_full_revenue_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs_GrossProfit')]).any():
            if ('IS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_GrossProfit'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_GrossProfit')]).any():
            if ('CIS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_GrossProfit'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
    def give_last_add_full_operating_profit(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
     
    
    def give_last_add_full_asset(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'frmtrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('BS', 'ifrs_Assets')]).any():
            if ('BS', 'ifrs_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs_Assets'), 'frmtrm_amount']
                return int(quarter_asset)
        else:
            return 0
        
    def give_last_add_full_liability(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'frmtrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('BS', 'ifrs_Liabilities')]).any():
            if ('BS', 'ifrs_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs_Liabilities'), 'frmtrm_amount']
                return int(quarter_liability)
        else:
            return 0
        
    def give_last_add_full_continue_earn(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_last_add_full_cashflow(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def get_fs_of_last_full(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        LAST_FULL_EARN = self.give_last_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        LAST_FULL_CASHFLOW = self.give_last_full_cashflow()
                        LAST_FULL_ASSET = self.give_last_full_asset()
                        LAST_FULL_LIABILITY = self.give_last_full_liability()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_CASHFLOW, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN, LAST_FULL_ASSET, LAST_FULL_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        LAST_FULL_EARN = self.give_last_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        LAST_FULL_CASHFLOW = self.give_last_full_cashflow()
                        LAST_FULL_ASSET = self.give_last_full_asset()
                        LAST_FULL_LIABILITY = self.give_last_full_liability()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_CASHFLOW, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN, LAST_FULL_ASSET, LAST_FULL_LIABILITY
            #except:
                #pass
        else:
            return
        
    def get_fs_of_add_last_full(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        FULL_EARN = self.give_last_full_earn_data()
                        FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        FULL_REVENUE = self.give_last_full_revenue_data()
                        FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        FULL_CASHFLOW = self.give_last_full_cashflow()
                        FULL_ASSET = self.give_last_full_asset()
                        FULL_LIABILITY = self.give_last_full_liability()
                        LAST_FULL_EARN = self.give_last_add_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_add_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_add_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_add_full_continue_earn()
                        LAST_FULL_CASHFLOW = self.give_last_add_full_cashflow()
                        LAST_FULL_ASSET = self.give_last_add_full_asset()
                        LAST_FULL_LIABILITY = self.give_last_add_full_liability()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return FULL_REVENUE, FULL_OPERATING_PROFIT, FULL_CASHFLOW, FULL_EARN, FULL_CONTINUE_EARN, LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_CASHFLOW, LAST_FULL_REVENUE, FULL_CONTINUE_EARN, FULL_ASSET, FULL_LIABILITY, LAST_FULL_ASSET, LAST_FULL_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        FULL_EARN = self.give_last_full_earn_data()
                        FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        FULL_REVENUE = self.give_last_full_revenue_data()
                        FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        FULL_CASHFLOW = self.give_last_full_cashflow()
                        FULL_ASSET = self.give_last_full_asset()
                        FULL_LIABILITY = self.give_last_full_liability()
                        LAST_FULL_EARN = self.give_last_add_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_add_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_add_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_add_full_continue_earn()
                        LAST_FULL_CASHFLOW = self.give_last_add_full_cashflow()
                        LAST_FULL_ASSET = self.give_last_add_full_asset()
                        LAST_FULL_LIABILITY = self.give_last_add_full_liability()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return FULL_REVENUE, FULL_OPERATING_PROFIT, FULL_CASHFLOW, FULL_EARN, FULL_CONTINUE_EARN, LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_CASHFLOW, LAST_FULL_REVENUE, FULL_CONTINUE_EARN, FULL_ASSET, FULL_LIABILITY, LAST_FULL_ASSET, LAST_FULL_LIABILITY
            #except:
                #pass
        else:
            return
        
    
    def get_real_report_month(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year
            report_month = 6
        if report_month == 6:
            report_year = report_year
            report_month = 9
        if report_month == 9:
            report_year = report_year
            report_month = 12
        if report_month == 12:
            report_year = report_year + 1
            report_month = 3
        return report_year, report_month
    
    def get_predict_earn_per_stock(self, report_month, earn_per_stock):
        earn_per_stock = int(earn_per_stock)
        
        '''if report_month == 3:
            earn_per_stock = earn_per_stock*4
        if report_month == 6:
            earn_per_stock = earn_per_stock*2
        if report_month == 9:
            earn_per_stock = earn_per_stock*(4/3)
        if report_month == 12:
            earn_per_stock = earn_per_stock'''
        earn_per_stock = earn_per_stock*4
        
        return earn_per_stock
    
    def get_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_fast_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            #report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            #report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_last_year_report(self, report_date):
        delimiters = "-"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_date)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        last_year = int(nm_num_list[0])-1
        last_month = int(nm_num_list[1])
        return last_year, last_month
    
    def get_only_number(self, report_data):
        delimiters = ","
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_data)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        element_amount = len(nm_num_list)
        report_data = 0
        for i in range(len(nm_num_list)):
            report_data = report_data + nm_num_list[i]*(10**((element_amount - i - 1)*3))
        return report_data
    
    def get_report_year_month(self, report_nm):
        delimiters = "(", ".", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_new(self, report_nm):
        delimiters = "(", "_", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_one_back(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year - 1
            report_month = 12
            return report_year, report_month
        if report_month == 6:
            report_year = report_year
            report_month = 3
            return report_year, report_month
        if report_month == 9:
            report_year = report_year
            report_month = 6
            return report_year, report_month
        if report_month == 12:
            report_year = report_year
            report_month = 9
            return report_year, report_month
            
    def get_symbol_six(self, analy_kosdaq, i):
        new_symbol = analy_kosdaq.loc[i, 'Symbol']
        if new_symbol != self.OLD_SYMBOL:
            self.OLD_COUNT = 0
            symbol = analy_kosdaq.loc[i, 'Symbol']
            symbol_length = self.digit_length(symbol)
            if symbol_length != 6:
                zero_need = 6 - symbol_length
                self.SYMBOL = '0'*zero_need + str(symbol)
            if symbol_length == 6:
                self.SYMBOL = str(symbol)
            self.OLD_SYMBOL = new_symbol
            return
        if new_symbol == self.OLD_SYMBOL:
            self.OLD_COUNT += 1
        return 
    
    #자릿수 찾기
    def digit_length(self, n):
        ans = 0
        n= int(n)

        while n:
            n //= 10
            ans += 1

        return ans


    def how_report(self, analyse_date, read_csv, write_csv,start,end):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(start,end):
            try:
                self.get_symbol_six(ANALY_KOSDAQ, I)
                #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
                ANALYSE_DATE = analyse_date
                ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
                ANALYSE_DATE = ANALYSE_DATE.date()
                START_DATE = ANALYSE_DATE - relativedelta(months = 18)
                self.START_DATE = START_DATE.strftime("%Y-%m-%d")
                ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
                S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
                a_dictionary = {} 
                REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
                REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
                OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            except:
                continue
            #if True:
            try:
                if REPORT_MONTH != 12:
                    #if True:
                    try:
                        QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R, LAST_ASSET_R, LAST_LIABILITY_R = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                        #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                        a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_REVENUE_R
                        a_dictionary['QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_OPERATING_PROFIT_R
                        a_dictionary['QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_CASHFLOW_R
                        a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_EARN_R
                        a_dictionary['QUARTER_CONTINUE_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = CONTINUE_EARN_R
                        a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_REVENUE_R
                        a_dictionary['LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_OPERATING_PROFIT_R
                        a_dictionary['LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_CASHFLOW_R
                        a_dictionary['LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_EARN_R
                        a_dictionary['LAST_CONTINUE_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_CONTINUE_EARN_R
                        a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_ASSET_R
                        a_dictionary['QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_LIABILITY_R
                        a_dictionary['LAST_ASSET' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_ASSET_R
                        a_dictionary['LAST_LIABILITY' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_LIABILITY_R
                        #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                        a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(0.1)
                    except:
                        pass
                if REPORT_MONTH == 12:
                    #if True:
                    try:
                        QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R, LAST_ASSET_R, LAST_LIABILITY_R = self.get_fs_of_add_last_full(REPORT_YEAR, REPORT_MONTH)
                        #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                        #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                        a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                        QUARTER_REVENUE_ADD_R, QUARTER_OPERATING_PROFIT_ADD_R, QUARTER_CASHFLOW_ADD_R, QUARTER_EARN_ADD_R, CONTINUE_EARN_ADD_R, LAST_EARN_ADD_R, LAST_OPERATING_PROFIT_ADD_R, LAST_CASHFLOW_ADD_R, LAST_REVENUE_ADD_R, LAST_CONTINUE_EARN_ADD_R, QUARTER_ASSET_ADD_R, QUARTER_LIABILITY_ADD_R, LAST_ASSET_ADD_R, LAST_LIABILITY_ADD_R = self.get_fs_of_add_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                        a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_REVENUE_R) - int(QUARTER_REVENUE_ADD_R)
                        a_dictionary['QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_OPERATING_PROFIT_R) - int(QUARTER_OPERATING_PROFIT_ADD_R)
                        a_dictionary['QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_CASHFLOW_R) - int(QUARTER_CASHFLOW_ADD_R)
                        a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_EARN_R) - int(QUARTER_EARN_ADD_R)
                        a_dictionary['QUARTER_CONTINUE_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(CONTINUE_EARN_R) - int(CONTINUE_EARN_ADD_R)
                        a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_REVENUE_R) - int(LAST_REVENUE_ADD_R)
                        a_dictionary['LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_OPERATING_PROFIT_R) - int(LAST_OPERATING_PROFIT_ADD_R)
                        a_dictionary['LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_CASHFLOW_R) - int(LAST_CASHFLOW_ADD_R)
                        a_dictionary['LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_EARN_R) - int(LAST_EARN_ADD_R)
                        a_dictionary['LAST_CONTINUE_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_CONTINUE_EARN_R) - int(LAST_CONTINUE_EARN_ADD_R)
                        a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_ASSET_R) - int(QUARTER_ASSET_ADD_R)
                        a_dictionary['QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_LIABILITY_R) - int(QUARTER_LIABILITY_ADD_R)
                        a_dictionary['LAST_ASSET' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_ASSET_R) - int(LAST_ASSET_ADD_R)
                        a_dictionary['LAST_LIABILITY' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_LIABILITY_R) - int(LAST_LIABILITY_ADD_R)
                        time.sleep(0.1)
                    except:
                        pass

                if BACK_REPORT_MONTH != 12:
                    #if True:
                    try:
                        QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R, LAST_ASSET_R, LAST_LIABILITY_R = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                        #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                        a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = QUARTER_REVENUE_R
                        a_dictionary['QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR)+ '_' + str(BACK_REPORT_MONTH)] = QUARTER_OPERATING_PROFIT_R
                        a_dictionary['QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = QUARTER_CASHFLOW_R
                        a_dictionary['QUARTER_EARN' + str(BACK_REPORT_YEAR)+ '_' + str(BACK_REPORT_MONTH)] = QUARTER_EARN_R
                        a_dictionary['QUARTER_CONTINUE_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = CONTINUE_EARN_R
                        a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_REVENUE_R
                        a_dictionary['LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_OPERATING_PROFIT_R
                        a_dictionary['LAST_CASHFLOW' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_CASHFLOW_R
                        a_dictionary['LAST_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_EARN_R
                        a_dictionary['LAST_CONTINUE_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_CONTINUE_EARN_R
                        a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = QUARTER_ASSET_R
                        a_dictionary['QUARTER_LIABILITY' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = QUARTER_LIABILITY_R
                        a_dictionary['LAST_ASSET' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_ASSET_R
                        a_dictionary['LAST_LIABILITY' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_LIABILITY_R
                        #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                        #a_dictionary['BACK_REPORT_DATE'] = BACK_REPORTREPORT_DT.strftime("%Y-%m-%d")
                        time.sleep(0.1)
                    except:
                        pass
                if BACK_REPORT_MONTH == 12:
                    #if True:
                    try:
                        QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R, LAST_ASSET_R, LAST_LIABILITY_R = self.get_fs_of_add_last_full(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                        BACK_BACK_REPORT_YEAR, BACK_BACK_REPORT_MONTH = self.get_report_year_month_one_back(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                        time.sleep(0.1)
                        QUARTER_REVENUE_ADD_R, QUARTER_OPERATING_PROFIT_ADD_R, QUARTER_CASHFLOW_ADD_R, QUARTER_EARN_ADD_R, CONTINUE_EARN_ADD_R, LAST_EARN_ADD_R, LAST_OPERATING_PROFIT_ADD_R, LAST_CASHFLOW_ADD_R, LAST_REVENUE_ADD_R, LAST_CONTINUE_EARN_ADD_R, QUARTER_ASSET_ADD_R, QUARTER_LIABILITY_ADD_R, LAST_ASSET_ADD_R, LAST_LIABILITY_ADD_R = self.get_fs_of_add_time(BACK_BACK_REPORT_YEAR, BACK_BACK_REPORT_MONTH)
                        a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_REVENUE_R) - int(QUARTER_REVENUE_ADD_R)
                        a_dictionary['QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_OPERATING_PROFIT_R) - int(QUARTER_OPERATING_PROFIT_ADD_R)
                        a_dictionary['QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_CASHFLOW_R) - int(QUARTER_CASHFLOW_ADD_R)
                        a_dictionary['QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_EARN_R) - int(QUARTER_EARN_ADD_R)
                        a_dictionary['QUARTER_CONTINUE_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(CONTINUE_EARN_R) - int(CONTINUE_EARN_ADD_R)
                        a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_REVENUE_R) - int(LAST_REVENUE_ADD_R)
                        a_dictionary['LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_OPERATING_PROFIT_R) - int(LAST_OPERATING_PROFIT_ADD_R)
                        a_dictionary['LAST_CASHFLOW' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_CASHFLOW_R) - int(LAST_CASHFLOW_ADD_R)
                        a_dictionary['LAST_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_EARN_R) - int(LAST_EARN_ADD_R)
                        a_dictionary['LAST_CONTINUE_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_CONTINUE_EARN_R) - int(LAST_CONTINUE_EARN_ADD_R)
                        a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_ASSET_R) - int(QUARTER_ASSET_ADD_R)
                        a_dictionary['QUARTER_LIABILITY' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_LIABILITY_R) - int(QUARTER_LIABILITY_ADD_R)
                        a_dictionary['LAST_ASSET' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_ASSET_R) - int(LAST_ASSET_ADD_R)
                        a_dictionary['LAST_LIABILITY' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_LIABILITY_R) - int(LAST_LIABILITY_ADD_R)
                        #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                        time.sleep(0.1)
                    except:
                        pass
            except:
                pass
                
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            try:
                LAST_FULL_EARN_R, LAST_FULL_OPERATING_PROFIT_R, LAST_FULL_CASHFLOW_R, LAST_FULL_REVENUE_R, LAST_FULL_CONTINUE_EARN_R, LAST_FULL_ASSET_R, LAST_FULL_LIABILITY_R = self.get_fs_of_last_full(OLD_REPORT_YEAR, 12)
                a_dictionary['LAST_FULL_REVENUE'] = LAST_FULL_REVENUE_R
                a_dictionary['LAST_FULL_OPERATING_PROFIT'] = LAST_FULL_OPERATING_PROFIT_R
                a_dictionary['LAST_FULL_CASHFLOW'] = LAST_FULL_CASHFLOW_R
                a_dictionary['LAST_FULL_EARN'] = LAST_FULL_EARN_R
                a_dictionary['LAST_FULL_CONTINUE_EARN'] = LAST_FULL_CONTINUE_EARN_R
                a_dictionary['LAST_FULL_ASSET'] = LAST_FULL_ASSET_R
                a_dictionary['LAST_FULL_LIABILITY'] = LAST_FULL_LIABILITY_R
                time.sleep(0.1)
            except:
                pass
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
                time.sleep(0.1)
            except:
                pass
           
            #if True:
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        if 'QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                            a_dictionary['QUARTER_EPS'] = int(int(a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_EARN' + str(REPORT_YEAR - 1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        if 'LAST_EARN' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():                                                                                                           
                            a_dictionary['LAST_EPS'] = int(int(a_dictionary['LAST_EARN' + str(REPORT_YEAR - 1) + '_' + str(REPORT_MONTH)])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['REAL_LAST_FULL_EPS'] = int(int(a_dictionary['LAST_FULL_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        if 'QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                            a_dictionary['QUARTER_BPS'] = int((int(a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)]) - (int(a_dictionary['QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_LIABILITY' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])))/int(a_dictionary['STOCK_AMOUNT']))
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_FULL_ASSET' in a_dictionary.keys():
                        if 'LAST_FULL_LIABILITY'in a_dictionary.keys():
                            a_dictionary['LAST_FULL_BPS'] = int((int(a_dictionary['LAST_FULL_ASSET'])-int(a_dictionary['LAST_FULL_LIABILITY']))/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_EPS' in a_dictionary.keys():
                        if 'STOCK_AMOUNT' in a_dictionary.keys():
                            QUARTER_FULL_EARN = int(self.get_predict_earn_per_stock(REPORT_MONTH, int(a_dictionary['QUARTER_EPS']) * int(a_dictionary['STOCK_AMOUNT'])))
                            a_dictionary['QUARTER_ROA'] = round(QUARTER_FULL_EARN / (int(a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)]))*100, 2)
                if 'LAST_FULL_ASSET' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_ROA'] = round(int(a_dictionary['LAST_FULL_EARN']) / int(a_dictionary['LAST_FULL_ASSET'])*100, 2)
            except:
                pass
            try:
                if 'QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_EPS' in a_dictionary.keys():
                        if 'STOCK_AMOUNT' in a_dictionary.keys():
                            QUARTER_FULL_EARN = int(self.get_predict_earn_per_stock(REPORT_MONTH, int(a_dictionary['QUARTER_EPS']) * int(a_dictionary['STOCK_AMOUNT'])))
                            a_dictionary['QUARTER_ROE'] = round(QUARTER_FULL_EARN / (int(a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)]) - (int(a_dictionary['QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_LIABILITY' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])))*100, 2)
                if 'LAST_FULL_ASSET' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_ROE'] = round(int(a_dictionary['LAST_FULL_EARN']) / (int(a_dictionary['LAST_FULL_ASSET']) - int(a_dictionary['LAST_FULL_LIABILITY']))*100, 2)
            except:
                pass
            try:
                if 'QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_OPERATING_PROFIT' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_FULL_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_FULL_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_FULL_PERCENT_OPERATING_PROFIT'] = round(int(a_dictionary['LAST_FULL_OPERATING_PROFIT'])/int(a_dictionary['LAST_FULL_REVENUE'])*100, 2)
                if 'QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_EARN' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_EARN' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['LAST_EARN' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_FULL_REVENUE' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_PERCENT_EARN'] = round(int(a_dictionary['LAST_FULL_EARN'])/int(a_dictionary['LAST_FULL_REVENUE'])*100, 2)
                if 'QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_CASHFLOW' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_CASHFLOW' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['LAST_CASHFLOW' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_FULL_REVENUE' in a_dictionary.keys():
                    if 'LAST_FULL_CASHFLOW' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_PERCENT_CASHFLOW'] = round(int(a_dictionary['LAST_FULL_CASHFLOW'])/int(a_dictionary['LAST_FULL_REVENUE'])*100, 2)
            except:
                pass
            try:
                if 'QUARTER_EPS' in a_dictionary.keys():
                    a_dictionary['QUARTER_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EPS']))
                '''if 'QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])'''
            except:
                pass
            try:
                if 'LAST_EPS' in a_dictionary.keys():
                    a_dictionary['FAKE_LAST_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['LAST_EPS']))   
                '''if 'OLD_QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_EARN'])
                if 'OLD_QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_OPERATING_PROFIT'])
                if 'OLD_QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_REVENUE'])'''
            except:
                pass
            #if True:
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_CONTINUE_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)],\
                                            a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_PERCENT_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_PERCENT_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], \
                                           a_dictionary['QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_CONTINUE_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_PERCENT_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], \
                                           a_dictionary['QUARTER_PERCENT_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_FULL_EPS'], a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_CONTINUE_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], \
                                           a_dictionary['LAST_PERCENT_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_PERCENT_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_PERCENT_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_CASHFLOW' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], \
                                           a_dictionary['LAST_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_CONTINUE_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_PERCENT_OPERATING_PROFIT' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_PERCENT_CASHFLOW' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_PERCENT_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['FAKE_LAST_FULL_EPS'], \
                                           a_dictionary['LAST_FULL_REVENUE'], a_dictionary['LAST_FULL_OPERATING_PROFIT'], a_dictionary['LAST_FULL_CASHFLOW'], a_dictionary['LAST_FULL_EARN'], a_dictionary['REAL_LAST_FULL_EPS'], a_dictionary['QUARTER_BPS'],a_dictionary['LAST_FULL_BPS'], a_dictionary['QUARTER_ROA'], a_dictionary['LAST_FULL_ROA'], a_dictionary['QUARTER_ROE'], a_dictionary['LAST_FULL_ROE']])
                print(self.OLD_SYMBOL)
                #print(self.BOOM_PER_LIST)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Quarter_Revenue' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Operating_Profit' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Cashflow' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Earn' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Continue_Earn' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Percent_Operating_Profit' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH),\
                                                                    'Quarter_Percent_Cashflow' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Percent_Earn' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Revenue' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Operating_Profit' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Cashflow' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), \
                                                                    'Quarter_Earn' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Continue_Earn' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Percent_Operating_Profit' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH),'Quarter_Percent_Cashflow' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Percent_Earn' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Full_EPS',\
                                                                    'Last_Revenue' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Operating_Profit' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Cashflow' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Earn' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Continue_Earn' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Percent_Operating_Profit' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Percent_Cashflow' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Percent_Earn' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), \
                                                                    'Last_Revenue' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Operating_Profit' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Cashflow' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Earn' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Continue_Earn' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), \
                                                                    'Last_Percent_Operating_Profit' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Percent_Cashflow' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Percent_Earn' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH),\
                                                                    'Fake_Last_Full_EPS', 'Last_Full_Revenue', 'Last_Full_Operating_Profit', 'Last_Full_Cashflow', 'Last_Full_Earn', 'Real_Last_Full_EPS', 'Quarter_BPS', 'Last_Full_BPS','Quarter_ROA','Last_Full_ROA', 'QUARTER_ROE', 'LAST_FULL_ROE'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
    
    def get_per(self, read_csv, write_csv, index_name):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            LAST_REPORT_DATE = REPORT_DATE - relativedelta(months = 12)
            LAST_TOP_SOCK_PRICE = self.get_top_last_year(LAST_REPORT_DATE, REPORT_DATE)
            OLD_REPORT_DATE = REPORT_DATE - relativedelta(years = 3)
            OLD_STOCK_PRICE = self.get_top_last_year(OLD_REPORT_DATE, REPORT_DATE)
            
            
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Full_EPS'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = round(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK,2)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            FAKE_LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Fake_Last_Full_EPS'])
            if FAKE_LAST_EARN_PER_STOCK != 0:
                FAKE_LAST_PER = round(LAST_STOCK_PRICE/FAKE_LAST_EARN_PER_STOCK,2)
            if FAKE_LAST_EARN_PER_STOCK == 0:
                FAKE_LAST_PER = 0
            #if 'LAST_STOCK_PRICE' in locals():
            REAL_LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Real_Last_Full_EPS'])
            if REAL_LAST_EARN_PER_STOCK != 0:
                REAL_LAST_PER = round(LAST_STOCK_PRICE/REAL_LAST_EARN_PER_STOCK,2)
            if REAL_LAST_EARN_PER_STOCK == 0:
                REAL_LAST_PER = 0
            #try:
            PRICE_PERCENT_CHANGE_TWO_YEAR = self.get_top_bottom_percent_change(OLD_REPORT_DATE, REPORT_DATE)
            PRICE_PERCENT_COMPARE_OLD_BIGGEST_NOW = round((OLD_STOCK_PRICE-RECENT_STOCK_PRICE)/RECENT_STOCK_PRICE*100,2)
            PRICE_PERCENT_CHANGE_YEAR = self.get_top_bottom_percent_change(LAST_REPORT_DATE, REPORT_DATE)
            PRICE_PERCENT_COMPARE_LAST_BIGGEST_NOW = round((LAST_STOCK_PRICE-RECENT_STOCK_PRICE)/RECENT_STOCK_PRICE*100,2)
            PRICE_PERCENT_CHANGE = self.check_report_month_price_data(REPORT_DATE)
            #except:
                #pass
            self.SYMBOL = index_name
            #INDEX_PERCENT_CHANGE_YEAR = self.get_top_bottom_percent_change(LAST_REPORT_DATE, REPORT_DATE)
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER), str(LAST_STOCK_PRICE), str(FAKE_LAST_PER), str(REAL_LAST_PER), str(PRICE_PERCENT_CHANGE), str(PRICE_PERCENT_CHANGE_TWO_YEAR), str(PRICE_PERCENT_COMPARE_OLD_BIGGEST_NOW), str(PRICE_PERCENT_CHANGE_YEAR), str(PRICE_PERCENT_COMPARE_LAST_BIGGEST_NOW)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price', 'Quarter_Per', 'Last_Stock_Price', 'Fake_Last_Per', 'Real_Last_Per', 'Price_Percent_Change', 'Price_Percent_Change_Two_Year', 'Price_Percent_Compare_Old_Biggest_Now', 'Price_Percent_Change_Year', 'Price_Percent_Compare_Last_Bigget_Now'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def how_report_sell(self, analyse_date, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            try:
                FULL_QUARTER_REVENUE, FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                time.sleep(1)
            except:
                pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
                print(STOCK_AMOUNT)
            except:
                pass
            #if True:
                
                try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = int(a_dictionary['QUARTER_EARN']/a_dictionary['STOCK_AMOUNT'])
                except:
                    pass
            #try:
                if 'QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN_PER_STOCK'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN_PER_STOCK'])
                        a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                    #a_dictionary['PER'] = int(STOCK_PRICE/a_dictionary['QUARTER_EARN_PER_STOCK'])
            #except:
                #a_dictionary['PER'] = 0
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])
                    
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                #pass
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_EARN_PER_STOCK']])
                print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date_Sell', 'Quarter_Revenue_Sell', 'Quarter_Operating_Profit_Sell', 'Quarter_Earn_Sell', 'Quarter_Earn_Per_Stock_Sell'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def get_per_sell(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I,'Report_Date_Sell']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price_Sell', 'Quarter_Per_Sell'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def erase_nan(self, read_csv, write_csv):
        df = pd.read_csv(read_csv)
        df.dropna(subset = ['Report_Date'], inplace = True)
        df.reset_index(drop=True, inplace = True)
        df.drop(['Unnamed: 0'], axis = 1, inplace = True)
        df.to_csv(write_csv)
    
    def find_suitable_stock(self, read_csv, write_csv):
        kosdaq_data = pd.read_csv(read_csv)
        REPORT_YEAR, REPORT_MONTH = self.get_report_year_month_new(read_csv)
        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
        kosdaq_data['ListingDate'] = pd.to_datetime(kosdaq_data['ListingDate'], format='%Y-%m-%d')
        '''kosdaq_data['Quarter_Percent_Operating_Profit'] = kosdaq_data['Quarter_Percent_Operating_Profit'].astype(float)'''
        kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
        '''kosdaq_data['Quarter_Cost_Of_Sale'] = kosdaq_data['Quarter_Revenue'] - kosdaq_data['Quarter_Operating_Profit']
        kosdaq_data['Last_Cost_Of_Sale'] = kosdaq_data['Last_REVENUE'] - kosdaq_data['Last_Operating_Profit']
        kosdaq_data['Quarter_Loss'] = (kosdaq_data['Quarter_Cost_Of_Sale']/kosdaq_data['Quarter_Revenue'])*100
        kosdaq_data['Last_Loss'] = (kosdaq_data['Last_Cost_Of_Sale']/kosdaq_data['Last_REVENUE'])*100'''
        kosdaq_data1 = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Revenue'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*1.1)]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data1['Last_Revenue'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*1.1)]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data1['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)] + 4)]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data1['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)] + 4)]
        '''kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*1.1)]
        kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data['Last_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*1.1)]'''
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -10]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -10]
        '''kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]]
        kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]]'''
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_6'] > -10000000000]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_3'] > -10000000000]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -10]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -10]
        
        kosdaq_data2 = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Revenue'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*1.1)]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data2['Last_Revenue'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*1.1)]
        '''kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)] + 1.1)]
        kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)] + 1.1)]'''
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data2['Last_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*1.2)]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data2['Last_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*1.2)]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)] > 0]
        '''kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]]
        kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]]'''
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_6'] > -10000000000]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_3'] > -10000000000]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Last_Percent_Earn'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Last_Percent_Earn'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)] > 0]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'] > -50]
        
        frames = [kosdaq_data1, kosdaq_data2]
        result = pd.concat(frames)
        result.reset_index(drop=True, inplace = True)
        result.to_csv(write_csv)
    
    def find_suitable_stock_later(self, read_csv, write_csv):
        kosdaq_data = pd.read_csv(read_csv)
        REPORT_YEAR, REPORT_MONTH = self.get_report_year_month_new(read_csv)
        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
        kosdaq_data['ListingDate'] = pd.to_datetime(kosdaq_data['ListingDate'], format='%Y-%m-%d')
        '''kosdaq_data['Quarter_Percent_Operating_Profit'] = kosdaq_data['Quarter_Percent_Operating_Profit'].astype(float)'''
        kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
        '''kosdaq_data['Quarter_Cost_Of_Sale'] = kosdaq_data['Quarter_Revenue'] - kosdaq_data['Quarter_Operating_Profit']
        kosdaq_data['Last_Cost_Of_Sale'] = kosdaq_data['Last_REVENUE'] - kosdaq_data['Last_Operating_Profit']
        kosdaq_data['Quarter_Loss'] = (kosdaq_data['Quarter_Cost_Of_Sale']/kosdaq_data['Quarter_Revenue'])*100
        kosdaq_data['Last_Loss'] = (kosdaq_data['Last_Cost_Of_Sale']/kosdaq_data['Last_REVENUE'])*100'''
        kosdaq_data = kosdaq_data[kosdaq_data['Price_Percent_Change_Year'] > kosdaq_data['Price_Percent_Compare_Last_Bigget_Now']*1.5]
        kosdaq_data = kosdaq_data[kosdaq_data['Price_Percent_Change_Year'] < 150]
        kosdaq_data = kosdaq_data[kosdaq_data['Price_Percent_Change_Two_Year'] < 300]
        #kosdaq_data = kosdaq_data[kosdaq_data['Price_Percent_Change_Two_Year'] > kosdaq_data['Price_Percent_Compare_Old_Bigget_Now']*1.5]
        kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Per'] >= 0]
        kosdaq_data = kosdaq_data[kosdaq_data['Real_Last_Per'] < 30]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'] > -50]
        kosdaq_data.reset_index(drop=True, inplace = True)
        kosdaq_data.drop_duplicates(subset = ['Symbol'], inplace = True)
        kosdaq_data.to_csv(write_csv)
        
    def how_much_earn(self, read_csv, write_csv):
        kosdaq_data = pd.read_csv(read_csv)
        movint_average_list = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            TODAY = dt.date.today()
            LAST_STOCK_PRICE = self.get_top_last_year(LAST_REPORT_DATE, REPORT_DATE)
        
                    
        
#have 2 keys 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788', '6048f2c5a065e754cee104b8c0edd8186f6b3d61', '94331def3e94db8a1aabb84e4257cec0ad3b576b', 'b1fe4037e43a6db43d8ac280d262a05e90bf9171'
#index_name = kospi('KS11'), kosdaq('KQ11')
'''akaps = how_per('6048f2c5a065e754cee104b8c0edd8186f6b3d61')
#akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(7).csv')
akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(2021_06).csv',1,1270)'''
'''paka = how_per('84acf17c852e9430843a589191330e0630cc19a7')
paka.how_report('2021-11-30','KOSPI(1).csv', 'A_PER_KOSPI(2021_6).csv',1,770)'''
INDEX_NAME = 'KQ11'
'''akaps = how_per('d088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9')
REPORT_NM = '2019_12'
akaps.how_report('2020-03-30','A_KOSDAQ.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',1,1260)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')

akaps = how_per('84acf17c852e9430843a589191330e0630cc19a7')
REPORT_NM = '2017_9'
akaps.how_report('2017-11-30','A_KOSDAQ.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',1,1260)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')

akaps = how_per('1eca9bad2afb8becdc5b1714112dd097e73da788')
REPORT_NM = '2017_6'
akaps.how_report('2017-08-30','A_KOSDAQ.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',1,1260)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')

akaps = how_per('6048f2c5a065e754cee104b8c0edd8186f6b3d61')
REPORT_NM = '2017_3'
akaps.how_report('2017-05-30','A_KOSDAQ.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',1,1260)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')

akaps = how_per('94331def3e94db8a1aabb84e4257cec0ad3b576b')
REPORT_NM = '2016_3'
akaps.how_report('2016-05-30','A_KOSDAQ.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',1,1260)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')'''
                                                                              
akaps = how_per('b1fe4037e43a6db43d8ac280d262a05e90bf9171')
REPORT_NM = '2020_6'
'''akaps.how_report('2016-08-30','A_KOSDAQ.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',1,1260)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')'''
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+')test.csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+')test.csv')

'''akaps = how_per('b1fe4037e43a6db43d8ac280d262a05e90bf9171')
REPORT_NM = '2016_9'
akaps.how_report('2018-05-30','A_KOSDAQ.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',1,1260)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')'''
'''akaps = how_per('b1fe4037e43a6db43d8ac280d262a05e90bf9171')
akaps.how_report('2021-03-30','KOSDAQ.csv', 'A_PER_KOSDAQ_real(2020_12).csv',1,1270)'''
'''akaps.get_per('AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6).csv')
akaps.how_report_sell('2020-11-30', 'AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6_9).csv')
akaps.get_per_sell('AN_PER_KOSDAQ(2020_6_9).csv', 'AN_PER_KOSDAQ(2020_6_9).csv')
paka = how_per('84acf17c852e9430843a589191330e0630cc19a7')
paka.how_report_sell('2021-03-30', 'AN_PER_KOSDAQ(2020_6).csv', 'AN_PER_KOSDAQ(2020_6_12).csv')
paka.get_per_sell('AN_PER_KOSDAQ(2020_6_12).csv', 'AN_PER_KOSDAQ(2020_6_12).csv')
#akaps.how_report('2020-08-30', 'AN_PER_KOSDAQ_real(2020_6)_test.csv','AN_PER_KOSDAQ_real(2020_6)_test(1).csv')'''

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2020-07-30


TypeError: int() argument must be a string, a bytes-like object or a number, not 'tuple'

In [17]:



#real__monthly_test




import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import datetime as dt 
from datetime import date
import time
from dateutil.relativedelta import relativedelta
import math
import warnings
import OpenDartReader
from pandas.core.common import SettingWithCopyWarning
import re
from collections.abc import Mapping

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)



class how_per():
    def __init__(self, api_key):
        self.api_key = api_key
        self.OLD_COUNT = 0
        self.OLD_SYMBOL = '가나다'
        self.SYMBOL = '가나다'
        self.TODAY = dt.date.today()
        self.BOOM_PER_LIST = []
        self.dart = OpenDartReader(self.api_key)
        
    def outlier_treatment(self, datacolumn):
        sorted(datacolumn)
        if True:
        #try:
            Q4 = np.percentile(datacolumn , 100)
            Q4_Date = datacolumn.index[datacolumn['Close'] == Q4].tolist()
            Q4_Date = Q4_Date[-1].date()
        #except:
            #return True
        return Q4, Q4_Date
    
    def low_high_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q1, Q2= np.percentile(datacolumn ,[15,100])
        except:
            return True
        return Q1, Q2
    
    def big_small_treatment(self, datacolumn):
        sorted(datacolumn)
        try:
            Q1, Q2= np.percentile(datacolumn ,[10,90])
        except:
            return True
        return Q1, Q2
    
    def has_no_fix(self,word):
        if '[기재정정]' in word:
            return True
        else:
            return False

    def get_want_fast_report(self, a_s_list, report_dt):
        a_s_list = a_s_list[a_s_list['report_nm'].str.contains('영업(잠정)실적(공정공시)', regex = False)]
        a_s_list['rcept_dt'] = pd.to_datetime(a_s_list['rcept_dt'], format = '%Y%m%d').dt.date
        a_s_list = a_s_list[a_s_list['rcept_dt'] +  relativedelta(months = 1) > report_dt]
        a_s_list = a_s_list[a_s_list['rcept_dt'] - relativedelta(months = 3) < report_dt]
        a_s_list.reset_index(drop = True, inplace = True)
        return a_s_list

    def get_before_report_time(self, report_year, report_month):
        before_report_year = report_year-1
        before_report_month = report_month
        return before_report_year, before_report_month
    
    def check_report_month_price_data(self, report_date):
        #report_plus_date = report_date + relativedelta(days = 10)
        report_minus_date = report_date - relativedelta(days = 10)
        #report_plus_date = report_plus_date.strftime("%Y-%m-%d")
        report_date = report_date.strftime("%Y-%m-%d")
        report_minus_date = report_minus_date.strftime("%Y-%m-%d")
        df_stock_price = fdr.DataReader(self.SYMBOL, report_minus_date, report_date)
        close_price_column = df_stock_price[['Close']]
        low_price, high_price = self.low_high_treatment(close_price_column)
        percent_change = round((int(high_price)-int(low_price))/int(low_price)*100,2)
        return percent_change
    
    def check_price_data(self, report_date):
        report_stock_date = date(2000,1,1)
        before_report_date = report_date - relativedelta(days = 14)
        now_report_date = report_date + relativedelta(days = 1)
        before_report_date = before_report_date.strftime("%Y-%m-%d")
        now_report_date = now_report_date.strftime("%Y-%m-%d")
        report_stock_price = 0
        df_stock_price = fdr.DataReader(self.SYMBOL, before_report_date, now_report_date)
        
        for i in range(0,14):
            report_stock_date = report_date - relativedelta(days = i)
            report_stock_date = report_stock_date.strftime("%Y-%m-%d")
            try:
                report_stock_price = df_stock_price.loc[report_stock_date, 'Close']
            except:
                continue
            if report_stock_price:
                break
            else:
                continue
        return int(report_stock_price)
    
    def get_top_last_year(self, last_report_year, recent_report_year):
        last_report_year = last_report_year.strftime("%Y-%m-%d")
        recent_report_year = recent_report_year.strftime("%Y-%m-%d")
        df_stock_price = fdr.DataReader(self.SYMBOL, last_report_year, recent_report_year)
        price_column = df_stock_price[['Close']]
        top_price_last_year, top_price_date = self.outlier_treatment(price_column)
        return int(top_price_last_year), top_price_date
    
    def get_top_bottom_percent_change(self, last_report_year, recent_report_year):
        last_report_year = last_report_year.strftime("%Y-%m-%d")
        recent_report_year = recent_report_year.strftime("%Y-%m-%d")
        df_stock_price = fdr.DataReader(self.SYMBOL, last_report_year, recent_report_year)
        price_column = df_stock_price[['Close']]
        bottom_price_last_year, top_price_last_year = self.low_high_treatment(price_column)
        percent_change = round((int(top_price_last_year)-int(bottom_price_last_year))/int(bottom_price_last_year)*100, 2)
        return percent_change
    
    '''def give_revenue_data(self):
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
            return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_revenue)
        else:
            return'''
    
    def get_stock_amount(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        if report_month == 3:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[0])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 6:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[1])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 9:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[2])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        if report_month == 12:
            self.report = self.dart.report(self.SYMBOL, '최대주주', report_year, reprt_code=code_list[3])
            self.report['stock_knd'] = self.report['stock_knd'].str.replace(" ","")
            try:
                if self.report['stock_knd'].str.contains('보통주', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('보통주', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            try:
                if self.report['stock_knd'].str.contains('의결권이있는', regex = False).any():
                    report = self.report[self.report['stock_knd'].str.contains('의결권이있는', regex = False)]
                    report = report[report['nm'].str.contains('계', regex = False)]
                    report.reset_index(drop = True, inplace = True)
                    individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                    individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                    individual_stock_amount = self.get_only_number(individual_stock_amount)
                    individual_stock_percent = float(individual_stock_percent)
                    stock_amount = individual_stock_amount*(100/individual_stock_percent)
                    return int(stock_amount)
            except:
                pass
            if self.report['stock_knd'].str.contains('-', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('-', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
            if self.report['stock_knd'].str.contains('계', regex = False).any():
                report = self.report[self.report['stock_knd'].str.contains('계', regex = False)]
                report = report[report['nm'].str.contains('계', regex = False)]
                report.reset_index(drop = True, inplace = True)
                individual_stock_amount = report.loc[0, 'trmend_posesn_stock_co']
                individual_stock_percent = report.loc[0, 'trmend_posesn_stock_qota_rt']
                individual_stock_amount = self.get_only_number(individual_stock_amount)
                individual_stock_percent = float(individual_stock_percent)
                stock_amount = individual_stock_amount*(100/individual_stock_percent)
                return int(stock_amount)
        else:
            return

        
    def give_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        self.report['account_nm'] = self.report['account_nm'].str.replace(" ","")
        if self.report['account_nm'].str.contains('지배기업의소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의소유주', regex = False)]
                    report = report[report['account_nm'].str.contains('귀속될', regex = False)]
                    report = report[report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        
    def give_operating_profit(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit= self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit = self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return -int(quarter_operating_profit)
        else:
            return 0
        
    def give_revenue_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs_GrossProfit'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs_GrossProfit'), 'thstrm_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
    def give_last_earn_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의 소유주', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('귀속될', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
    
   
    def give_last_revenue_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs_GrossProfit')]).any():
            if ('IS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_GrossProfit'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_GrossProfit')]).any():
            if ('CIS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_GrossProfit'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                return int(quarter_operating_profit)
        else:
            return 0 
    
    def give_last_operating_profit(self):
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'frmtrm_q_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_q_amount']
                return -int(quarter_operating_profit)
        else:
            return 0 
    
    def give_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('BS', 'ifrs_Assets')]).any():
            if ('BS', 'ifrs_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return 0 
    
    def give_last_asset(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'frmtrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('BS', 'ifrs_Assets')]).any():
            if ('BS', 'ifrs_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs_Assets'), 'frmtrm_amount']
                return int(quarter_asset)
        else:
            return 0 
        
    def give_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('BS', 'ifrs_Liabilities')]).any():
            if ('BS', 'ifrs_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return 0 
        
    def give_last_liability(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'frmtrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('BS', 'ifrs_Liabilities')]).any():
            if ('BS', 'ifrs_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs_Liabilities'), 'frmtrm_amount']
                return int(quarter_liability)
        else:
            return 0 
        
    def give_continue_earn(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def give_last_continue_earn(self):
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_q_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_cashflow(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0 
    
    def give_last_cashflow(self):
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_q_amount']
                return int(quarter_earn)
        else:
            return 0 
        
    def check_lenz(self, df):
        return df is None
    
    def check_empty(self, df):
        return df.empty
        
    def get_fs_of_time(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 3:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL , str(report_year), reprt_code=code_list[0], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[0], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
        if report_month == 6:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[1], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_operating_profit()
                        QUARTER_CASHFLOW = self.give_cashflow()
                        QUARTER_REVENUE = self.give_revenue_data()
                        CONTINUE_EARN = self.give_continue_earn()
                        QUARTER_ASSET = self.give_asset()
                        QUARTER_LIABILITY = self.give_liability()
                        LAST_EARN = self.give_last_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_operating_profit()
                        LAST_CASHFLOW = self.give_last_cashflow()
                        LAST_REVENUE = self.give_last_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_continue_earn()
                        LAST_ASSET = self.give_last_asset
                        LAST_LIABILITY = self.give_last_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
        else:
            return 
    
    def give_add_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        self.report['account_nm'] = self.report['account_nm'].str.replace(" ","")
        if self.report['account_nm'].str.contains('지배기업의소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의소유주', regex = False)]
                    report = report[report['account_nm'].str.contains('귀속될', regex = False)]
                    report = report[report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        
    def give_add_operating_profit(self):
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit= self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            quarter_operating_profit = self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return -int(quarter_operating_profit)
        else:
            return 0
        
    def give_add_revenue_data(self):
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_add_amount']
            #print(quarter_earn, quarter_earn_per_stock)
            return int(float(quarter_revenue))
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
    def give_last_add_earn_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의 소유주', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('귀속될', regex = False)]
                    report = self.report[self.report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
    
   
    def give_last_add_revenue_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs_GrossProfit')]).any():
            if ('IS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_GrossProfit')]).any():
            if ('CIS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_GrossProfit'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
    def give_last_add_operating_profit(self):
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'frmtrm_add_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업손실', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업손실', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return -int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_add_amount']
                return -int(quarter_operating_profit)
        else:
            return 0
    
    def give_add_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('BS', 'ifrs_Assets')]).any():
            if ('BS', 'ifrs_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return 0
    
    def give_last_add_asset(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'frmtrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('BS', 'ifrs_Assets')]).any():
            if ('BS', 'ifrs_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs_Assets'), 'frmtrm_amount']
                return int(quarter_asset)
        else:
            return 0
        
    def give_add_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('BS', 'ifrs_Liabilities')]).any():
            if ('BS', 'ifrs_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return 0
        
    def give_last_add_liability(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'frmtrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('BS', 'ifrs_Liabilities')]).any():
            if ('BS', 'ifrs_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs_Liabilities'), 'frmtrm_amount']
                return int(quarter_liability)
        else:
            return 0
        
    def give_add_continue_earn(self):
        #to erase all the white spaces
        #self.report.account_id = self.report.account_id.str.replace(' ', '')
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['thstrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_add_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def give_last_add_continue_earn(self):
        #self.report.set_index(['sj_div','account_id'], inplace = True)
        report_nan_list = self.report.index[self.report['frmtrm_add_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_add_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_add_cashflow(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0, 0
    
    def give_last_add_cashflow(self):
        report_nan_list = self.report.index[self.report['frmtrm_q_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_q_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_q_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def get_fs_of_add_time(self, report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 9:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_add_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_add_operating_profit()
                        QUARTER_CASHFLOW = self.give_add_cashflow()
                        QUARTER_REVENUE = self.give_add_revenue_data()
                        CONTINUE_EARN = self.give_add_continue_earn()
                        QUARTER_ASSET = self.give_add_asset()
                        QUARTER_LIABILITY = self.give_add_liability()
                        LAST_EARN = self.give_last_add_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_add_operating_profit()
                        LAST_CASHFLOW = self.give_last_add_cashflow()
                        LAST_REVENUE = self.give_last_add_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_add_continue_earn()
                        LAST_ASSET = self.give_last_add_asset()
                        LAST_LIABILITY = self.give_last_add_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[2], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        QUARTER_EARN = self.give_add_earn_data()
                        QUARTER_OPERATING_PROFIT = self.give_add_operating_profit()
                        QUARTER_CASHFLOW = self.give_add_cashflow()
                        QUARTER_REVENUE = self.give_add_revenue_data()
                        CONTINUE_EARN = self.give_add_continue_earn()
                        QUARTER_ASSET = self.give_add_asset()
                        QUARTER_LIABILITY = self.give_add_liability()
                        LAST_EARN = self.give_last_add_earn_data()
                        LAST_OPERATING_PROFIT = self.give_last_add_operating_profit()
                        LAST_CASHFLOW = self.give_last_add_cashflow()
                        LAST_REVENUE = self.give_last_add_revenue_data()
                        LAST_CONTINUE_EARN = self.give_last_add_continue_earn()
                        LAST_ASSET = self.give_last_add_asset()
                        LAST_LIABILITY = self.give_last_add_liability()
                        
                        return QUARTER_REVENUE, QUARTER_OPERATING_PROFIT, QUARTER_CASHFLOW, QUARTER_EARN, CONTINUE_EARN, LAST_EARN, LAST_OPERATING_PROFIT, LAST_CASHFLOW, LAST_REVENUE, LAST_CONTINUE_EARN, QUARTER_ASSET, QUARTER_LIABILITY, LAST_ASSET, LAST_LIABILITY
            #except:
                #pass
        else:
            return 
        
    def give_last_full_earn_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        self.report['account_nm'] = self.report['account_nm'].str.replace(" ","")
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의소유주', regex = False)]
                    report = report[report['account_nm'].str.contains('귀속될', regex = False)]
                    report = report[report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_last_full_revenue_data(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs_GrossProfit')]).any():
            if ('IS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_GrossProfit')]).any():
            if ('CIS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_GrossProfit'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
    def give_last_full_operating_profit(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'thstrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'thstrm_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
     
    
    def give_last_full_asset(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('BS', 'ifrs_Assets')]).any():
            if ('BS', 'ifrs_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs_Assets'), 'thstrm_amount']
                return int(quarter_asset)
        else:
            return
        
    def give_last_full_liability(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('BS', 'ifrs_Liabilities')]).any():
            if ('BS', 'ifrs_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs_Liabilities'), 'thstrm_amount']
                return int(quarter_liability)
        else:
            return 0 
        
    def give_last_full_continue_earn(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_last_full_cashflow(self):
        report_nan_list = self.report.index[self.report['thstrm_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs_CashFlowsFromUsedInOperatingActivities'), 'thstrm_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_last_add_full_earn_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        self.report['account_nm'] = self.report['account_nm'].str.replace(" ","")
        if self.report['account_nm'].str.contains('지배기업의 소유주', regex = False).any():
            if self.report['account_nm'].str.contains('귀속될', regex = False).any():
                if self.report['account_nm'].str.contains('당기순이익', regex = False).any():
                    report = self.report[self.report['account_nm'].str.contains('지배기업의소유주', regex = False)]
                    report = report[report['account_nm'].str.contains('귀속될', regex = False)]
                    report = report[report['account_nm'].str.contains('당기순이익', regex = False)]
                    if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_amount']
                        return int(quarter_earn)
                    if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                        quarter_earn = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_amount']
                        return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('IS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossAttributableToOwnersOfParent')]).any():
            if ('CIS','ifrs_ProfitLossAttributableToOwnersOfParent') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossAttributableToOwnersOfParent'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
            if ('IS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLoss'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLoss')]).any():
            if ('CIS', 'ifrs-full_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLoss'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLoss')]).any():
            if ('IS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLoss'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLoss')]).any():
            if ('CIS', 'ifrs_ProfitLoss') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLoss'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_last_add_full_revenue_data(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs_Revenue')]).any():
            if ('IS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_Revenue'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_Revenue')]).any():
            if ('CIS', 'ifrs_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_Revenue'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_Revenue')]).any():
            if ('IS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_Revenue'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_Revenue')]).any():
            if ('CIS', 'ifrs-full_Revenue') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_Revenue'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs-full_GrossProfit')]).any():
            if ('IS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs-full_GrossProfit'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs-full_GrossProfit')]).any():
            if ('CIS', 'ifrs-full_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs-full_GrossProfit'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('IS', 'ifrs_GrossProfit')]).any():
            if ('IS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('IS','ifrs_GrossProfit'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report.index.isin([('CIS', 'ifrs_GrossProfit')]).any():
            if ('CIS', 'ifrs_GrossProfit') not in report_nan_list:
                quarter_revenue = self.report.loc[('CIS','ifrs_GrossProfit'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_revenue)
        if self.report['account_nm'].str.contains('영업수익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업수익', regex = False)]
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
    def give_last_add_full_operating_profit(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'dart_OperatingIncomeLoss')]).any():
            if ('IS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit= self.report.loc[('IS','dart_OperatingIncomeLoss'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report.index.isin([('CIS', 'dart_OperatingIncomeLoss')]).any():
            if ('CIS','dart_OperatingIncomeLoss') not in report_nan_list:
                quarter_operating_profit = self.report.loc[('CIS','dart_OperatingIncomeLoss'), 'frmtrm_amount']
                #print(quarter_earn, quarter_earn_per_stock)
                return int(quarter_operating_profit)
        if self.report['account_nm'].str.contains('영업이익', regex = False).any():
            report = self.report[self.report['account_nm'].str.contains('영업이익', regex = False)]
            if report.index.isin([('IS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('IS','-표준계정코드 미사용-'), 'frmtrm_amount']
                return int(quarter_operating_profit)
            if report.index.isin([('CIS', '-표준계정코드 미사용-')]).any():
                quarter_operating_profit = report.loc[('CIS','-표준계정코드 미사용-'), 'frmtrm_amount']
                return int(quarter_operating_profit)
        else:
            return 0
    
     
    
    def give_last_add_full_asset(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Assets')]).any():
            if ('BS', 'ifrs-full_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs-full_Assets'), 'frmtrm_amount']
                return int(quarter_asset)
        if self.report.index.isin([('BS', 'ifrs_Assets')]).any():
            if ('BS', 'ifrs_Assets') not in report_nan_list:
                quarter_asset = self.report.loc[('BS','ifrs_Assets'), 'frmtrm_amount']
                return int(quarter_asset)
        else:
            return 0
        
    def give_last_add_full_liability(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('BS', 'ifrs-full_Liabilities')]).any():
            if ('BS', 'ifrs-full_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs-full_Liabilities'), 'frmtrm_amount']
                return int(quarter_liability)
        if self.report.index.isin([('BS', 'ifrs_Liabilities')]).any():
            if ('BS', 'ifrs_Liabilities') not in report_nan_list:
                quarter_liability = self.report.loc[('BS','ifrs_Liabilities'), 'frmtrm_amount']
                return int(quarter_liability)
        else:
            return 0
        
    def give_last_add_full_continue_earn(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('IS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs-full_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('IS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('IS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('IS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CIS', 'ifrs_ProfitLossFromContinuingOperations')]).any():
            if ('CIS', 'ifrs_ProfitLossFromContinuingOperations') not in report_nan_list:
                quarter_earn = self.report.loc[('CIS','ifrs_ProfitLossFromContinuingOperations'), 'frmtrm_amount']
                return int(quarter_earn)
        else:
            return 0
    
    def give_last_add_full_cashflow(self):
        report_nan_list = self.report.index[self.report['frmtrm_amount'] == ''].tolist()
        if self.report.index.isin([('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs-full_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs-full_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_amount']
                return int(quarter_earn)
        if self.report.index.isin([('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities')]).any():
            if ('CF', 'ifrs_CashFlowsFromUsedInOperatingActivities') not in report_nan_list:
                quarter_earn = self.report.loc[('CF','ifrs_CashFlowsFromUsedInOperatingActivities'), 'frmtrm_amount']
                return int(quarter_earn)
        else:
            return 0
        
    def get_fs_of_last_full(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        LAST_FULL_EARN = self.give_last_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        LAST_FULL_CASHFLOW = self.give_last_full_cashflow()
                        LAST_FULL_ASSET = self.give_last_full_asset()
                        LAST_FULL_LIABILITY = self.give_last_full_liability()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_CASHFLOW, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN, LAST_FULL_ASSET, LAST_FULL_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        LAST_FULL_EARN = self.give_last_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        LAST_FULL_CASHFLOW = self.give_last_full_cashflow()
                        LAST_FULL_ASSET = self.give_last_full_asset()
                        LAST_FULL_LIABILITY = self.give_last_full_liability()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_CASHFLOW, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN, LAST_FULL_ASSET, LAST_FULL_LIABILITY
            #except:
                #pass
        else:
            return
        
    def get_fs_of_add_last_full(self,report_year, report_month):
        code_list = ['11013', '11012', '11014', '11011']
        QUARTER_EARN = 0
        PLUS_EARN = 0
        QUARTER_EARN_PER_STOCK = 0
        PLUS_EARN_PER_STOCK = 0
        self.report = pd.DataFrame()
        if report_month == 12:
            if True:
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='CFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        FULL_EARN = self.give_last_full_earn_data()
                        FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        FULL_REVENUE = self.give_last_full_revenue_data()
                        FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        FULL_CASHFLOW = self.give_last_full_cashflow()
                        FULL_ASSET = self.give_last_full_asset()
                        FULL_LIABILITY = self.give_last_full_liability()
                        LAST_FULL_EARN = self.give_last_add_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_add_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_add_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_add_full_continue_earn()
                        LAST_FULL_CASHFLOW = self.give_last_add_full_cashflow()
                        LAST_FULL_ASSET = self.give_last_add_full_asset()
                        LAST_FULL_LIABILITY = self.give_last_add_full_liability()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return FULL_REVENUE, FULL_OPERATING_PROFIT, FULL_CASHFLOW, FULL_EARN, FULL_CONTINUE_EARN, LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_CASHFLOW, LAST_FULL_REVENUE, FULL_CONTINUE_EARN, FULL_ASSET, FULL_LIABILITY, LAST_FULL_ASSET, LAST_FULL_LIABILITY
            #except:
                #pass
            #try:
                self.report = self.dart.finstate_all(self.SYMBOL, str(report_year), reprt_code=code_list[3], fs_div='OFS')
                if not self.check_lenz(self.report):
                    if not self.check_empty(self.report):
                        self.report.set_index(['sj_div','account_id'], inplace = True)
                        #QUARTER_ASSET = self.give_oldasset()
                        #QUARTER_LIABILITY = self.give_liability()
                        FULL_EARN = self.give_last_full_earn_data()
                        FULL_OPERATING_PROFIT = self.give_last_full_operating_profit()
                        FULL_REVENUE = self.give_last_full_revenue_data()
                        FULL_CONTINUE_EARN = self.give_last_full_continue_earn()
                        FULL_CASHFLOW = self.give_last_full_cashflow()
                        FULL_ASSET = self.give_last_full_asset()
                        FULL_LIABILITY = self.give_last_full_liability()
                        LAST_FULL_EARN = self.give_last_add_full_earn_data()
                        LAST_FULL_OPERATING_PROFIT = self.give_last_add_full_operating_profit()
                        LAST_FULL_REVENUE = self.give_last_add_full_revenue_data()
                        LAST_FULL_CONTINUE_EARN = self.give_last_add_full_continue_earn()
                        LAST_FULL_CASHFLOW = self.give_last_add_full_cashflow()
                        LAST_FULL_ASSET = self.give_last_add_full_asset()
                        LAST_FULL_LIABILITY = self.give_last_add_full_liability()
                        #print(LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_REVENUE, LAST_FULL_CONTINUE_EARN)
                        return FULL_REVENUE, FULL_OPERATING_PROFIT, FULL_CASHFLOW, FULL_EARN, FULL_CONTINUE_EARN, LAST_FULL_EARN, LAST_FULL_OPERATING_PROFIT, LAST_FULL_CASHFLOW, LAST_FULL_REVENUE, FULL_CONTINUE_EARN, FULL_ASSET, FULL_LIABILITY, LAST_FULL_ASSET, LAST_FULL_LIABILITY
            #except:
                #pass
        else:
            return
        
    
    def get_real_report_month(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year
            report_month = 6
        if report_month == 6:
            report_year = report_year
            report_month = 9
        if report_month == 9:
            report_year = report_year
            report_month = 12
        if report_month == 12:
            report_year = report_year + 1
            report_month = 3
        return report_year, report_month
    
    def get_predict_earn_per_stock(self, report_month, earn_per_stock):
        earn_per_stock = int(earn_per_stock)
        
        '''if report_month == 3:
            earn_per_stock = earn_per_stock*4
        if report_month == 6:
            earn_per_stock = earn_per_stock*2
        if report_month == 9:
            earn_per_stock = earn_per_stock*(4/3)
        if report_month == 12:
            earn_per_stock = earn_per_stock'''
        earn_per_stock = earn_per_stock*4
        
        return earn_per_stock
    
    def get_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            report_dt = report_dt.date()
            break
        #return report_nm, report_dt
        return report_nm, report_dt
    
    def get_fast_report_date(self, s_list):
        report_nm = ''
        report_dt = dt.date(1990,1,1)
        for r in range(s_list.index.size):
            report_nm = s_list.loc[r, 'report_nm']
            if self.has_no_fix(report_nm):
                continue
            report_dt = s_list.loc[r, 'rcept_dt']
            #report_dt = dt.datetime.strptime(report_dt, "%Y%m%d")
            #report_dt = report_dt.date()
            break
        return report_nm, report_dt
    
    def get_last_year_report(self, report_date):
        delimiters = "-"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_date)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        last_year = int(nm_num_list[0])-1
        last_month = int(nm_num_list[1])
        return last_year, last_month
    
    def get_only_number(self, report_data):
        delimiters = ","
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_data)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        element_amount = len(nm_num_list)
        report_data = 0
        for i in range(len(nm_num_list)):
            report_data = report_data + nm_num_list[i]*(10**((element_amount - i - 1)*3))
        return report_data
    
    def get_report_year_month_old(self, report_nm):
        delimiters = "_"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month(self, report_nm):
        delimiters = "(", ".", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_new(self, report_nm):
        delimiters = "(", "_", ")"
        regexPattern = '|'.join(map(re.escape, delimiters))
        nm_list = re.split(regexPattern, report_nm)
        nm_num_list = [int(s) for s in nm_list if s.isdigit()]
        report_year = int(nm_num_list[0])
        report_month = int(nm_num_list[1])
        return report_year, report_month
    
    def get_report_year_month_one_back(self, report_year, report_month):
        if report_month == 3:
            report_year = report_year - 1
            report_month = 12
            return report_year, report_month
        if report_month == 6:
            report_year = report_year
            report_month = 3
            return report_year, report_month
        if report_month == 9:
            report_year = report_year
            report_month = 6
            return report_year, report_month
        if report_month == 12:
            report_year = report_year
            report_month = 9
            return report_year, report_month
            
    def get_symbol_six(self, analy_kosdaq, i):
        new_symbol = analy_kosdaq.loc[i, 'Symbol']
        if new_symbol != self.OLD_SYMBOL:
            self.OLD_COUNT = 0
            symbol = analy_kosdaq.loc[i, 'Symbol']
            symbol_length = self.digit_length(symbol)
            if symbol_length != 6:
                zero_need = 6 - symbol_length
                self.SYMBOL = '0'*zero_need + str(symbol)
            if symbol_length == 6:
                self.SYMBOL = str(symbol)
            self.OLD_SYMBOL = new_symbol
            return
        if new_symbol == self.OLD_SYMBOL:
            self.OLD_COUNT += 1
        return 
    
    #자릿수 찾기
    def digit_length(self, n):
        ans = 0
        n= int(n)

        while n:
            n //= 10
            ans += 1

        return ans


    def how_report(self, report_nm, analyse_date, read_csv, write_csv,start,end):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(start,end):
            try:
                self.get_symbol_six(ANALY_KOSDAQ, I)
                #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
                ANALYSE_DATE = analyse_date
                ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
                ANALYSE_DATE = ANALYSE_DATE.date()
                START_DATE = ANALYSE_DATE - relativedelta(months = 18)
                self.START_DATE = START_DATE.strftime("%Y-%m-%d")
                ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
                S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
                a_dictionary = {} 
                REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
                REPORT_NM = report_nm
                REPORT_YEAR, REPORT_MONTH = self.get_report_year_month_old(REPORT_NM)
                OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            except:
                continue
            #if True:
            try:
                if REPORT_MONTH != 12:
                    #if True:
                    try:
                        QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R, LAST_ASSET_R, LAST_LIABILITY_R = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                        #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                        a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_REVENUE_R
                        a_dictionary['QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_OPERATING_PROFIT_R
                        a_dictionary['QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_CASHFLOW_R
                        a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_EARN_R
                        a_dictionary['QUARTER_CONTINUE_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = CONTINUE_EARN_R
                        a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_REVENUE_R
                        a_dictionary['LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_OPERATING_PROFIT_R
                        a_dictionary['LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_CASHFLOW_R
                        a_dictionary['LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_EARN_R
                        a_dictionary['LAST_CONTINUE_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_CONTINUE_EARN_R
                        a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_ASSET_R
                        a_dictionary['QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_LIABILITY_R
                        a_dictionary['LAST_ASSET' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_ASSET_R
                        a_dictionary['LAST_LIABILITY' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_LIABILITY_R
                        #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                        a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    except:
                        pass
                if REPORT_MONTH == 12:
                    #if True:
                    try:
                        QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R, LAST_ASSET_R, LAST_LIABILITY_R = self.get_fs_of_add_last_full(REPORT_YEAR, REPORT_MONTH)
                        #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                        #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                        a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                        QUARTER_REVENUE_ADD_R, QUARTER_OPERATING_PROFIT_ADD_R, QUARTER_CASHFLOW_ADD_R, QUARTER_EARN_ADD_R, CONTINUE_EARN_ADD_R, LAST_EARN_ADD_R, LAST_OPERATING_PROFIT_ADD_R, LAST_CASHFLOW_ADD_R, LAST_REVENUE_ADD_R, LAST_CONTINUE_EARN_ADD_R, QUARTER_ASSET_ADD_R, QUARTER_LIABILITY_ADD_R, LAST_ASSET_ADD_R, LAST_LIABILITY_ADD_R = self.get_fs_of_add_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                        a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_REVENUE_R) - int(QUARTER_REVENUE_ADD_R)
                        a_dictionary['QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_OPERATING_PROFIT_R) - int(QUARTER_OPERATING_PROFIT_ADD_R)
                        a_dictionary['QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_CASHFLOW_R) - int(QUARTER_CASHFLOW_ADD_R)
                        a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_EARN_R) - int(QUARTER_EARN_ADD_R)
                        a_dictionary['QUARTER_CONTINUE_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(CONTINUE_EARN_R) - int(CONTINUE_EARN_ADD_R)
                        a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_REVENUE_R) - int(LAST_REVENUE_ADD_R)
                        a_dictionary['LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_OPERATING_PROFIT_R) - int(LAST_OPERATING_PROFIT_ADD_R)
                        a_dictionary['LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_CASHFLOW_R) - int(LAST_CASHFLOW_ADD_R)
                        a_dictionary['LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_EARN_R) - int(LAST_EARN_ADD_R)
                        a_dictionary['LAST_CONTINUE_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_CONTINUE_EARN_R) - int(LAST_CONTINUE_EARN_ADD_R)
                        a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_ASSET_R) - int(QUARTER_ASSET_ADD_R)
                        a_dictionary['QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_LIABILITY_R) - int(QUARTER_LIABILITY_ADD_R)
                        a_dictionary['LAST_ASSET' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_ASSET_R) - int(LAST_ASSET_ADD_R)
                        a_dictionary['LAST_LIABILITY' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_LIABILITY_R) - int(LAST_LIABILITY_ADD_R)
                        time.sleep(1)
                    except:
                        pass

                if BACK_REPORT_MONTH != 12:
                    #if True:
                    try:
                        QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R, LAST_ASSET_R, LAST_LIABILITY_R = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                        #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                        a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = QUARTER_REVENUE_R
                        a_dictionary['QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR)+ '_' + str(BACK_REPORT_MONTH)] = QUARTER_OPERATING_PROFIT_R
                        a_dictionary['QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = QUARTER_CASHFLOW_R
                        a_dictionary['QUARTER_EARN' + str(BACK_REPORT_YEAR)+ '_' + str(BACK_REPORT_MONTH)] = QUARTER_EARN_R
                        a_dictionary['QUARTER_CONTINUE_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = CONTINUE_EARN_R
                        a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_REVENUE_R
                        a_dictionary['LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_OPERATING_PROFIT_R
                        a_dictionary['LAST_CASHFLOW' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_CASHFLOW_R
                        a_dictionary['LAST_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_EARN_R
                        a_dictionary['LAST_CONTINUE_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_CONTINUE_EARN_R
                        a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = QUARTER_ASSET_R
                        a_dictionary['QUARTER_LIABILITY' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = QUARTER_LIABILITY_R
                        a_dictionary['LAST_ASSET' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_ASSET_R
                        a_dictionary['LAST_LIABILITY' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_LIABILITY_R
                        #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                        #a_dictionary['BACK_REPORT_DATE'] = BACK_REPORTREPORT_DT.strftime("%Y-%m-%d")
                        time.sleep(1)
                    except:
                        pass
                if BACK_REPORT_MONTH == 12:
                    #if True:
                    try:
                        QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R, LAST_ASSET_R, LAST_LIABILITY_R = self.get_fs_of_add_last_full(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                        BACK_BACK_REPORT_YEAR, BACK_BACK_REPORT_MONTH = self.get_report_year_month_one_back(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                        time.sleep(0.1)
                        QUARTER_REVENUE_ADD_R, QUARTER_OPERATING_PROFIT_ADD_R, QUARTER_CASHFLOW_ADD_R, QUARTER_EARN_ADD_R, CONTINUE_EARN_ADD_R, LAST_EARN_ADD_R, LAST_OPERATING_PROFIT_ADD_R, LAST_CASHFLOW_ADD_R, LAST_REVENUE_ADD_R, LAST_CONTINUE_EARN_ADD_R, QUARTER_ASSET_ADD_R, QUARTER_LIABILITY_ADD_R, LAST_ASSET_ADD_R, LAST_LIABILITY_ADD_R = self.get_fs_of_add_time(BACK_BACK_REPORT_YEAR, BACK_BACK_REPORT_MONTH)
                        a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_REVENUE_R) - int(QUARTER_REVENUE_ADD_R)
                        a_dictionary['QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_OPERATING_PROFIT_R) - int(QUARTER_OPERATING_PROFIT_ADD_R)
                        a_dictionary['QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_CASHFLOW_R) - int(QUARTER_CASHFLOW_ADD_R)
                        a_dictionary['QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_EARN_R) - int(QUARTER_EARN_ADD_R)
                        a_dictionary['QUARTER_CONTINUE_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(CONTINUE_EARN_R) - int(CONTINUE_EARN_ADD_R)
                        a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_REVENUE_R) - int(LAST_REVENUE_ADD_R)
                        a_dictionary['LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_OPERATING_PROFIT_R) - int(LAST_OPERATING_PROFIT_ADD_R)
                        a_dictionary['LAST_CASHFLOW' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_CASHFLOW_R) - int(LAST_CASHFLOW_ADD_R)
                        a_dictionary['LAST_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_EARN_R) - int(LAST_EARN_ADD_R)
                        a_dictionary['LAST_CONTINUE_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_CONTINUE_EARN_R) - int(LAST_CONTINUE_EARN_ADD_R)
                        a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_ASSET_R) - int(QUARTER_ASSET_ADD_R)
                        a_dictionary['QUARTER_LIABILITY' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_LIABILITY_R) - int(QUARTER_LIABILITY_ADD_R)
                        a_dictionary['LAST_ASSET' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_ASSET_R) - int(LAST_ASSET_ADD_R)
                        a_dictionary['LAST_LIABILITY' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_LIABILITY_R) - int(LAST_LIABILITY_ADD_R)
                        #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                        time.sleep(1)
                    except:
                        pass
            except:
                pass
                
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            try:
                TODAY = self.TODAY.strftime("%Y-%m-%d")
                R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
                RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
                RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            except:
                pass
            #if True:
            try:
                LAST_FULL_EARN_R, LAST_FULL_OPERATING_PROFIT_R, LAST_FULL_CASHFLOW_R, LAST_FULL_REVENUE_R, LAST_FULL_CONTINUE_EARN_R, LAST_FULL_ASSET_R, LAST_FULL_LIABILITY_R = self.get_fs_of_last_full(OLD_REPORT_YEAR, 12)
                a_dictionary['LAST_FULL_REVENUE'] = LAST_FULL_REVENUE_R
                a_dictionary['LAST_FULL_OPERATING_PROFIT'] = LAST_FULL_OPERATING_PROFIT_R
                a_dictionary['LAST_FULL_CASHFLOW'] = LAST_FULL_CASHFLOW_R
                a_dictionary['LAST_FULL_EARN'] = LAST_FULL_EARN_R
                a_dictionary['LAST_FULL_CONTINUE_EARN'] = LAST_FULL_CONTINUE_EARN_R
                a_dictionary['LAST_FULL_ASSET'] = LAST_FULL_ASSET_R
                a_dictionary['LAST_FULL_LIABILITY'] = LAST_FULL_LIABILITY_R
                time.sleep(1)
            except:
                pass
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
                time.sleep(1)
            except:
                pass
           
            #if True:
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        if 'QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                            a_dictionary['QUARTER_EPS'] = int(int(a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_EARN' + str(REPORT_YEAR - 1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        if 'LAST_EARN' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():                                                                                                           
                            a_dictionary['LAST_EPS'] = int(int(a_dictionary['LAST_EARN' + str(REPORT_YEAR - 1) + '_' + str(REPORT_MONTH)])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['REAL_LAST_FULL_EPS'] = int(int(a_dictionary['LAST_FULL_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        if 'QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                            a_dictionary['QUARTER_BPS'] = int((int(a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)]) - (int(a_dictionary['QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_LIABILITY' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])))/int(a_dictionary['STOCK_AMOUNT']))
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_FULL_ASSET' in a_dictionary.keys():
                        if 'LAST_FULL_LIABILITY'in a_dictionary.keys():
                            a_dictionary['LAST_FULL_BPS'] = int((int(a_dictionary['LAST_FULL_ASSET'])-int(a_dictionary['LAST_FULL_LIABILITY']))/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_EPS' in a_dictionary.keys():
                        if 'STOCK_AMOUNT' in a_dictionary.keys():
                            QUARTER_FULL_EARN = int(self.get_predict_earn_per_stock(REPORT_MONTH, int(a_dictionary['QUARTER_EPS']) * int(a_dictionary['STOCK_AMOUNT'])))
                            a_dictionary['QUARTER_ROA'] = round(QUARTER_FULL_EARN / (int(a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)]))*100, 2)
                if 'LAST_FULL_ASSET' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_ROA'] = round(int(a_dictionary['LAST_FULL_EARN']) / int(a_dictionary['LAST_FULL_ASSET'])*100, 2)
            except:
                pass
            try:
                if 'QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_EPS' in a_dictionary.keys():
                        if 'STOCK_AMOUNT' in a_dictionary.keys():
                            QUARTER_FULL_EARN = int(self.get_predict_earn_per_stock(REPORT_MONTH, int(a_dictionary['QUARTER_EPS']) * int(a_dictionary['STOCK_AMOUNT'])))
                            a_dictionary['QUARTER_ROE'] = round(QUARTER_FULL_EARN / (int(a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)]) - (int(a_dictionary['QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_LIABILITY' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])))*100, 2)
                if 'LAST_FULL_ASSET' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_ROE'] = round(int(a_dictionary['LAST_FULL_EARN']) / (int(a_dictionary['LAST_FULL_ASSET']) - int(a_dictionary['LAST_FULL_LIABILITY']))*100, 2)
            except:
                pass
            try:
                if 'QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_OPERATING_PROFIT' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_FULL_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_FULL_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_FULL_PERCENT_OPERATING_PROFIT'] = round(int(a_dictionary['LAST_FULL_OPERATING_PROFIT'])/int(a_dictionary['LAST_FULL_REVENUE'])*100, 2)
                if 'QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_EARN' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_EARN' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['LAST_EARN' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_FULL_REVENUE' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_PERCENT_EARN'] = round(int(a_dictionary['LAST_FULL_EARN'])/int(a_dictionary['LAST_FULL_REVENUE'])*100, 2)
                if 'QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_CASHFLOW' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_CASHFLOW' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['LAST_CASHFLOW' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_FULL_REVENUE' in a_dictionary.keys():
                    if 'LAST_FULL_CASHFLOW' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_PERCENT_CASHFLOW'] = round(int(a_dictionary['LAST_FULL_CASHFLOW'])/int(a_dictionary['LAST_FULL_REVENUE'])*100, 2)
            except:
                pass
            try:
                if 'QUARTER_EPS' in a_dictionary.keys():
                    a_dictionary['QUARTER_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EPS']))
                '''if 'QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])'''
            except:
                pass
            try:
                if 'LAST_EPS' in a_dictionary.keys():
                    a_dictionary['FAKE_LAST_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['LAST_EPS']))   
                '''if 'OLD_QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_EARN'])
                if 'OLD_QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_OPERATING_PROFIT'])
                if 'OLD_QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_REVENUE'])'''
            except:
                pass
            #if True:
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['STOCK_AMOUNT'], a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_CONTINUE_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)],\
                                            a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_PERCENT_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_PERCENT_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], \
                                           a_dictionary['QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_CONTINUE_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_PERCENT_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], \
                                           a_dictionary['QUARTER_PERCENT_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_FULL_EPS'], a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_CONTINUE_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], \
                                           a_dictionary['LAST_PERCENT_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_PERCENT_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_PERCENT_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_CASHFLOW' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], \
                                           a_dictionary['LAST_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_CONTINUE_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_PERCENT_OPERATING_PROFIT' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_PERCENT_CASHFLOW' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_PERCENT_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['FAKE_LAST_FULL_EPS'], \
                                           a_dictionary['LAST_FULL_REVENUE'], a_dictionary['LAST_FULL_OPERATING_PROFIT'], a_dictionary['LAST_FULL_CASHFLOW'], a_dictionary['LAST_FULL_EARN'], a_dictionary['REAL_LAST_FULL_EPS'], a_dictionary['QUARTER_BPS'],a_dictionary['LAST_FULL_BPS'], a_dictionary['QUARTER_ROA'], a_dictionary['LAST_FULL_ROA'], a_dictionary['QUARTER_ROE'], a_dictionary['LAST_FULL_ROE']])
                print(self.OLD_SYMBOL)
                #print(self.BOOM_PER_LIST)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Stock_Amount', 'Quarter_Revenue' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Operating_Profit' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Cashflow' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Earn' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Continue_Earn' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Percent_Operating_Profit' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH),\
                                                                    'Quarter_Percent_Cashflow' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Percent_Earn' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Revenue' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Operating_Profit' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Cashflow' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), \
                                                                    'Quarter_Earn' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Continue_Earn' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Percent_Operating_Profit' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH),'Quarter_Percent_Cashflow' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Percent_Earn' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Full_EPS',\
                                                                    'Last_Revenue' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Operating_Profit' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Cashflow' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Earn' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Continue_Earn' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Percent_Operating_Profit' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Percent_Cashflow' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Percent_Earn' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), \
                                                                    'Last_Revenue' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Operating_Profit' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Cashflow' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Earn' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Continue_Earn' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), \
                                                                    'Last_Percent_Operating_Profit' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Percent_Cashflow' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Percent_Earn' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH),\
                                                                    'Fake_Last_Full_EPS', 'Last_Full_Revenue', 'Last_Full_Operating_Profit', 'Last_Full_Cashflow', 'Last_Full_Earn', 'Real_Last_Full_EPS', 'Quarter_BPS', 'Last_Full_BPS','Quarter_ROA','Last_Full_ROA', 'QUARTER_ROE', 'LAST_FULL_ROE'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def how_report_before(self, analyse_date, read_csv, write_csv,start,end):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(start,end):
            try:
                self.get_symbol_six(ANALY_KOSDAQ, I)
                #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
                ANALYSE_DATE = analyse_date
                ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
                ANALYSE_DATE = ANALYSE_DATE.date()
                START_DATE = ANALYSE_DATE - relativedelta(months = 18)
                self.START_DATE = START_DATE.strftime("%Y-%m-%d")
                ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
                S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
                a_dictionary = {} 
                REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
                REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
                OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            except:
                continue
            #if True:
            try:
                if REPORT_MONTH != 12:
                    #if True:
                    try:
                        QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R, LAST_ASSET_R, LAST_LIABILITY_R = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                        #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                        a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_REVENUE_R
                        a_dictionary['QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_OPERATING_PROFIT_R
                        a_dictionary['QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_CASHFLOW_R
                        a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_EARN_R
                        a_dictionary['QUARTER_CONTINUE_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = CONTINUE_EARN_R
                        a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_REVENUE_R
                        a_dictionary['LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_OPERATING_PROFIT_R
                        a_dictionary['LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_CASHFLOW_R
                        a_dictionary['LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_EARN_R
                        a_dictionary['LAST_CONTINUE_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_CONTINUE_EARN_R
                        a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_ASSET_R
                        a_dictionary['QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = QUARTER_LIABILITY_R
                        a_dictionary['LAST_ASSET' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_ASSET_R
                        a_dictionary['LAST_LIABILITY' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = LAST_LIABILITY_R
                        #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                        a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                    except:
                        pass
                if REPORT_MONTH == 12:
                    #if True:
                    try:
                        QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R, LAST_ASSET_R, LAST_LIABILITY_R = self.get_fs_of_add_last_full(REPORT_YEAR, REPORT_MONTH)
                        #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                        #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                        a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
                        time.sleep(1)
                        QUARTER_REVENUE_ADD_R, QUARTER_OPERATING_PROFIT_ADD_R, QUARTER_CASHFLOW_ADD_R, QUARTER_EARN_ADD_R, CONTINUE_EARN_ADD_R, LAST_EARN_ADD_R, LAST_OPERATING_PROFIT_ADD_R, LAST_CASHFLOW_ADD_R, LAST_REVENUE_ADD_R, LAST_CONTINUE_EARN_ADD_R, QUARTER_ASSET_ADD_R, QUARTER_LIABILITY_ADD_R, LAST_ASSET_ADD_R, LAST_LIABILITY_ADD_R = self.get_fs_of_add_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                        a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_REVENUE_R) - int(QUARTER_REVENUE_ADD_R)
                        a_dictionary['QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_OPERATING_PROFIT_R) - int(QUARTER_OPERATING_PROFIT_ADD_R)
                        a_dictionary['QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_CASHFLOW_R) - int(QUARTER_CASHFLOW_ADD_R)
                        a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_EARN_R) - int(QUARTER_EARN_ADD_R)
                        a_dictionary['QUARTER_CONTINUE_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(CONTINUE_EARN_R) - int(CONTINUE_EARN_ADD_R)
                        a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_REVENUE_R) - int(LAST_REVENUE_ADD_R)
                        a_dictionary['LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_OPERATING_PROFIT_R) - int(LAST_OPERATING_PROFIT_ADD_R)
                        a_dictionary['LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_CASHFLOW_R) - int(LAST_CASHFLOW_ADD_R)
                        a_dictionary['LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_EARN_R) - int(LAST_EARN_ADD_R)
                        a_dictionary['LAST_CONTINUE_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_CONTINUE_EARN_R) - int(LAST_CONTINUE_EARN_ADD_R)
                        a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_ASSET_R) - int(QUARTER_ASSET_ADD_R)
                        a_dictionary['QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = int(QUARTER_LIABILITY_R) - int(QUARTER_LIABILITY_ADD_R)
                        a_dictionary['LAST_ASSET' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_ASSET_R) - int(LAST_ASSET_ADD_R)
                        a_dictionary['LAST_LIABILITY' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = int(LAST_LIABILITY_R) - int(LAST_LIABILITY_ADD_R)
                        time.sleep(1)
                    except:
                        pass

                if BACK_REPORT_MONTH != 12:
                    #if True:
                    try:
                        QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R, LAST_ASSET_R, LAST_LIABILITY_R = self.get_fs_of_time(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                        #print(FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN)
                        a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = QUARTER_REVENUE_R
                        a_dictionary['QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR)+ '_' + str(BACK_REPORT_MONTH)] = QUARTER_OPERATING_PROFIT_R
                        a_dictionary['QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = QUARTER_CASHFLOW_R
                        a_dictionary['QUARTER_EARN' + str(BACK_REPORT_YEAR)+ '_' + str(BACK_REPORT_MONTH)] = QUARTER_EARN_R
                        a_dictionary['QUARTER_CONTINUE_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = CONTINUE_EARN_R
                        a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_REVENUE_R
                        a_dictionary['LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_OPERATING_PROFIT_R
                        a_dictionary['LAST_CASHFLOW' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_CASHFLOW_R
                        a_dictionary['LAST_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_EARN_R
                        a_dictionary['LAST_CONTINUE_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_CONTINUE_EARN_R
                        a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = QUARTER_ASSET_R
                        a_dictionary['QUARTER_LIABILITY' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = QUARTER_LIABILITY_R
                        a_dictionary['LAST_ASSET' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_ASSET_R
                        a_dictionary['LAST_LIABILITY' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = LAST_LIABILITY_R
                        #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                        #a_dictionary['BACK_REPORT_DATE'] = BACK_REPORTREPORT_DT.strftime("%Y-%m-%d")
                        time.sleep(1)
                    except:
                        pass
                if BACK_REPORT_MONTH == 12:
                    #if True:
                    try:
                        QUARTER_REVENUE_R, QUARTER_OPERATING_PROFIT_R, QUARTER_CASHFLOW_R, QUARTER_EARN_R, CONTINUE_EARN_R, LAST_EARN_R, LAST_OPERATING_PROFIT_R, LAST_CASHFLOW_R, LAST_REVENUE_R, LAST_CONTINUE_EARN_R, QUARTER_ASSET_R, QUARTER_LIABILITY_R, LAST_ASSET_R, LAST_LIABILITY_R = self.get_fs_of_add_last_full(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                        BACK_BACK_REPORT_YEAR, BACK_BACK_REPORT_MONTH = self.get_report_year_month_one_back(BACK_REPORT_YEAR, BACK_REPORT_MONTH)
                        time.sleep(0.1)
                        QUARTER_REVENUE_ADD_R, QUARTER_OPERATING_PROFIT_ADD_R, QUARTER_CASHFLOW_ADD_R, QUARTER_EARN_ADD_R, CONTINUE_EARN_ADD_R, LAST_EARN_ADD_R, LAST_OPERATING_PROFIT_ADD_R, LAST_CASHFLOW_ADD_R, LAST_REVENUE_ADD_R, LAST_CONTINUE_EARN_ADD_R, QUARTER_ASSET_ADD_R, QUARTER_LIABILITY_ADD_R, LAST_ASSET_ADD_R, LAST_LIABILITY_ADD_R = self.get_fs_of_add_time(BACK_BACK_REPORT_YEAR, BACK_BACK_REPORT_MONTH)
                        a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_REVENUE_R) - int(QUARTER_REVENUE_ADD_R)
                        a_dictionary['QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_OPERATING_PROFIT_R) - int(QUARTER_OPERATING_PROFIT_ADD_R)
                        a_dictionary['QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_CASHFLOW_R) - int(QUARTER_CASHFLOW_ADD_R)
                        a_dictionary['QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_EARN_R) - int(QUARTER_EARN_ADD_R)
                        a_dictionary['QUARTER_CONTINUE_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(CONTINUE_EARN_R) - int(CONTINUE_EARN_ADD_R)
                        a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_REVENUE_R) - int(LAST_REVENUE_ADD_R)
                        a_dictionary['LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_OPERATING_PROFIT_R) - int(LAST_OPERATING_PROFIT_ADD_R)
                        a_dictionary['LAST_CASHFLOW' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_CASHFLOW_R) - int(LAST_CASHFLOW_ADD_R)
                        a_dictionary['LAST_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_EARN_R) - int(LAST_EARN_ADD_R)
                        a_dictionary['LAST_CONTINUE_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_CONTINUE_EARN_R) - int(LAST_CONTINUE_EARN_ADD_R)
                        a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_ASSET_R) - int(QUARTER_ASSET_ADD_R)
                        a_dictionary['QUARTER_LIABILITY' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = int(QUARTER_LIABILITY_R) - int(QUARTER_LIABILITY_ADD_R)
                        a_dictionary['LAST_ASSET' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_ASSET_R) - int(LAST_ASSET_ADD_R)
                        a_dictionary['LAST_LIABILITY' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)] = int(LAST_LIABILITY_R) - int(LAST_LIABILITY_ADD_R)
                        #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                        time.sleep(1)
                    except:
                        pass
            except:
                pass
                
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            try:
                LAST_FULL_EARN_R, LAST_FULL_OPERATING_PROFIT_R, LAST_FULL_CASHFLOW_R, LAST_FULL_REVENUE_R, LAST_FULL_CONTINUE_EARN_R, LAST_FULL_ASSET_R, LAST_FULL_LIABILITY_R = self.get_fs_of_last_full(OLD_REPORT_YEAR, 12)
                a_dictionary['LAST_FULL_REVENUE'] = LAST_FULL_REVENUE_R
                a_dictionary['LAST_FULL_OPERATING_PROFIT'] = LAST_FULL_OPERATING_PROFIT_R
                a_dictionary['LAST_FULL_CASHFLOW'] = LAST_FULL_CASHFLOW_R
                a_dictionary['LAST_FULL_EARN'] = LAST_FULL_EARN_R
                a_dictionary['LAST_FULL_CONTINUE_EARN'] = LAST_FULL_CONTINUE_EARN_R
                a_dictionary['LAST_FULL_ASSET'] = LAST_FULL_ASSET_R
                a_dictionary['LAST_FULL_LIABILITY'] = LAST_FULL_LIABILITY_R
                time.sleep(1)
            except:
                pass
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
                time.sleep(1)
            except:
                pass
           
            #if True:
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        if 'QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                            a_dictionary['QUARTER_EPS'] = int(int(a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_EARN' + str(REPORT_YEAR - 1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        if 'LAST_EARN' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():                                                                                                           
                            a_dictionary['LAST_EPS'] = int(int(a_dictionary['LAST_EARN' + str(REPORT_YEAR - 1) + '_' + str(REPORT_MONTH)])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['REAL_LAST_FULL_EPS'] = int(int(a_dictionary['LAST_FULL_EARN'])/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            
            try:
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        if 'QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                            a_dictionary['QUARTER_BPS'] = int((int(a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)]) - (int(a_dictionary['QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_LIABILITY' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])))/int(a_dictionary['STOCK_AMOUNT']))
                if 'STOCK_AMOUNT' in a_dictionary.keys():
                    if 'LAST_FULL_ASSET' in a_dictionary.keys():
                        if 'LAST_FULL_LIABILITY'in a_dictionary.keys():
                            a_dictionary['LAST_FULL_BPS'] = int((int(a_dictionary['LAST_FULL_ASSET'])-int(a_dictionary['LAST_FULL_LIABILITY']))/int(a_dictionary['STOCK_AMOUNT']))
            except:
                pass
            try:
                if 'QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_EPS' in a_dictionary.keys():
                        if 'STOCK_AMOUNT' in a_dictionary.keys():
                            QUARTER_FULL_EARN = int(self.get_predict_earn_per_stock(REPORT_MONTH, int(a_dictionary['QUARTER_EPS']) * int(a_dictionary['STOCK_AMOUNT'])))
                            a_dictionary['QUARTER_ROA'] = round(QUARTER_FULL_EARN / (int(a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)]))*100, 2)
                if 'LAST_FULL_ASSET' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_ROA'] = round(int(a_dictionary['LAST_FULL_EARN']) / int(a_dictionary['LAST_FULL_ASSET'])*100, 2)
            except:
                pass
            try:
                if 'QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_EPS' in a_dictionary.keys():
                        if 'STOCK_AMOUNT' in a_dictionary.keys():
                            QUARTER_FULL_EARN = int(self.get_predict_earn_per_stock(REPORT_MONTH, int(a_dictionary['QUARTER_EPS']) * int(a_dictionary['STOCK_AMOUNT'])))
                            a_dictionary['QUARTER_ROE'] = round(QUARTER_FULL_EARN / (int(a_dictionary['QUARTER_ASSET' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_ASSET' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)]) - (int(a_dictionary['QUARTER_LIABILITY' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)]) + int(a_dictionary['QUARTER_LIABILITY' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])))*100, 2)
                if 'LAST_FULL_ASSET' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_ROE'] = round(int(a_dictionary['LAST_FULL_EARN']) / (int(a_dictionary['LAST_FULL_ASSET']) - int(a_dictionary['LAST_FULL_LIABILITY']))*100, 2)
            except:
                pass
            try:
                if 'QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_OPERATING_PROFIT' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_FULL_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_FULL_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_FULL_PERCENT_OPERATING_PROFIT'] = round(int(a_dictionary['LAST_FULL_OPERATING_PROFIT'])/int(a_dictionary['LAST_FULL_REVENUE'])*100, 2)
                if 'QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_EARN' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_EARN' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['LAST_EARN' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_FULL_REVENUE' in a_dictionary.keys():
                    if 'LAST_FULL_EARN' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_PERCENT_EARN'] = round(int(a_dictionary['LAST_FULL_EARN'])/int(a_dictionary['LAST_FULL_REVENUE'])*100, 2)
                if 'QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['QUARTER_PERCENT_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)] = round(int(a_dictionary['LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)])*100, 2)
                if 'LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                    if 'LAST_CASHFLOW' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH) in a_dictionary.keys():
                        a_dictionary['LAST_PERCENT_CASHFLOW' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)] = round(int(a_dictionary['LAST_CASHFLOW' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])/int(a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR - 1) + '_' + str(BACK_REPORT_MONTH)])*100, 2)
                if 'LAST_FULL_REVENUE' in a_dictionary.keys():
                    if 'LAST_FULL_CASHFLOW' in a_dictionary.keys():
                        a_dictionary['LAST_FULL_PERCENT_CASHFLOW'] = round(int(a_dictionary['LAST_FULL_CASHFLOW'])/int(a_dictionary['LAST_FULL_REVENUE'])*100, 2)
            except:
                pass
            try:
                if 'QUARTER_EPS' in a_dictionary.keys():
                    a_dictionary['QUARTER_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EPS']))
                '''if 'QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])'''
            except:
                pass
            try:
                if 'LAST_EPS' in a_dictionary.keys():
                    a_dictionary['FAKE_LAST_FULL_EPS'] = int(self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['LAST_EPS']))   
                '''if 'OLD_QUARTER_EARN' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_EARN'])
                if 'OLD_QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_OPERATING_PROFIT'])
                if 'OLD_QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['OLD_QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['OLD_QUARTER_REVENUE'])'''
            except:
                pass
            #if True:
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_CONTINUE_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)],\
                                            a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_PERCENT_CASHFLOW' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_PERCENT_EARN' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH)], a_dictionary['QUARTER_REVENUE' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], \
                                           a_dictionary['QUARTER_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_CONTINUE_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_PERCENT_OPERATING_PROFIT' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_PERCENT_CASHFLOW' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], \
                                           a_dictionary['QUARTER_PERCENT_EARN' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['QUARTER_FULL_EPS'], a_dictionary['LAST_REVENUE' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_CONTINUE_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], \
                                           a_dictionary['LAST_PERCENT_OPERATING_PROFIT' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_PERCENT_CASHFLOW' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_PERCENT_EARN' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH)], a_dictionary['LAST_REVENUE' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_OPERATING_PROFIT' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_CASHFLOW' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], \
                                           a_dictionary['LAST_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_CONTINUE_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_PERCENT_OPERATING_PROFIT' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_PERCENT_CASHFLOW' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['LAST_PERCENT_EARN' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH)], a_dictionary['FAKE_LAST_FULL_EPS'], \
                                           a_dictionary['LAST_FULL_REVENUE'], a_dictionary['LAST_FULL_OPERATING_PROFIT'], a_dictionary['LAST_FULL_CASHFLOW'], a_dictionary['LAST_FULL_EARN'], a_dictionary['REAL_LAST_FULL_EPS'], a_dictionary['QUARTER_BPS'],a_dictionary['LAST_FULL_BPS'], a_dictionary['QUARTER_ROA'], a_dictionary['LAST_FULL_ROA'], a_dictionary['QUARTER_ROE'], a_dictionary['LAST_FULL_ROE']])
                print(self.OLD_SYMBOL)
                #print(self.BOOM_PER_LIST)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date', 'Quarter_Revenue' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Operating_Profit' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Cashflow' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Earn' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Continue_Earn' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Percent_Operating_Profit' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH),\
                                                                    'Quarter_Percent_Cashflow' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Percent_Earn' + str(REPORT_YEAR) + '_' + str(REPORT_MONTH), 'Quarter_Revenue' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Operating_Profit' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Cashflow' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), \
                                                                    'Quarter_Earn' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Continue_Earn' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Percent_Operating_Profit' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH),'Quarter_Percent_Cashflow' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Percent_Earn' + str(BACK_REPORT_YEAR) + '_' + str(BACK_REPORT_MONTH), 'Quarter_Full_EPS',\
                                                                    'Last_Revenue' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Operating_Profit' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Cashflow' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Earn' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Continue_Earn' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Percent_Operating_Profit' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Percent_Cashflow' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), 'Last_Percent_Earn' + str(REPORT_YEAR-1) + '_' + str(REPORT_MONTH), \
                                                                    'Last_Revenue' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Operating_Profit' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Cashflow' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Earn' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Continue_Earn' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), \
                                                                    'Last_Percent_Operating_Profit' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Percent_Cashflow' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH), 'Last_Percent_Earn' + str(BACK_REPORT_YEAR-1) + '_' + str(BACK_REPORT_MONTH),\
                                                                    'Fake_Last_Full_EPS', 'Last_Full_Revenue', 'Last_Full_Operating_Profit', 'Last_Full_Cashflow', 'Last_Full_Earn', 'Real_Last_Full_EPS', 'Quarter_BPS', 'Last_Full_BPS','Quarter_ROA','Last_Full_ROA', 'QUARTER_ROE', 'LAST_FULL_ROE'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
    
    def get_per(self, read_csv, write_csv, index_name):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            STOCK_AMOUNT = ANALY_KOSDAQ.loc[I, 'Stock_Amount']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            RECENT_MARKET_GUN = int(int(STOCK_AMOUNT)*int(RECENT_STOCK_PRICE))
            LAST_REPORT_DATE = REPORT_DATE - relativedelta(months = 12)
            LAST_TOP_STOCK_PRICE, LAST_TOP_STOCK_PRICE_DATE = self.get_top_last_year(LAST_REPORT_DATE, REPORT_DATE)
            OLD_REPORT_DATE = REPORT_DATE - relativedelta(years = 3)
            OLD_TOP_STOCK_PRICE, OLD_TOP_STOCK_PRICE_DATE = self.get_top_last_year(OLD_REPORT_DATE, REPORT_DATE)
            
            
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Full_EPS'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = round(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK,2)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            FAKE_LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Fake_Last_Full_EPS'])
            if FAKE_LAST_EARN_PER_STOCK != 0:
                FAKE_LAST_PER = round(LAST_TOP_STOCK_PRICE/FAKE_LAST_EARN_PER_STOCK,2)
            if FAKE_LAST_EARN_PER_STOCK == 0:
                FAKE_LAST_PER = 0
            #if 'LAST_STOCK_PRICE' in locals():
            REAL_LAST_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Real_Last_Full_EPS'])
            if REAL_LAST_EARN_PER_STOCK != 0:
                REAL_LAST_PER = round(LAST_TOP_STOCK_PRICE/REAL_LAST_EARN_PER_STOCK,2)
            if REAL_LAST_EARN_PER_STOCK == 0:
                REAL_LAST_PER = 0
            #try:
            PRICE_PERCENT_CHANGE_THREE_YEAR = self.get_top_bottom_percent_change(OLD_REPORT_DATE, REPORT_DATE)
            PRICE_PERCENT_COMPARE_OLD_BIGGEST_NOW = round((OLD_TOP_STOCK_PRICE-RECENT_STOCK_PRICE)/RECENT_STOCK_PRICE*100,2)
            PRICE_PERCENT_CHANGE_YEAR = self.get_top_bottom_percent_change(LAST_REPORT_DATE, REPORT_DATE)
            PRICE_PERCENT_COMPARE_LAST_BIGGEST_NOW = round((LAST_TOP_STOCK_PRICE-RECENT_STOCK_PRICE)/RECENT_STOCK_PRICE*100,2)
            PRICE_PERCENT_CHANGE = self.check_report_month_price_data(REPORT_DATE)
            
            LAST_TOP_STOCK_PRICE_DATE = LAST_TOP_STOCK_PRICE_DATE.strftime("%Y-%m-%d")
            OLD_TOP_STOCK_PRICE_DATE = OLD_TOP_STOCK_PRICE_DATE.strftime("%Y-%m-%d")
            #except:
                #pass
            self.SYMBOL = index_name
            #INDEX_PERCENT_CHANGE_YEAR = self.get_top_bottom_percent_change(LAST_REPORT_DATE, REPORT_DATE)
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_MARKET_GUN), str(RECENT_PER), str(LAST_TOP_STOCK_PRICE), str(FAKE_LAST_PER), str(REAL_LAST_PER), str(PRICE_PERCENT_CHANGE), str(PRICE_PERCENT_CHANGE_THREE_YEAR), str(PRICE_PERCENT_COMPARE_OLD_BIGGEST_NOW), str(OLD_TOP_STOCK_PRICE_DATE), str(PRICE_PERCENT_CHANGE_YEAR), str(PRICE_PERCENT_COMPARE_LAST_BIGGEST_NOW), str(LAST_TOP_STOCK_PRICE_DATE)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Top_Stock_Price', 'Recent_Market_Gun', 'Quarter_Per', 'Last_Top_Stock_Price', 'Fake_Last_Per', 'Real_Last_Per', 'Price_Percent_Change', 'Price_Percent_Change_Three_Year', 'Price_Percent_Compare_Old_Biggest_Now', 'Old_Top_Stock_Price_Date', 'Price_Percent_Change_Year', 'Price_Percent_Compare_Last_Bigget_Now', 'Last_Top_Stock_Price_Date'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.drop_duplicates(subset = ['Symbol'], inplace = True)
        AN_KOSDAQ.sort_values(by=['Sector'])
        AN_KOSDAQ.to_csv(write_csv)
        
    def how_report_sell(self, analyse_date, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            #NEW_BOOM_DATE = ANALY_KOSDAQ.loc[I, 'Boom_Date']
            ANALYSE_DATE = analyse_date
            ANALYSE_DATE = dt.datetime.strptime(ANALYSE_DATE, "%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.date()
            START_DATE = ANALYSE_DATE - relativedelta(months = 18)
            self.START_DATE = START_DATE.strftime("%Y-%m-%d")
            ANALYSE_DATE = ANALYSE_DATE.strftime("%Y-%m-%d")
            S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=ANALYSE_DATE, kind = 'A', final = False)
            a_dictionary = {} 
            REPORT_NM, REPORT_DT = self.get_report_date(S_LIST)
            REPORT_YEAR, REPORT_MONTH = self.get_report_year_month(REPORT_NM)
            OLD_REPORT_YEAR, OLD_REPORT_MONTH = self.get_before_report_time(REPORT_YEAR, REPORT_MONTH)
            try:
                FULL_QUARTER_REVENUE, FULL_QUARTER_OPERATING_PROFIT, BOOM_QUARTER_EARN = self.get_fs_of_time(REPORT_YEAR, REPORT_MONTH)
                a_dictionary['QUARTER_REVENUE'] = FULL_QUARTER_REVENUE
                a_dictionary['QUARTER_OPERATING_PROFIT'] = FULL_QUARTER_OPERATING_PROFIT
                a_dictionary['QUARTER_EARN'] = BOOM_QUARTER_EARN
                #a_dictionary['QUARTER_EARN_PER_STOCK'] = BOOM_QUARTER_EARN_PER_STOCK
                a_dictionary['REPORT_DATE'] = REPORT_DT.strftime("%Y-%m-%d")
                time.sleep(1)
            except:
                pass
            #GET RECETNT_REPORT AND GET EARN AND PER_STORCK INORDER TO CONVERT PER_STOCK
            TODAY = self.TODAY.strftime("%Y-%m-%d")
            R_S_LIST = self.dart.list(self.SYMBOL, start=self.START_DATE, end=TODAY, kind = 'A', final = False)
            RECENT_REPORT_NM, RECENT_REPORT_DT = self.get_report_date(R_S_LIST)
            RECENT_REPORT_YEAR, RECENT_REPORT_MONTH = self.get_report_year_month(RECENT_REPORT_NM)
            #if True:
            #get stock amount
            try:
                STOCK_AMOUNT = self.get_stock_amount(RECENT_REPORT_YEAR, RECENT_REPORT_MONTH)
                a_dictionary['STOCK_AMOUNT'] = STOCK_AMOUNT
                print(STOCK_AMOUNT)
            except:
                pass
            #if True:
                
                try:
                    if 'STOCK_AMOUNT' in a_dictionary.keys():
                        if 'QUARTER_EARN' in a_dictionary.keys():
                            a_dictionary['QUARTER_EARN_PER_STOCK'] = int(a_dictionary['QUARTER_EARN']/a_dictionary['STOCK_AMOUNT'])
                except:
                    pass
            #try:
                if 'QUARTER_EARN_PER_STOCK' in a_dictionary.keys():
                    if 'QUARTER_EARN' in a_dictionary.keys():
                        a_dictionary['QUARTER_EARN_PER_STOCK'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN_PER_STOCK'])
                        a_dictionary['QUARTER_EARN'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_EARN'])
                    #a_dictionary['PER'] = int(STOCK_PRICE/a_dictionary['QUARTER_EARN_PER_STOCK'])
            #except:
                #a_dictionary['PER'] = 0
                if 'QUARTER_REVENUE' in a_dictionary.keys():
                    a_dictionary['QUARTER_REVENUE'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_REVENUE'])
                    
                if 'QUARTER_OPERATING_PROFIT' in a_dictionary.keys():
                    a_dictionary['QUARTER_OPERATING_PROFIT'] = self.get_predict_earn_per_stock(REPORT_MONTH, a_dictionary['QUARTER_OPERATING_PROFIT'])
                #pass
            try:
                self.BOOM_PER_LIST.append([str(self.OLD_SYMBOL), a_dictionary['REPORT_DATE'], a_dictionary['QUARTER_REVENUE'], a_dictionary['QUARTER_OPERATING_PROFIT'], a_dictionary['QUARTER_EARN'], a_dictionary['QUARTER_EARN_PER_STOCK']])
                print(self.OLD_SYMBOL)
                time.sleep(1)
            except:
                continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(self.BOOM_PER_LIST), columns=['Symbol', 'Report_Date_Sell', 'Quarter_Revenue_Sell', 'Quarter_Operating_Profit_Sell', 'Quarter_Earn_Sell', 'Quarter_Earn_Per_Stock_Sell'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def get_per_sell(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        PER_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I,'Report_Date_Sell']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            #if 'RECENT_STOCK_PRICE' in locals():
            RECENT_EARN_PER_STOCK = int(ANALY_KOSDAQ.loc[I, 'Quarter_Earn_Per_Stock'])
            if RECENT_EARN_PER_STOCK != 0:
                RECENT_PER = int(RECENT_STOCK_PRICE/RECENT_EARN_PER_STOCK)
            if RECENT_EARN_PER_STOCK == 0:
                RECENT_PER = 0
            #try:
            PER_LIST.append([str(self.OLD_SYMBOL), str(RECENT_STOCK_PRICE), str(RECENT_PER)])
            #print(self.OLD_SYMBOL)
            #except:
                #continue
        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(PER_LIST), columns=['Symbol', 'Quarter_Stock_Price_Sell', 'Quarter_Per_Sell'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        #left['Quarter_Earn'] = left['Quarter_Earn'].astype(str)
        #left['Quarter_Earn_Per_Stock'] = left['Quarter_Earn_Per_Stock'].astype(str)
        #left['Old_Earn'] = left['Old_Earn'].astype(str)
        #left['Old_Earn_Per_Stock'] = left['Old_Earn_Per_Stock'].astype(str)
        AN_KOSDAQ = left.merge(right, on='Symbol', how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def erase_nan(self, read_csv, write_csv):
        df = pd.read_csv(read_csv)
        df.dropna(subset = ['Report_Date'], inplace = True)
        df.reset_index(drop=True, inplace = True)
        df.drop(['Unnamed: 0'], axis = 1, inplace = True)
        df.to_csv(write_csv)
    
    def erase_nan_before(self, read_csv, write_csv):
        df = pd.read_csv(read_csv)
        df.dropna(subset = ['Report_Date'], inplace = True)
        df.reset_index(drop=True, inplace = True)
        df.drop(['Unnamed: 0'], axis = 1, inplace = True)
        df.to_csv(write_csv)
    
    def find_suitable_stock(self, read_csv, write_csv):
        kosdaq_data = pd.read_csv(read_csv)
        REPORT_YEAR, REPORT_MONTH = self.get_report_year_month_new(read_csv)
        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
        kosdaq_data['ListingDate'] = pd.to_datetime(kosdaq_data['ListingDate'], format='%Y-%m-%d')
        '''kosdaq_data['Quarter_Percent_Operating_Profit'] = kosdaq_data['Quarter_Percent_Operating_Profit'].astype(float)'''
        kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
        '''kosdaq_data['Quarter_Cost_Of_Sale'] = kosdaq_data['Quarter_Revenue'] - kosdaq_data['Quarter_Operating_Profit']
        kosdaq_data['Last_Cost_Of_Sale'] = kosdaq_data['Last_REVENUE'] - kosdaq_data['Last_Operating_Profit']
        kosdaq_data['Quarter_Loss'] = (kosdaq_data['Quarter_Cost_Of_Sale']/kosdaq_data['Quarter_Revenue'])*100
        kosdaq_data['Last_Loss'] = (kosdaq_data['Last_Cost_Of_Sale']/kosdaq_data['Last_REVENUE'])*100'''
        
        
        
        kosdaq_data1 = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Revenue'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*1.1)]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data1['Last_Revenue'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*1.1)]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data1['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)] + 4)]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data1['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)] + 4)]
        '''kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*1.5)]
        kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data['Last_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*1.5)]'''
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -10]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -10]
        '''kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]]
        kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]]'''
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_6'] > -10000000000]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_3'] > -10000000000]
        
        kosdaq_data2 = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Revenue'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*1.1)]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data2['Last_Revenue'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*1.1)]
        '''kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)] + 1.1)]
        kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)] + 1.1)]'''
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data2['Last_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*1.5)]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data2['Last_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*1.5)]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)] > 0]
        '''kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]]
        kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]]'''
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_6'] > -10000000000]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_3'] > -10000000000]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Last_Percent_Earn'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Last_Percent_Earn'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)] > 0]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'] > -50]
        
        kosdaq_data3 = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Revenue'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*0.7)]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data3['Last_Revenue'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*0.7)]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data3['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)] + 4)]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data3['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)] + 4)]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*1.1)]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data['Last_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*1.1)]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -10]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -10]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_6'] > -10000000000]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_3'] > -10000000000]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -10]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -10]
        
        
        kosdaq_data4 = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Revenue'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*1.5)]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data4['Last_Revenue'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*1.5)]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)] + 1.1)]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)] + 1.1)]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data4['Last_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*3.0)]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data4['Last_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*3.0)]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > 0]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -20]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -20]
        
        #kosdaq_data1, kosdaq_data2,kosdaq_data3
        frames = [kosdaq_data4]
        kosdaq_data = pd.concat(frames)
        kosdaq_data.reset_index(drop=True, inplace = True)
        kosdaq_data['Operating_Profit_Change_Now'] = kosdaq_data['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] - kosdaq_data['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]
        kosdaq_data['Operating_Profit_Change_Before'] = kosdaq_data['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] - kosdaq_data['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]
        kosdaq_data['Revenue_Change_Now'] = (kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] - kosdaq_data['Last_Revenue'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)])/(kosdaq_data['Last_Revenue'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)])*100
        kosdaq_data['Revenue_Change_Before'] = (kosdaq_data['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] - kosdaq_data1['Last_Revenue'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)])/(kosdaq_data1['Last_Revenue'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)])*100
        kosdaq_data.to_csv(write_csv)
    
    def find_suitable_stock_before(self, read_csv, write_csv):
        kosdaq_data = pd.read_csv(read_csv)
        REPORT_YEAR, REPORT_MONTH = self.get_report_year_month_new(read_csv)
        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
        kosdaq_data['ListingDate'] = pd.to_datetime(kosdaq_data['ListingDate'], format='%Y-%m-%d')
        '''kosdaq_data['Quarter_Percent_Operating_Profit'] = kosdaq_data['Quarter_Percent_Operating_Profit'].astype(float)'''
        kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
        '''kosdaq_data['Quarter_Cost_Of_Sale'] = kosdaq_data['Quarter_Revenue'] - kosdaq_data['Quarter_Operating_Profit']
        kosdaq_data['Last_Cost_Of_Sale'] = kosdaq_data['Last_REVENUE'] - kosdaq_data['Last_Operating_Profit']
        kosdaq_data['Quarter_Loss'] = (kosdaq_data['Quarter_Cost_Of_Sale']/kosdaq_data['Quarter_Revenue'])*100
        kosdaq_data['Last_Loss'] = (kosdaq_data['Last_Cost_Of_Sale']/kosdaq_data['Last_REVENUE'])*100'''
        kosdaq_data1 = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Revenue'+str(REPORT_YEAR-2)+'_'+str(REPORT_MONTH)]*0.5)]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data1['Last_Revenue'+str(BACK_REPORT_YEAR-2)+'_'+str(BACK_REPORT_MONTH)]*0.5)]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data1['Last_Percent_Operating_Profit'+str(REPORT_YEAR-2)+'_'+str(REPORT_MONTH)] + 4)]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data1['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-2)+'_'+str(BACK_REPORT_MONTH)] + 4)]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*1.5)]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data['Last_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*1.5)]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -10]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -10]
        '''kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]]
        kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]]'''
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_6'] > -10000000000]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_3'] > -10000000000]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -10]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -10]
        
        '''kosdaq_data2 = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Revenue'+str(REPORT_YEAR-2)+'_'+str(REPORT_MONTH)]*1.2)]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data2['Last_Revenue'+str(BACK_REPORT_YEAR-2)+'_'+str(BACK_REPORT_MONTH)]*1.1)]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data2['Last_Operating_Profit'+str(REPORT_YEAR-2)+'_'+str(REPORT_MONTH)]*1.2)]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data2['Last_Operating_Profit'+str(BACK_REPORT_YEAR-2)+'_'+str(BACK_REPORT_MONTH)]*1.2)]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Last_Percent_Operating_Profit'+str(REPORT_YEAR-2)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-2)+'_'+str(BACK_REPORT_MONTH)] > 0]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_6'] > -10000000000]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_3'] > -10000000000]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Last_Percent_Earn'+str(REPORT_YEAR-2)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data2 = kosdaq_data2[kosdaq_data2['Last_Percent_Earn'+str(BACK_REPORT_YEAR-2)+'_'+str(BACK_REPORT_MONTH)] > 0]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'] > -50]
        
        kosdaq_data3 = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Revenue'+str(REPORT_YEAR-2)+'_'+str(REPORT_MONTH)]*0.7)]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data3['Last_Revenue'+str(BACK_REPORT_YEAR-2)+'_'+str(BACK_REPORT_MONTH)]*0.7)]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data3['Last_Percent_Operating_Profit'+str(REPORT_YEAR-2)+'_'+str(REPORT_MONTH)] + 4)]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data3['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-2)+'_'+str(BACK_REPORT_MONTH)] + 4)]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -10]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -10]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_6'] > -10000000000]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_3'] > -10000000000]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -10]
        kosdaq_data3 = kosdaq_data3[kosdaq_data3['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -10]
        
        
        kosdaq_data4 = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Revenue'+str(REPORT_YEAR-2)+'_'+str(REPORT_MONTH)]*0.8)]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data4['Last_Revenue'+str(BACK_REPORT_YEAR-2)+'_'+str(BACK_REPORT_MONTH)]*0.8)]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)] + 1.1)]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)] + 1.1)]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data4['Last_Operating_Profit'+str(REPORT_YEAR-2)+'_'+str(REPORT_MONTH)]*1.2)]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data4['Last_Operating_Profit'+str(BACK_REPORT_YEAR-2)+'_'+str(BACK_REPORT_MONTH)]*1.2)]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > 0]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -20]
        kosdaq_data4 = kosdaq_data4[kosdaq_data4['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -20]'''
        
        frames = [kosdaq_data]
        result = pd.concat(frames)
        result.drop_duplicates(subset = ['Symbol'], inplace = True)
        result.reset_index(drop=True, inplace = True)
        result.to_csv(write_csv)
    
    def find_suitable_stock_later(self, read_csv, write_csv):
        kosdaq_data = pd.read_csv(read_csv)
        REPORT_YEAR, REPORT_MONTH = self.get_report_year_month_new(read_csv)
        BACK_REPORT_YEAR, BACK_REPORT_MONTH = self.get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
        kosdaq_data['ListingDate'] = pd.to_datetime(kosdaq_data['ListingDate'], format='%Y-%m-%d')
        '''kosdaq_data['Quarter_Percent_Operating_Profit'] = kosdaq_data['Quarter_Percent_Operating_Profit'].astype(float)'''
        kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
        '''kosdaq_data['Quarter_Cost_Of_Sale'] = kosdaq_data['Quarter_Revenue'] - kosdaq_data['Quarter_Operating_Profit']
        kosdaq_data['Last_Cost_Of_Sale'] = kosdaq_data['Last_REVENUE'] - kosdaq_data['Last_Operating_Profit']
        kosdaq_data['Quarter_Loss'] = (kosdaq_data['Quarter_Cost_Of_Sale']/kosdaq_data['Quarter_Revenue'])*100
        kosdaq_data['Last_Loss'] = (kosdaq_data['Last_Cost_Of_Sale']/kosdaq_data['Last_REVENUE'])*100'''
        #kosdaq_data1 = kosdaq_data[kosdaq_data['Price_Percent_Change_Year'] > kosdaq_data['Price_Percent_Compare_Last_Bigget_Now']*1.5]
        #kosdaq_data1 = kosdaq_data1[kosdaq_data1['Price_Percent_Change_Year'] < 300]
        #kosdaq_data1 = kosdaq_data1[kosdaq_data1['Price_Percent_Change_Three_Year'] < 500]
        #kosdaq_data = kosdaq_data[kosdaq_data['Price_Percent_Change_Two_Year'] > kosdaq_data['Price_Percent_Compare_Old_Bigget_Now']*1.5]
        kosdaq_data1 = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > 0]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Recent_Market_Gun'] < 1000000000000]
        kosdaq_data1 = kosdaq_data1[kosdaq_data1['Recent_Market_Gun'] > 100000000000]
        #kosdaq_data1 = kosdaq_data1[kosdaq_data1['Real_Last_Per'] < 30]
        #kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > 5]
        #kosdaq_data2 = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > kosdaq_data['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)]*1.2]
        #kosdaq_data2 = kosdaq_data2[kosdaq_data2['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > 5]
        #kosdaq_data2 = kosdaq_data2[kosdaq_data2['Real_Last_Per'] < 30]
        
        frames = [kosdaq_data1]
        result = pd.concat(frames)
        result.drop_duplicates(subset = ['Symbol'], inplace = True)
        result.reset_index(drop=True, inplace = True)
        result.to_csv(write_csv)
        
    def how_much_earn(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        HOW_MUCH_EARN_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            TODAY = dt.date.today()
            TOP_STOCK_PRICE, TOP_STOCK_PRICE_DATE = self.get_top_last_year(REPORT_DATE, TODAY)
            HOW_MUCH_EARN = round((TOP_STOCK_PRICE-RECENT_STOCK_PRICE)/RECENT_STOCK_PRICE*100, 2)
            #try:
            HOW_MUCH_EARN_LIST.append([str(self.OLD_SYMBOL),TOP_STOCK_PRICE_DATE, HOW_MUCH_EARN])
            print(self.OLD_SYMBOL)
            #except:
                #continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(HOW_MUCH_EARN_LIST), columns=['Symbol', 'Top_Stock_Price_Date', 'How_Much_Earn'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def how_much_earn(self, read_csv, write_csv):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        HOW_MUCH_EARN_LIST = []
        for I in range(ANALY_KOSDAQ.index.size):
            self.get_symbol_six(ANALY_KOSDAQ, I)
            REPORT_DATE = ANALY_KOSDAQ.loc[I, 'Report_Date']
            if isinstance(REPORT_DATE, float):
                continue
            REPORT_DATE = dt.datetime.strptime(REPORT_DATE, "%Y-%m-%d")
            REPORT_DATE = REPORT_DATE.date()
            RECENT_STOCK_PRICE = self.check_price_data(REPORT_DATE)
            TODAY = dt.date.today()
            TOP_STOCK_PRICE, TOP_STOCK_PRICE_DATE = self.get_top_last_year(REPORT_DATE, TODAY)
            HOW_MUCH_EARN = round((TOP_STOCK_PRICE-RECENT_STOCK_PRICE)/RECENT_STOCK_PRICE*100, 2)
            #try:
            HOW_MUCH_EARN_LIST.append([str(self.OLD_SYMBOL),TOP_STOCK_PRICE_DATE, HOW_MUCH_EARN])
            print(self.OLD_SYMBOL)
            #except:
                #continue


        left = pd.read_csv(read_csv)
        right = pd.DataFrame(np.array(HOW_MUCH_EARN_LIST), columns=['Symbol', 'Top_Stock_Price_Date', 'How_Much_Earn'])
        left['Symbol'] = left['Symbol'].astype(str)
        right['Symbol'] = right['Symbol'].astype(str)
        AN_KOSDAQ = pd.merge(left, right, on=['Symbol'], how='left')
        AN_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        AN_KOSDAQ.to_csv(write_csv)
        
    def sort_values_ascendings(self, read_csv, write_csv, values):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        ascendings = []
        for i in values:
            ascendings.append(False)
            
        ANALY_KOSDAQ = ANALY_KOSDAQ.sort_values(by=values, ascending=ascendings)
        ANALY_KOSDAQ.drop_duplicates(subset = ['Symbol'], inplace = True)
        ANALY_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        ANALY_KOSDAQ.to_csv(write_csv)
        
        
    def sort_values_descendings(self, read_csv, write_csv, values):
        ANALY_KOSDAQ = pd.read_csv(read_csv)
        descendings = []
        for i in values:
            descendings.append(True)
            
        ANALY_KOSDAQ = ANALY_KOSDAQ.sort_values(by=values, ascending=descendings)
        ANALY_KOSDAQ.drop_duplicates(subset = ['Symbol'], inplace = True)
        ANALY_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
        ANALY_KOSDAQ.to_csv(write_csv)
        
        
            
        
                    
        
#have 2 keys 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788', '6048f2c5a065e754cee104b8c0edd8186f6b3d61', '94331def3e94db8a1aabb84e4257cec0ad3b576b', 'b1fe4037e43a6db43d8ac280d262a05e90bf9171', 'cb5c4fff01f2b92088c9679b935676210916c648', 'cb5c4fff01f2b92088c9679b935676210916c648'
#index_name = kospi('KS11'), kosdaq('KQ11')
'''akaps = how_per('6048f2c5a065e754cee104b8c0edd8186f6b3d61')
#akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(7).csv')
akaps.how_report('2021-08-30','KOSDAQ.csv', 'A_PER_KOSDAQ(2021_06).csv',1,1270)'''
'''paka = how_per('84acf17c852e9430843a589191330e0630cc19a7')
paka.how_report('2021-11-30','KOSPI(1).csv', 'A_PER_KOSPI(2021_6).csv',1,770)'''
def get_report_year_month(report_nm):
    delimiters =  "_"
    regexPattern = '|'.join(map(re.escape, delimiters))
    nm_list = re.split(regexPattern, report_nm)
    nm_num_list = [int(s) for s in nm_list if s.isdigit()]
    report_year = int(nm_num_list[0])
    report_month = int(nm_num_list[1])
    return report_year, report_month

def get_report_day(report_year, report_month):
    if report_month == 3:
        report_day = '05-30'
        return str(report_year)+'-'+report_day
    if report_month == 6:
        report_day = '08-30'
        return str(report_year)+'-'+report_day
    if report_month == 9:
        report_day = '11-30'
        return str(report_year)+'-'+report_day
    if report_month == 12:
        report_year = report_year+1
        report_day = '04-30'
        return str(report_year)+'-'+report_day

def get_num_pile_stocks(stock_numbers):
    share ,remain = divmod(stock_numbers, 500)
    if remain == 0:
        return share
    else:
        return share + 1 
'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788', '6048f2c5a065e754cee104b8c0edd8186f6b3d61', '94331def3e94db8a1aabb84e4257cec0ad3b576b', 'b1fe4037e43a6db43d8ac280d262a05e90bf9171',
REPORT_NAMES = ['2022_3', '2021_3', '2021_6', '2021_9', '2021_12', '2020_3', '2020_6', '2020_9', '2020_12','2019_3', '2019_6', '2019_9', '2019_12', '2018_3', '2018_6', '2018_9', '2018_12']
API_KEYS = ['cb5c4fff01f2b92088c9679b935676210916c648','d088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788', '6048f2c5a065e754cee104b8c0edd8186f6b3d61', '94331def3e94db8a1aabb84e4257cec0ad3b576b', 'b1fe4037e43a6db43d8ac280d262a05e90bf9171', 'cb5c4fff01f2b92088c9679b935676210916c648', 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788', '6048f2c5a065e754cee104b8c0edd8186f6b3d61', '94331def3e94db8a1aabb84e4257cec0ad3b576b', 'b1fe4037e43a6db43d8ac280d262a05e90bf9171', 'cb5c4fff01f2b92088c9679b935676210916c648', 'cb5c4fff01f2b92088c9679b935676210916c648', 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788', '6048f2c5a065e754cee104b8c0edd8186f6b3d61', '94331def3e94db8a1aabb84e4257cec0ad3b576b', 'b1fe4037e43a6db43d8ac280d262a05e90bf9171', 'cb5c4fff01f2b92088c9679b935676210916c648', 'cb5c4fff01f2b92088c9679b935676210916c648', 'd088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9','84acf17c852e9430843a589191330e0630cc19a7','1eca9bad2afb8becdc5b1714112dd097e73da788', '6048f2c5a065e754cee104b8c0edd8186f6b3d61', '94331def3e94db8a1aabb84e4257cec0ad3b576b', 'b1fe4037e43a6db43d8ac280d262a05e90bf9171', 'cb5c4fff01f2b92088c9679b935676210916c648', 'cb5c4fff01f2b92088c9679b935676210916c648']
REPORT_NMS = ['2022_6','2022_3']
TODAY =  date.today().strftime("%Y-%m-%d")
INDEX_NAME = 'KS11'
INDEX_NAME = 'KS11'
if INDEX_NAME == 'KS11':
    NUMBER = 0
    for REPORT_NM in REPORT_NMS:
        akaps = how_per(API_KEYS[NUMBER])
        REPORT_YEAR, REPORT_MONTH = get_report_year_month(REPORT_NM)
        REPORT_DAY = get_report_day(REPORT_YEAR, REPORT_MONTH)
        df = pd.read_csv('A_KOSPI.csv')
        STOCK_NUMBERS = len(df.index)
        '''akaps.how_report(REPORT_NM, TODAY, 'A_KOSPI.csv', 'A_PER_KOSPI('+REPORT_NM+').csv',0,STOCK_NUMBERS)
        akaps.erase_nan('A_PER_KOSPI('+REPORT_NM+').csv','AN_PER_KOSPI('+REPORT_NM+').csv')
        akaps.find_suitable_stock('AN_PER_KOSPI('+REPORT_NM+').csv','ANA_PER_KOSPI('+REPORT_NM+').csv')
        akaps.get_per('ANA_PER_KOSPI('+REPORT_NM+').csv','ANAL_PER_KOSPI('+REPORT_NM+').csv', INDEX_NAME)'''
        #akaps.find_suitable_stock_before('ANAL_PER_KOSPI('+REPORT_NM+').csv', 'ANALY_PER_KOSPI('+REPORT_NM+').csv')
        akaps.find_suitable_stock('ANAL_PER_KOSPI('+REPORT_NM+').csv','ANALY_PER_KOSPI('+REPORT_NM+').csv')
        akaps.sort_values_descendings('ANALY_PER_KOSPI('+REPORT_NM+').csv', 'ANALYS_PER_KOSPI('+REPORT_NM+').csv', ['Recent_Market_Gun'])
        akaps.sort_values_ascendings('ANALYS_PER_KOSPI('+REPORT_NM+').csv', 'ANALYSE_PER_KOSPI('+REPORT_NM+').csv', ['Revenue_Change_Now', 'Operating_Profit_Change_Now'])
        akaps.find_suitable_stock_later('ANALYSE_PER_KOSPI('+REPORT_NM+').csv','ANALYSES_PER_KOSPI('+REPORT_NM+').csv')
        #akaps.how_much_earn('ANALY_PER_KOSPI('+REPORT_NM+').csv','ANALYS_PER_KOSPI('+REPORT_NM+').csv')
        NUMBER = NUMBER + 1
        
INDEX_NAME = 'KQ11'        
REPORT_NMS = ['2022_6']
if INDEX_NAME == 'KQ11':
    NUMBER = 0
    for REPORT_NM in REPORT_NMS:
        akaps = how_per(API_KEYS[NUMBER])
        REPORT_YEAR, REPORT_MONTH = get_report_year_month(REPORT_NM)
        REPORT_DAY = get_report_day(REPORT_YEAR, REPORT_MONTH)
        df = pd.read_csv('A_KOSDAQ.csv')
        STOCK_NUMBERS = len(df.index)
        NUMS = get_num_pile_stocks(STOCK_NUMBERS)
        '''for NUM in range(NUMS):
            if (NUM+1)*500 <= STOCK_NUMBERS:
                akaps.how_report(REPORT_NM, TODAY, 'A_KOSDAQ.csv', 'A_PER_KOSDAQ('+REPORT_NM+')'+str(NUM)+'.csv',(NUM)*500, (NUM+1)*500)
            else:
                akaps.how_report(REPORT_NM, TODAY, 'A_KOSDAQ.csv', 'A_PER_KOSDAQ('+REPORT_NM+')'+str(NUM)+'.csv',(NUM)*500, STOCK_NUMBERS)
        
        kosdaq_data0 = pd.read_csv('A_PER_KOSDAQ('+REPORT_NM+')0.csv')
        kosdaq_data1 = pd.read_csv('A_PER_KOSDAQ('+REPORT_NM+')1.csv')
        kosdaq_data2 = pd.read_csv('A_PER_KOSDAQ('+REPORT_NM+')2.csv')
        result = kosdaq_data2
        #result = pd.concat(frames)
        result.drop_duplicates(subset = ['Symbol'], inplace = True)
        result.reset_index(drop=True, inplace = True)
        result.to_csv('A_PER_KOSDAQ('+REPORT_NM+').csv')
        
        akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
        akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
        akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)'''
        #akaps.find_suitable_stock_before('ANAL_PER_KOSDAQ('+REPORT_NM+').csv', 'ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
        akaps.find_suitable_stock('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
        akaps.sort_values_descendings('ANALY_PER_KOSDAQ('+REPORT_NM+').csv', 'ANALYS_PER_KOSDAQ('+REPORT_NM+').csv', ['Recent_Market_Gun'])
        akaps.sort_values_ascendings('ANALYS_PER_KOSDAQ('+REPORT_NM+').csv', 'ANALYSE_PER_KOSDAQ('+REPORT_NM+').csv', ['Revenue_Change_Now', 'Operating_Profit_Change_Now'])
        akaps.find_suitable_stock_later('ANALYSE_PER_KOSDAQ('+REPORT_NM+').csv','ANALYSES_PER_KOSDAQ('+REPORT_NM+').csv')
        #akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')
        #akaps.erase_nan_before('ANALYS_PER_KOSDAQ('+REPORT_NM+').csv','ANALYSE_PER_KOSDAQ('+REPORT_NM+').csv')
        #akaps.find_suitable_stock_before('ANALYSE_PER_KOSDAQ('+REPORT_NM+').csv', 'ANALYSEA_PER_KOSDAQ('+REPORT_NM+').csv')'''
        NUMBER = NUMBER + 1
        
INDEX_NAME = 'KK'
REPORT_NMS = []
if INDEX_NAME == 'KQ11':
    NUMBER = 1
    for REPORT_NM in REPORT_NMS:
        akaps = how_per(API_KEYS[NUMBER])
        REPORT_YEAR, REPORT_MONTH = get_report_year_month(REPORT_NM)
        '''REPORT_DAY = get_report_day(REPORT_YEAR, REPORT_MONTH)
        #akaps.how_report(REPORT_DAY, 'ANALYS_PER_KOSDAQ(2021_9).csv', 'ANALYSE_PER_KOSDAQ(2021_9).csv',0,56)
        akaps.how_report(REPORT_NM, '2022-03-18', 'A_KOSDAQ.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',500, 1000)
        akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
        akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
        akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
        akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
        akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')
        #akaps.erase_nan_before('ANALYS_PER_KOSDAQ('+REPORT_NM+').csv','ANALYSE_PER_KOSDAQ('+REPORT_NM+').csv')
        #akaps.find_suitable_stock_before('ANALYSE_PER_KOSDAQ('+REPORT_NM+').csv', 'ANALYSEA_PER_KOSDAQ('+REPORT_NM+').csv')'''
        
        
'''INDEX_NAME = 'KQ11'
REPORT_NMS = ['2021_12']
if INDEX_NAME == 'KQ11':
    NUMBER = 2
    for REPORT_NM in REPORT_NMS:
        akaps = how_per(API_KEYS[NUMBER])
        REPORT_YEAR, REPORT_MONTH = get_report_year_month(REPORT_NM)
        REPORT_DAY = get_report_day(REPORT_YEAR, REPORT_MONTH)
        #akaps.how_report(REPORT_DAY, 'ANALYS_PER_KOSDAQ(2021_9).csv', 'ANALYSE_PER_KOSDAQ(2021_9).csv',0,56)
        akaps.how_report(REPORT_NM, '2022-03-18', 'A_KOSDAQ.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',1000, 1260)
        akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
        akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
        akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
        akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
        akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')
        #akaps.erase_nan_before('ANALYS_PER_KOSDAQ('+REPORT_NM+').csv','ANALYSE_PER_KOSDAQ('+REPORT_NM+').csv')
        #akaps.find_suitable_stock_before('ANALYSE_PER_KOSDAQ('+REPORT_NM+').csv', 'ANALYSEA_PER_KOSDAQ('+REPORT_NM+').csv')'''
            
'''akaps = how_per('d088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9')
REPORT_NM = '2018_9'
akaps.how_report('2017-04-30','A_KOSPI.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',0,682)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')

akaps = how_per('d088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9')
REPORT_NM = '2018_3'
akaps.how_report('2017-04-30','A_KOSPI.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',0,682)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')'''

'''INDEX_NAME = 'KQ11'
akaps = how_per('b1fe4037e43a6db43d8ac280d262a05e90bf9171')
REPORT_NM = '2020_3'
akaps.how_report('2018-04-30','A_KOSPI.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',0,682)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')

akaps = how_per('84acf17c852e9430843a589191330e0630cc19a7')
REPORT_NM = '2019_6'
akaps.how_report('2016-11-30','A_KOSPI.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',0,682)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')

akaps = how_per('1eca9bad2afb8becdc5b1714112dd097e73da788')
REPORT_NM = '2019_3'
akaps.how_report('2016-08-30','A_KOSPI.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',0,682)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')

akaps = how_per('6048f2c5a065e754cee104b8c0edd8186f6b3d61')
REPORT_NM = '2017_9'
akaps.how_report('2017-05-30','A_KOSPI.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',0,682)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')

akaps = how_per('94331def3e94db8a1aabb84e4257cec0ad3b576b')
REPORT_NM = '2017_6'
akaps.how_report('2016-05-30','A_KOSPI.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',0,682)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')
                                                                              
akaps = how_per('d088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9')
REPORT_NM = '2018_12'
akaps.how_report('2017-04-30','A_KOSPI.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',0,682)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')

akaps = how_per('d088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9')
REPORT_NM = '2018_9'
akaps.how_report('2017-04-30','A_KOSPI.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',0,682)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')

akaps = how_per('d088ee9b7af573cb92b0e4ea9eca9d5eff37d6a9')
REPORT_NM = '2018_3'
akaps.how_report('2017-04-30','A_KOSPI.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',0,682)
akaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)
akaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')
akaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')'''

"INDEX_NAME = 'KQ11'\nakaps = how_per('b1fe4037e43a6db43d8ac280d262a05e90bf9171')\nREPORT_NM = '2020_3'\nakaps.how_report('2018-04-30','A_KOSPI.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',0,682)\nakaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')\nakaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+').csv','ANA_PER_KOSDAQ('+REPORT_NM+').csv')\nakaps.get_per('ANA_PER_KOSDAQ('+REPORT_NM+').csv','ANAL_PER_KOSDAQ('+REPORT_NM+').csv', INDEX_NAME)\nakaps.find_suitable_stock_later('ANAL_PER_KOSDAQ('+REPORT_NM+').csv','ANALY_PER_KOSDAQ('+REPORT_NM+').csv')\nakaps.how_much_earn('ANALY_PER_KOSDAQ('+REPORT_NM+').csv','ANALYS_PER_KOSDAQ('+REPORT_NM+').csv')\n\nakaps = how_per('84acf17c852e9430843a589191330e0630cc19a7')\nREPORT_NM = '2019_6'\nakaps.how_report('2016-11-30','A_KOSPI.csv', 'A_PER_KOSDAQ('+REPORT_NM+').csv',0,682)\nakaps.erase_nan('A_PER_KOSDAQ('+REPORT_NM+').csv','AN_PER_KOSDAQ('+REPORT_NM+').csv')\nakaps.find_suitable_stock('AN_PER_KOSDAQ('+REPORT_NM+'

In [ ]:
import pandas as pd
df = pd.read_csv('A_PER_KOSDAQ(2021_9).csv')
df_new = df.rename(columns={'Quarter_Full_EPS2021_6': 'Quarter_Full_EPS'})
df_new.to_csv('A_PER_KOSDAQ(2021_9).csv')

In [10]:
for i in range(4):
    print(i)

0
1
2
3


In [ ]:
#drop that is not nan
import pandas as pd
data = pd.read_csv('A_PER_KOSPI(2021_9).csv')
data = data[data['Report_Date'].isnull()]
data.reset_index(drop=True, inplace = True)
data.drop(['Unnamed: 0'], axis = 1, inplace = True)
data.to_csv('AN_PER_KOSPI(2021_9).csv')

In [5]:
#drop that is not nan
import pandas as pd
data = pd.read_csv('KOSDAQ.csv')
data = data[~data.Sector.str.contains("신탁업 및 집합투자업", na=False)]
data = data[~data.Sector.str.contains("금융 지원 서비스업", na=False)]
data.reset_index(drop=True, inplace = True)
data.drop(['Unnamed: 0'], axis = 1, inplace = True)
data.to_csv('A_KOSDAQ.csv')

In [ ]:
#erase row that has nan
import pandas as pd
df = pd.read_csv('A_PER_KOSDAQ_real(2021_3).csv')
df.dropna(subset = ['Report_Date'], inplace = True)
df.reset_index(drop=True, inplace = True)
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.to_csv('AN_PER_KOSDAQ(2021_3).csv')

In [ ]:
#compare quarter revenue and quarter operating profit
import numpy as np
import datetime
import pandas as pd
import re
def get_report_year_month(report_nm):
    delimiters = "(", "_", ")"
    regexPattern = '|'.join(map(re.escape, delimiters))
    nm_list = re.split(regexPattern, report_nm)
    nm_num_list = [int(s) for s in nm_list if s.isdigit()]
    report_year = int(nm_num_list[0])
    report_month = int(nm_num_list[1])
    return report_year, report_month

def get_report_year_month_one_back(report_year, report_month):
    if report_month == 3:
        report_year = report_year - 1
        report_month = 12
        return report_year, report_month
    if report_month == 6:
        report_year = report_year
        report_month = 3
        return report_year, report_month
    if report_month == 9:
        report_year = report_year
        report_month = 6
        return report_year, report_month
    if report_month == 12:
        report_year = report_year
        report_month = 9
        return report_year, report_month
    
REPORT_NM = 'AN_PER_KOSDAQ(2021_6).csv'
kosdaq_data = pd.read_csv(REPORT_NM)
REPORT_YEAR, REPORT_MONTH = get_report_year_month(REPORT_NM)
BACK_REPORT_YEAR, BACK_REPORT_MONTH = get_report_year_month_one_back(REPORT_YEAR, REPORT_MONTH)
kosdaq_data['ListingDate'] = pd.to_datetime(kosdaq_data['ListingDate'], format='%Y-%m-%d')
'''kosdaq_data['Quarter_Percent_Operating_Profit'] = kosdaq_data['Quarter_Percent_Operating_Profit'].astype(float)'''
kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
'''kosdaq_data['Quarter_Cost_Of_Sale'] = kosdaq_data['Quarter_Revenue'] - kosdaq_data['Quarter_Operating_Profit']
kosdaq_data['Last_Cost_Of_Sale'] = kosdaq_data['Last_REVENUE'] - kosdaq_data['Last_Operating_Profit']
kosdaq_data['Quarter_Loss'] = (kosdaq_data['Quarter_Cost_Of_Sale']/kosdaq_data['Quarter_Revenue'])*100
kosdaq_data['Last_Loss'] = (kosdaq_data['Last_Cost_Of_Sale']/kosdaq_data['Last_REVENUE'])*100'''
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Revenue'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]*1.2)]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Revenue'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data['Last_Revenue'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]*1.2)]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > (kosdaq_data['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)] + 10)]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > (kosdaq_data['Last_Percent_Operating_Profit'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)] + 10)]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -20]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -20]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -10]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -10]
'''kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > kosdaq_data['Last_Percent_Earn'+str(BACK_REPORT_YEAR-1)+'_'+str(BACK_REPORT_MONTH)]]'''
#kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_6'] > -10000000000]
#kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_3'] > -10000000000]
'''kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] > -10]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn'+str(BACK_REPORT_YEAR)+'_'+str(BACK_REPORT_MONTH)] > -10]'''
kosdaq_data = kosdaq_data[~kosdaq_data.Sector.str.contains("의료용품 및 기타 의약 관련제품 제조업", na=False)]
kosdaq_data = kosdaq_data[~kosdaq_data.Sector.str.contains("기초 의약물질 및 생물학적 제제 제조업", na=False)]
kosdaq_data = kosdaq_data[~kosdaq_data.Sector.str.contains("의료용 기기 제조업", na=False)]
kosdaq_data = kosdaq_data[~kosdaq_data.Sector.str.contains("자연과학 및 공학 연구개발업", na=False)]
kosdaq_data['Quarter_Percent_Operating_Profit_Difference'] = kosdaq_data['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] - kosdaq_data['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]
kosdaq_data.sort_values(by='Quarter_Percent_Operating_Profit_Difference', ascending=False, inplace = True)
#kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'] > -50]
kosdaq_data.reset_index(drop=True, inplace = True)
kosdaq_data.to_csv('ANA_PER_KOSDAQ('+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)+').csv')

In [ ]:
#compare quarter revenue and quarter operating profit
import numpy as np
import datetime
import pandas as pd
import re
kosdaq_data['Quarter_Percent_Operating_Profit_Difference'] = kosdaq_data['Quarter_Percent_Operating_Profit'+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)] - kosdaq_data['Last_Percent_Operating_Profit'+str(REPORT_YEAR-1)+'_'+str(REPORT_MONTH)]
kosdaq_data.sort_values(by='Quarter_Percent_Operating_Profit_Difference', ascending=False, inplace = True)
#kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'] > -50]
kosdaq_data.reset_index(drop=True, inplace = True)
kosdaq_data.to_csv('ANA_PER_KOSDAQ('+str(REPORT_YEAR)+'_'+str(REPORT_MONTH)+').csv')

In [ ]:
#compare quarter revenue and quarter operating profit
import numpy as np
import datetime
import pandas as pd
kosdaq_data = pd.read_csv('AN_PER_KOSDAQ(2021_3).csv')
kosdaq_data['ListingDate'] = pd.to_datetime(kosdaq_data['ListingDate'], format='%Y-%m-%d')
'''kosdaq_data['Quarter_Percent_Operating_Profit'] = kosdaq_data['Quarter_Percent_Operating_Profit'].astype(float)
kosdaq_data['Last_Percent_Operating_Profit'] = kosdaq_data['Last_Percent_Operating_Profit'].astype(float)
kosdaq_data['Quarter_Revenue'] = kosdaq_data['Quarter_Revenue'].astype(float)
kosdaq_data['Last_Revenue'] = kosdaq_data['Last_Revenue'].astype(float)
#kosdaq_data['Quarter_Per'] = kosdaq_data['Quarter_Per'].astype(float)
#kosdaq_data['Last_Per'] = kosdaq_data['Last_Per'].astype(float)'''
kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
'''kosdaq_data['Quarter_Cost_Of_Sale'] = kosdaq_data['Quarter_Revenue'] - kosdaq_data['Quarter_Operating_Profit']
kosdaq_data['Last_Cost_Of_Sale'] = kosdaq_data['Last_REVENUE'] - kosdaq_data['Last_Operating_Profit']
kosdaq_data['Quarter_Loss'] = (kosdaq_data['Quarter_Cost_Of_Sale']/kosdaq_data['Quarter_Revenue'])*100
kosdaq_data['Last_Loss'] = (kosdaq_data['Last_Cost_Of_Sale']/kosdaq_data['Last_REVENUE'])*100'''
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Revenue2021_3'] > (kosdaq_data['Last_Revenue2020_3']*1.5)]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Revenue2020_6'] > (kosdaq_data['Last_Revenue2020_6']*1.5)]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit2021_9'] > (kosdaq_data['Last_Percent_Operating_Profit2020_9'] + 15)]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit2021_6'] > (kosdaq_data['Last_Percent_Operating_Profit2020_6'] + 15)]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit2021_9'] > 10]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit2021_6'] > -10]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_9'] > kosdaq_data['Last_Percent_Earn2020_9']]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_6'] > kosdaq_data['Last_Percent_Earn2020_6']]
#kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_6'] > -10000000000]
#kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_3'] > -10000000000]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_9'] > -10]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Earn2021_6'] > -10]
#kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Percent_Operating_Profit'] > -50]
kosdaq_data.reset_index(drop=True, inplace = True)
kosdaq_data.to_csv('ANA_PER_KOSDAQ(2021_9).csv')

In [ ]:
#get data of only name and boom date and how big
import numpy as np
import datetime
import pandas as pd
kosdaq_data = pd.read_csv('ANA_PER_KOSDAQ(2).csv')
kosdaq_data['ListingDate'] = pd.to_datetime(kosdaq_data['ListingDate'], format='%Y-%m-%d')
kosdaq_data['Quarter_Earn'] = kosdaq_data['Quarter_Earn'].astype(float)
kosdaq_data['Last_Earn'] = kosdaq_data['Last_Earn'].astype(float)
kosdaq_data['Quarter_Per'] = kosdaq_data['Quarter_Per'].astype(float)
kosdaq_data['Last_Per'] = kosdaq_data['Last_Per'].astype(float)
kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Earn'] > 1.5*kosdaq_data['Last_Earn'] ]
kosdaq_data = kosdaq_data[2*kosdaq_data['Quarter_Per'] < kosdaq_data['Last_Per']]
#kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Per'] > kosdaq_data['Last_Per']]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Per'] > 0]
anal_kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Per'] < 15]
anal_kosdaq_data = anal_kosdaq_data[anal_kosdaq_data['Quarter_Operating_Profit']>anal_kosdaq_data['Quarter_Earn']]
anal_kosdaq_data = anal_kosdaq_data[anal_kosdaq_data['Quarter_Operating_Profit']>(2*anal_kosdaq_data['Last_Operating_Profit'])]
#abs는 절댓값
#anal_kosdaq_data = anal_kosdaq_data[anal_kosdaq_data['Quarter_Earn']> 2*(abs(anal_kosdaq_data['Last_Earn']))]
#anal_kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Per'] < 15]
anal_kosdaq_data.reset_index(drop=True, inplace = True)
anal_kosdaq_data.to_csv('ANAL_PER_KOSDAQ(8).csv')

In [ ]:
#get data that earns best
import numpy as np
import datetime
import pandas as pd
kosdaq_data = pd.read_csv('ANA_PER_KOSDAQ(2020_6_12).csv')
kosdaq_data['ListingDate'] = pd.to_datetime(kosdaq_data['ListingDate'], format='%Y-%m-%d')
kosdaq_data['Quarter_Stock_Price'] = kosdaq_data['Quarter_Stock_Price'].astype(float)
kosdaq_data['Quarter_Stock_Price_Sell'] = kosdaq_data['Quarter_Stock_Price_Sell'].astype(float)
#kosdaq_data['Quarter_Per'] = kosdaq_data['Quarter_Per'].astype(float)
#kosdaq_data['Last_Per'] = kosdaq_data['Last_Per'].astype(float)
kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Stock_Price_Sell'] > 2*kosdaq_data['Quarter_Stock_Price']]
kosdaq_data['Stock_Growth'] = (kosdaq_data['Quarter_Stock_Price_Sell']/kosdaq_data['Quarter_Stock_Price']*100)-100
kosdaq_data['Stock_Growth'] = kosdaq_data['Stock_Growth'].astype(int)
kosdaq_data.reset_index(drop=True, inplace = True)
kosdaq_data.to_csv('ANA_PER_KOSDAQ(2020_6_12)_test.csv')

In [ ]:
#check if my condition fits
import numpy as np
import datetime
import pandas as pd
kosdaq_data = pd.read_csv('ANA_PER_KOSDAQ(2020_6_12).csv')
kosdaq_data['ListingDate'] = pd.to_datetime(kosdaq_data['ListingDate'], format='%Y-%m-%d')
kosdaq_data['Last_Per'] = kosdaq_data['Last_Per'].astype(float)
kosdaq_data['Quarter_Per'] = kosdaq_data['Quarter_Per'].astype(float)
kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
kosdaq_data = kosdaq_data[kosdaq_data['Last_Per'] > 0]
#kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Per'] < kosdaq_data['Last_Per']/3]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Earn_Per_Stock'] > *(abs(kosdaq_data['Last_Earn_Per_Stock']))]
#kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Operating_Profit']>(1.5*(abs(kosdaq_data['Last_Operating_Profit'])))]
#kosdaq_data = kosdaq_data[50 > abs(kosdaq_data['Last_Per'])]
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Per'] > 0]
anal_kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Per'] < 20]
anal_kosdaq_data.reset_index(drop=True, inplace = True)
anal_kosdaq_data.to_csv('ANA_PER_KOSDAQ(2020_6_12)_test.csv')

In [ ]:
#check if my condition fits
import numpy as np
import datetime
import pandas as pd
kosdaq_data = pd.read_csv('AN_PER_KOSDAQ(2020_6).csv')
kosdaq_data['ListingDate'] = pd.to_datetime(kosdaq_data['ListingDate'], format='%Y-%m-%d')
kosdaq_data['Last_Per'] = kosdaq_data['Last_Per'].astype(float)
kosdaq_data['Quarter_Per'] = kosdaq_data['Quarter_Per'].astype(float)
kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Earn_Per_Stock'] > *(abs(kosdaq_data['Last_Earn_Per_Stock']))]

In [ ]:
import numpy as np
import datetime
import pandas as pd
kosdaq_data = pd.read_csv('ANA_PER_KOSDAQ.csv')
kosdaq_data['Quarter_Per'] = kosdaq_data['Quarter_Per'].astype(float)
kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
anal_kosdaq_data = kosdaq_data[kosdaq_data['Quarter_Per'] < 15]
anal_kosdaq_data.reset_index(drop=True, inplace = True)
anal_kosdaq_data.to_csv('ANA_PER_KOSDAQ.csv')

In [ ]:
#drop duplicates
import pandas as pd
ANALY_KOSDAQ = pd.read_csv('ANALY_KOSDAQ(7).csv')
ANALY_KOSDAQ.drop_duplicates(subset=['Symbol', 'Boom_Date'], inplace = True)
ANALY_KOSDAQ.reset_index(drop = True, inplace = True)
ANALY_KOSDAQ.drop(['Unnamed: 0'], axis = 1, inplace = True)
ANALY_KOSDAQ.to_csv('ANALY_KOSDAQ(7).csv')

In [ ]:
def a():
    try:
        ldf
    except:
        goto exceed;
    exceed:
        print('True')
a()

In [ ]:
import math
x = float('nan')
if math.isnan(x):
    print('yes')

In [ ]:
import re
import pandas as pd
import OpenDartReader
import datetime as dt

# ==== 0. 객체 생성 ====
# 객체 생성 (API KEY 지정) 
api_key = '84acf17c852e9430843a589191330e0630cc19a7'
dart = OpenDartReader(api_key) 

html_text = dart.document('20211102900491')
DF_LIST = pd.read_html(html_text)
if DF_LIST[0][0].str.contains('※ 동 정보는 확정치가 아닌 잠정치로서 향후 확정치와는 다를 수 있음.').any():
    print(True)
DF_LIST[0].set_index([0,1], inplace = True)
DF_LIST[0].loc[('당기순이익','당해실적'), 2].item()
#기업의 개황정보

In [ ]:
import re
import pandas as pd
import OpenDartReader
import datetime as dt
import numpy as np

# ==== 0. 객체 생성 ====
# 객체 생성 (API KEY 지정) 
api_key = '84acf17c852e9430843a589191330e0630cc19a7'
dart = OpenDartReader(api_key)

def check_empty(df):
    return df.empty
code_list = ['11013', '11012', '11014', '11011']
report = dart.finstate_all('054620','2018', reprt_code='11011', fs_div='CFS')
#report.to_csv('test(4).csv')
#report = dart.report('112040', '소액주주', '2021', reprt_code='11014')
#report = report[report['stock_knd'].str.contains('보통주', regex = False)]
report.to_csv('test(6).csv')
'''if report['stock_knd'].str.contains('-', regex = False).any():
    report = report[report['stock_knd'].str.contains('-', regex = False)]
print(report)
report.reset_index(drop = True, inplace = True)
#report.to_csv('test.csv')
#report.set_index(['sj_div','account_id'], inplace = True)
#report = report[['thstrm_amount']]
#report_nan_list = report[report['thstrm_amount'].notna()].index.tolist()
#report_nan_list = [index for index, row in report.iterrows() if row.isnull().any()]
#report_nan_list = report[report[report['thstrm_amount'] == ''].index]
#report_nan_list = np.where(report.applymap(lambda x: x == ''))
#report_nan_list = report.index[report['thstrm_amount'] == ''].tolist()
#print(report_nan_list)
if ('BS', 'dart_LongTermTradeAndOtherNonCurrentPayables') in report_nan_list:
    print('true')   
                #print(quarter_earn, quarter_earn_per_stock)
#report.to_csv('test.csv')
#'ifrs-full_ProfitLossAttributableToOwnersOfParent'
#'ifrs-full_ProfitLossFromContinuingOperations'
#if report.index.isin([('CIS', 'ifrs-full_ProfitLossFromContinuingOperations')]).any():
    #print('true')

        #quarter_earn = report.loc[('CIS','ifrs-full_ProfitLossFromContinuingOperations'), 'thstrm_amount']
        #quarter_earn_per_stock = report.loc[('CIS','ifrs-full_BasicEarningsLossPerShareFromContinuingOperations'), 'thstrm_amount']
        #print(quarter_earn, quarter_earn_per_stock)
        #print(int(quarter_earn), int(quarter_earn_per_stock))
#if not check_empty(report):
    #print('true')
#report.set_index(['sj_div','account_id'], inplace = True)
#if report.index.isin([('IS', 'ifrs-full_ProfitLoss')]).any():
    #print('true')
#report.account_nm = report.account_nm.str.replace(' ', '')
#if report['account_nm'].str.contains('계속영업분기순').any():
    #print('True')'''

In [ ]:
import re
import pandas as pd
report = pd.read_csv('test.csv')
report = report[report['stock_knd'].str.contains('보통주', regex = False)]
report = report[report['nm'].str.contains('계', regex = False)]


In [ ]:
import datetime as dt
Boom_Date = '20210514'
Boom_Date = dt.datetime.strptime(Boom_Date, "%Y%m%d")
Boom_Date

In [ ]:
import pandas as pd
import datetime as dt
A_S_LIST = pd.read_csv('정보들.csv')
Report_DT = dt.date(2021,7,1)
A_S_LIST = A_S_LIST[A_S_LIST['report_nm'].str.contains('연결재무제표기준영업', regex = False)]
A_S_LIST['rcept_dt'] = pd.to_datetime(A_S_LIST['rcept_dt'], format = '%Y%m%d').dt.date
A_S_LIST = A_S_LIST[A_S_LIST['rcept_dt'] > Report_DT]
A_S_LIST.drop(['Unnamed: 0'], axis =1, inplace = True)
A_S_LIST.reset_index(drop = True, inplace = True)
A_S_LIST

In [ ]:
 #get number data only
    txt = "h3110 23 cat 444.4 rabbit 11 2 dog"
    [int(s) for s in txt.split() if s.isdigit()]
    [23, 11, 2]
    
    #get pandas value by name
    df.index[df['BoolCol'] == True].tolist()
    
    In [56]: idx = df.index[df['BoolCol']]

    In [57]: idx
    Out[57]: Int64Index([10, 40, 50], dtype='int64')
    
    In [58]: df.loc[idx]
    Out[58]: 
           BoolCol
    10    True
    40    True
    50    True

    [3 rows x 1 columns]

In [ ]:
#check recent 2months per of the boom_time and compare check when lower(test)
#deficit to surplus means it will be profitable and check if this continues it will be better and sell when they anounce how the earned
#compare with last year if double it worths
import pandas as pd
import OpenDartReader
# ==== 0. 객체 생성 ====
# 객체 생성 (API KEY 지정) 
api_key = '84acf17c852e9430843a589191330e0630cc19a7'

dart = OpenDartReader(api_key) 
#buy on the time boom and sell when earn reveals and see how much it worths
if True:
    s_list = dart.list('112040', start='2005-08-11', end='2021-08-11', kind = 'A')
    print(type(s_list))
    s_list.to_csv('정보들(2).csv')

In [ ]:
import OpenDartReader
import pandas as pd

# ==== 0. 객체 생성 ====
# 객체 생성 (API KEY 지정) 
api_key = '84acf17c852e9430843a589191330e0630cc19a7'
dart = OpenDartReader(api_key) 

html_text = dart.document('20211102900491')
df_list = pd.read_html(html_text)
for i in df_list:
    print(i.loc[0,0])

In [ ]:
import OpenDartReader
import pandas as pd

# ==== 0. 객체 생성 ====
# 객체 생성 (API KEY 지정) 
api_key = '84acf17c852e9430843a589191330e0630cc19a7'
dart = OpenDartReader(api_key) 

html_text = dart.document('20210506900476')
df_list = pd.read_html(html_text)
df_list[3].to_csv('동사.csv')

In [ ]:
txt = '분기보고서 (-2021.03)'
import re
delimiters = "(", ".", ")"
regexPattern = '|'.join(map(re.escape, delimiters))
print(regexPattern)
txt_list = re.split(regexPattern, txt)
[int(s) for s in txt_list if s.isdigit()]
#list 로 나오면 negetive 로

In [ ]:
import re
s= '분기보고서 (-2021.03)'
result = [int(d) for d in re.findall(r'-?\d+', s)]
result

In [ ]:
#get firm_code on google_trends
import numpy as np
import datetime
import pandas as pd
import pytrends
import time
from pytrends.request import TrendReq
from pytrends.exceptions import ResponseError
pytrends = TrendReq(hl='en-US', tz=360)

keyword = '/m/04l4d7'
def _fetch_data(trendreq, kw_list, timeframe='2016-12-30 2021-12-20', cat=0, geo='', gprop='') -> pd.DataFrame:
    
    """Download google trends data using pytrends TrendReq and retries in case of a ResponseError."""
    attempts, fetched = 0, False
    while not fetched:
        try:
            trendreq.build_payload(kw_list=kw_list, timeframe=timeframe, cat=cat, geo=geo, gprop=gprop)
        except ResponseError as err:
            print(err)
            print(f'Trying again in {60 + 5 * attempts} seconds.')
            time.sleep(60 + 5 * attempts)
            attempts += 1
            if attempts > 1:
                print('Failed after 3 attemps, abort fetching.')
                break
        else:
            fetched = True
    return trendreq.interest_over_time()
keyword_trend = _fetch_data(pytrends, [keyword])
keyword_trend